## Web Search and Scrape
The **Web Search and Scrape** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.8.6"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.8.6

## Ensure the Web Search and Scrape assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search and Scrape" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search and Scrape" contains the following files:
```
│   HtmlAgilityPack.dll - Loaded by the notebook, a great open library for c# web scraping
│   HtmlAgilityPackExtensions.dll - Loaded by the notebook, part of this project to do a simple conversion to markdown for less tokens and noise
│   HtmlAgilityPackExtensions.pdb - If you want to debug
│   instructions.md - The prompt
│   manifest.json - The assistant definition
│   
└───OpenAPI
        api.bing.microsoft.com.json - Azure Web Search (crawl operation)
        auth.json - Auth setup for web search
        tool.localhost.json - Local HtmlAgility.HtmlAgilityPackExtensions.ConvertUrlToMarkdown c# function 
```

## Crawl versus ConvertUrlToMarkdown tools

Crawl returns search results with snippets and only has a small summary or excerpt of the page

ConvertUrlToMarkdown fetches the whole page and turns it all into markdown for consumption

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search and Scrape", config);
Console.WriteLine(assistantId)

asst_wFcK9MyszkzLa8MZp6WtaAqX


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown")


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.now().strftime("%Y-%m-%d")
today_date
```

I called the tool named crawl with these arguments:
```
{
  "q": "things to do in Atlanta on 2024-09-24",
  "textFormat": "HTML"
}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-09-24"},"webPages":{"value":[{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. ... Sep 24, 2024 6:00 PM . Edwin Raphael . Upcoming Events: 2. Altar at Masquerade - GA ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4712253893443652\u0026mkt=en-US\u0026setlang=en-US\u0026w=-nsAOlbynJFe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Venture Atlanta 2024 10/8/24 \u2013 10/9/24. Atlanta miniFEST 10/9/24 \u2013 10/10/24. Homeshake ... Every Saturday until 09/28/2024. Protect Yo Energy Con 9/23/24 \u2013 11/9/24.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Best Things to Do in Atlanta in September","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september","snippet":"Atlanta Events: September 2024 Experience the Best of Atlanta\u2019s Food \u0026amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u2019s food and history. On this tour, you\u2019ll taste eight modern and Southern foods from three outstanding locally owned food stops.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4781604725337408\u0026mkt=en-US\u0026setlang=en-US\u0026w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O","thumbnailUrl":"https://www.bing.com/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Upcoming Events - Discover Atlanta","url":"https://discoveratlanta.com/events/all/","displayUrl":"https://discoveratlanta.com/events/all","snippet":"9/22/24 \u2013 11/3/24. Pumpkin Festival. Play by Day as you enjoy family-friendly park attractions and special fall entertainment featuring some\u2026","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4829094184357962\u0026mkt=en-US\u0026setlang=en-US\u0026w=7n30JJ0KXcuUBKZlUeD3eG4MmVcBwK29"},{"name":"September events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/september/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/september","snippet":"Share this event: BUNT | Friday September 27th 2024 | District Atlanta Save this event: BUNT | Friday September 27th 2024 | District Atlanta. Just added. September MixxedFit with Jamie. Thu, Sep 26, 7:00 PM. I Am/I Can Strength and Conditioning Gym.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4895150772333840\u0026mkt=en-US\u0026setlang=en-US\u0026w=SLlGH3HuLduyDsvIkk1oKXILCVW8I5z3"},{"name":"Atlanta September 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/september","displayUrl":"https://www.experism.com/atlanta/september","snippet":"Sep 12-15, 2024 Historic Fourth Ward Park View on Map. Play Video. The Atlanta Food \u0026amp; Wine Festival highlights the best food, wine, beer \u0026amp; cocktails Atlanta has to offer. This unique event brings in 150\u002B chefs, sommeliers, mixologists and industry insiders for 5 days of mouthwatering and tempting cuisines \u0026amp; special Tasting Tents.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=4642636760352967\u0026mkt=en-US\u0026setlang=en-US\u0026w=CrMoojBamU4_Ok9mubm7deAiSBD33s7x","thumbnailUrl":"https://www.bing.com/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"April 24\u2013May 4, 2025 ... July 20\u201321, 2024 Atlanta Expo Center The Atlanta Street Wear Market, which usually happens in April, is back for a new and exciting summer installment. This time ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"101 Things To Do In Atlanta - Fall 2024 - Student Engagement","url":"https://engagement.gsu.edu/2024/09/19/101-things-to-do-in-atlanta-fall-2024/","displayUrl":"https://engagement.gsu.edu/2024/09/19/101-things-to-do-in-atlanta-fall-2024","snippet":"101 Things To Do In Atlanta - Fall 2024. 101 Things To Do In Atlanta - Fall 2024 101 Things To Do In Atlanta - Fall 2024. September 19, 2024. This list provides some unique suggestions for exploring all that our great city (and the surrounding metro area) has to offer!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-09-24\u0026d=523467767885\u0026mkt=en-US\u0026setlang=en-US\u0026w=pefDSGUqGEWH0R2Ley_9-wyaSXXJ4Vub"}]}}
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/"}
```
and got this result:
```
"Cool Things To Do in September in Atlanta - Discover Atlanta[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading# Cool Things To Do in September in Atlanta\r\n\r\nBy Mary Welch\r\n\r\nPublished August 23, 2024Last Updated September 11, 2024\r\n\r\nIt\u2019s September. In Atlanta that means it\u2019s time for festivals, costumed aliens walking down Peachtree Street and food \u2014 glorious food.\r\n\r\n## Food Extravaganza\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723648163/newAtlanta.com/ATL-food-Credit-raftermen-photography-copy/ATL-food-Credit-raftermen-photography-copy.jpg?_i=AA)\r\nTake a big bite. There is plenty to sample at the Atlanta Food \u0026#038; Wine Festival with more than 200 chefs, sommeliers and mixologists. (Photo by Raftermen Photography)The Atlanta Food \u0026amp; Wine Festival returns to Historic Old Fourth Ward Park for a four-day celebration of incomparable food, wine, beer and craft spirits. More than 200 of the most celebrated chefs, sommeliers, mixologists and industry insiders will offer bites and drinks that showcase the exceptional flavors of the region. The festival runs Sept. 12-15.\r\n\r\n## Burger Bash\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647613/newAtlanta.com/Burger-benefit-resized/Burger-benefit-resized.jpg?_i=AA)\r\nThe team from Fox Bros. BBQ takes to the grill to showcase their burger skills at the Burger Benefit Sept. 22. (Photo courtesy of NFA Burger)Some of the city\u2019s and the nation\u2019s leading chefs including personality Andrew Zimmern; Shai Livi of The Third Space; Billy Kramer of NFA Burger; and Nick Leahy of Vice Kitchen will duke it out for an all-day burger showdown at NFA Burger in Chamblee. The Burger Benefit takes place Sept. 22 and features four burger masters commanding the grill in shifts of 90 minutes. Last year the burger benefit raised more than $53,000 in support of The Giving Kitchen, an Atlanta-based nonprofit for hospitality workers. Tickets start at $30, which includes one burger and a side.\r\n\r\n## Spooky Time\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg?_i=AA)\r\nNecrotis is just one of the friendly characters waiting to greet \u2014 and scare \u2014 you at NETHERWORLD. (Photo courtesy of NETHERWORLD)As the haunt season approaches, NETHERWORLD is opening on weekends starting Sept. 20 and 21 and will continue for 34 nights of terror through September, October and November. For its 28th Season of Screams, the attraction is offering two terrifying new haunts, Wake the Dead and Mr. Grendel\u2019s Birthday Party of Horrors. Also returning is Dr. Octane\u2019s Bizarre Beverages featuring colorful, bubbling nonalcoholic drinks served in an optional collectible light-up glass. There\u2019s no shortage of Halloween things to do in Atlanta.\r\n\r\n## Harvest on the Hooch\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_790,h_1024,c_scale/v1723648296/newAtlanta.com/Harvest-on-the-hooch/Harvest-on-the-hooch.jpg?_i=AA)\r\nEnjoy getting together with friends at the ninth annual Harvest on the Hooch at Chattahoochee Nature Center. (Photo courtesy of Chattahoochee Nature Center)The Chattahoochee Nature Center is hosting the ninth annual Harvest on the Hooch Sept. 22 to raise awareness for the Nature Center\u2019s Unity Garden, which has supplied more than 50 tons of fresh produce to North Fulton Community Charities food pantry. The event will feature food tastings from local chefs, live bluegrass music, garden games, samples from local craft breweries, face painting, and meet and greets with animals.\r\n\r\n## Hispanic Heritage Month\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647690/newAtlanta.com/hispanic-festival/hispanic-festival.jpg?_i=AA)\r\nEnjoy a cultural experience Sept. 17 for the Fiestas Patrias, celebrating Mexican Independence.Plaza Fiesta on Buford Highway will host a Fiestas Patrias celebration Sept. 17. The Mexican Consulate\u2019s \u201CEl Grito \u2014 Mexican Independence Celebration\u201D has been a local favorite for more than 19 years. Last year more than 60,000 people celebrated. Plan to go and enjoy a cultural experience that embraces heritage, entertainment and community. Celebrate Hispanic Heritage Month around Atlanta.\r\n\r\n## Baldie Con\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg?_i=AA)\r\nBaldie Con is an empowerment conference for the bald community.Happening at the JW Marriott Atlanta Buckhead, Baldie Con\u0026nbsp;celebrates, empowers and unites women and children who are bald (Baldies) from alopecia, lupus, cancer or by choice. The three-day conference starts with a meet and greet on Sept. 27. There will be a Baldie Ball with an award ceremony and gala dinner on Saturday night. The weekend wraps up on Sept. 29 with a Sunday Jazz Brunch.\r\n\r\n## It\u2019s so Sweet\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg?_i=AA)\r\nAuburn Avenue is where you\u2019ll find the Sweet Auburn Music Fest.The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.\r\n\r\n## Forward Warrior\r\n\r\nThe annual Forward Warrior live art event is happening all day, 9 a.m. until 9 p.m. Saturday, on Wylie Street in Cabbagetown.\r\n\r\n## Celebrate a President\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723649158/newAtlanta.com/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb.jpg?_i=AA)\r\nThe Fox Theatre plans a musical extravaganza to celebrate President Carter\u2019s 100th birthday. (Photo courtesy of Fox Theatre)Jimmy Carter turns 100 on Oct. 1, and the Fox Theatre is celebrating in a big way. Presented by Delta Air Lines, a variety of artists including Chuck Leavell, a Georgia resident and keyboardist for the Rolling Stones, D-Nice, Eric Church, Aren Morris and the Atlanta Symphony Orchestra Chamber Chorus will take the stage Sept. 17 to present a musical extravaganza. The event is a fund-raiser for the Carter Center.\r\n\r\n## Sandy Springs Salutes Creativity\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_658,c_scale/v1723648070/newAtlanta.com/sandy-springs-arts-festival/sandy-springs-arts-festival.jpg?_i=AA)\r\nA couple takes a break from the Sandy Springs Arts Festival to enjoy people watching. (Photo courtesy of Caren West PR)The Sandy Springs Fall Arts Festival returns on Sept. 14-15, featuring more than 150 painters, photographers, sculptors, leather and metal craft persons, glass blowers, jewelers and more. Here are more September (and fall) festivals to visit.\r\n\r\nRead More* Cool Things to Do in November in Atlanta\r\n* Cool Things To Do in October in Atlanta\r\n* Cool Things To Do in August in Atlanta\r\n\r\n## Dragon Con Takes Over\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1723647734/newAtlanta.com/Media_Relations23-1024x682-1/Media_Relations23-1024x682-1.jpg?_i=AA)\r\nYou never know who you\u2019ll meet on Peachtree Street during Dragon Con. (Photo courtesy of Dragon Con Photography \u00C2\u00A9 Dragon Con, Inc.)Dragon Con, the worlds largest multimedia, popular culture convention focusing on science fiction and fantasy, gaming, comics, literature, art, music and film in the universe comes to Atlanta and, frankly, takes over the city. Walk the streets and meet thousands of costumed folks participating in the workshops, meet and greets with celebrities, the art show, and comic and pop artist alley. But, the biggest fun comes with the parade down Peachtree Street on Aug. 31. Among the celebrities appearing at Dragon Con, which takes place Aug. 29-Sept. 2, are: Gil Gerard, Erin Grey, Seth Green, Greg Grunberg, Cobie Smulders and Ming-Na Wen.\r\n\r\n## Piedmont Park Celebrates 120 Years\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_1024,c_scale/v1725563202/newAtlanta.com/P4PP-IG-Post/P4PP-IG-Post.png?_i=AA)\r\nParty like it\u2019s 1989 as Piedmont Parks celebrates its 120th birthday and the 35th anniversary of the Piedmont Park Conservatory! The nostalgia-themed party will have food from local restaurants, cocktails, a silent auction and other throwback vibes. All proceeds will support the Piedmont Park Conservatory and its ongoing work to maintain and enhance Atlanta\u2019s most visited public park. Find more information about the event here.\r\n\r\n## College Football is Back\r\n\r\nSeptember, of course, heralds the return of college football season. No matter your alma mater, there are bars all around town, which may show your team\u2019s games, which means you can cheer along with like-minded fans. Find out Where to Watch College Football in Atlanta.\r\n\r\n## Gone Fishin\u2019\r\n\r\nSummer is winding down so it\u2019s the perfect time to drop a line and enjoy a day of fishing. There are several places to fish including Jones Creek Park, Lullwater Park, the Chattahoochee River National Recreation Area, Morgan Falls Overlook Park, Sweetwater Creek State Park, Piedmont Park, Murphey Candler Park and Lake Lanier\r\n\r\n## Try an Omakase Restaurant\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647780/newAtlanta.com/Mujo-Andrew-Thomas-Lee-resized/Mujo-Andrew-Thomas-Lee-resized.jpg?_i=AA)\r\nMuj\u00C5\u008D is one of two Atlanta omakase restaurants to receive a MICHELIN star, but there are several other restaurants that offer this unique culinary experience. (Photo of Muj\u00C5\u008D by Andrew Thomas Lee)Omakase is a Japanese dining experience where guests leave the menu in the hands of the chef and receive a seasonal, creative and unforgettable dinner with a progression of small courses. Atlanta has several omakase restaurants, and two \u2014 Muj\u014D and Hayakawa \u2014 were awarded MICHELIN STARS. But Atlanta has several others that offer amazing experiences as well. Try Omakase Table (West Marietta Street); Nagomiya (West Peachtree Street); Saito (Andrew Young International Boulevard); Taka Sushi (Roswell Road); MF Sushi (North Highland Avenue); Chirori (14th Street); Umi (Peachtree Road); and Brush (Peachtree Road). Brush, in fact, just started a \u201Crestaurant-within-a restaurant\u201D concept by offering a chef-driven sushi omakase-only experience in addition to a regular dining experience.\r\n\r\n## News for Kids\r\n\r\nAtlanta\u2019s must-see attractions are stretching your summer fun \u2013 and dollars \u2013 with a September-only promotion for families. Kids free admission this September is available at most attractions and all offer exciting experiences.\u0026nbsp;Plan your Kids Free September Atlanta discovery outing today. All offers start after Labor Day (Sept. 3-30).\r\n\r\n## Yellow Daisy Festival\r\n\r\nVoted one of the top arts and crafts shows in the nation by Sunshine Artist magazine, the Yellow Daisy Festival at Stone Mountain Park draws more than 400 artists and crafters as well as food vendors, crafter demonstrations, live music and a beer garden. The festival, which takes place Sept. 5-8, is great family fun \u2014 and you can get a head start on holiday shopping.\r\n\r\n## Buckhead Arts Festival\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_819,c_scale/v1723647966/newAtlanta.com/Buckhead-arts-festival/Buckhead-arts-festival.jpg?_i=AA)\r\nThe Buckhead Fine Arts Festival is a great way to sip, snack and shop in the heart of Buckhead. (Photo courtesy of Caren West PR.)Sip, shop and stroll in the heart of Buckhead at the Buckhead Fine Arts Festival Sept. 21-22. Presented by the Atlanta Foundation for Public Spaces, the outdoor event will feature more than 100 painters, photographers, sculptors, metalworkers, glass artists, jewelers and more. Enjoy a variety of food and beverage options from vendors and merchants in Buckhead Village.\r\n\r\n## Inaugural Street Festival\r\n\r\nHosted by Blacks in Technology Foundation, the Atlanta Street Food Festival \u0026amp; Bazaar takes place Sept. 2 at Westside Park, Atlanta\u2019s largest park. The first-time festival is a family-friendly event with music, interactive demos and other entertainment. There\u2019s even a dedicated children\u2019s zone with games, activities and entertainment. Come hungry because it is a gastronomic adventure featuring an array of foods from the city\u2019s finest chefs, food trucks and local eateries. There will be everything from barbecue to gourmet burgers to vegan delicacies.\r\n\r\n## Opera in the Yards\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1723647893/newAtlanta.com/pullman-credit-Smily-Eyes-Media/pullman-credit-Smily-Eyes-Media.jpg?_i=AA)\r\nYes, the Atlanta Opera performs at Pullman Yards. (Photo by Smile Eyes Media)The Atlanta Opera Discoveries series continues the \u201CBoh\u00E8me Project\u201D with two immersive productions at Pullman Yards. Produced by\u0026nbsp;Tomer Zvulun, the project comprises repertory performances of a modern-day\u0026nbsp;\u201CLa boh\u00E8me\u201D \u2014 with the COVID-19 pandemic in place of tuberculosis \u2014 with the Broadway show it inspired,\u0026nbsp;\u201CRent,\u201D which updated Puccini\u2019s story of friendship, passion and art by setting it in the midst of the 1990\u2019s HIV/AIDS crisis. Both performed on the same set, with the action taking place in and around the seated audience, the productions will feature different casts, and musical forces and will alternate nights for most of the run.\r\n\r\n## JapanFest\r\n\r\nCelebrate 50 years of culture, friendship and cultural exchange between Japan and Georgia with JapanFest Sept. 21-22 at Gas South Convention Center in Duluth. Enjoy the mesmerizing beauty of traditional Japanese Rakugo, Kabuki, taiko drumming, martial arts demonstration and workshops in bonsai, calligraphy, ikebana flower design and Kimono fitting. Don\u2019t miss the Japanese Food Court where local restaurants will offer bento boxes, rice bowls, ramen noodles, sushi, yahisoba and shaved ice. In addition there will be three traditional Japanese craftsman giving demonstrations in stone carving, Kumihimo braided cords and tatami weaving.\r\n\r\n## PGA\u2019s TOUR Championship Returns\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1685605842/newAtlanta.com/18_TTC_0921_26792_CCp_cc_20604e9ab6/18_TTC_0921_26792_CCp_cc_20604e9ab6.jpg?_i=AA)\r\nFans of professional golf should head to East Lake Golf Club late August through Sept. 1 for the PGA TOUR Championship.East Lake Golf Club hosts the PGA\u2019s TOUR Championship Aug. 29 through Sept. 1.\r\n\r\nAlways keep an eye on our calendar of events for even more things to do in Atlanta.\r\n\r\n![](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/pixel.png)\r\nJournalist Mary Welch writes business and lifestyle stories for local and national publications.\r\n\r\n## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[AttractionsNovember in Atlanta brings popular festivals, a new Cirque du Soleil show and the beginning of holiday things to do. Cool Things to Do in November in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-november-in-atlanta/)[Things to doFrom end of summer festivals to new exhibits at the Atlanta History Center, discover cool things to do in August in Atlanta. Cool Things To Do in August in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-august-in-atlanta/)[Arts \u0026amp; CultureEnjoy all that Atlanta has to offer during the awesome month of April. Cool Things To Do in April in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-april-in-atlanta/)[Arts \u0026amp; CultureIn May, Atlantans head outdoors to enjoy the sun, flowers and the diversity of things to do. Cool Things To Do in May in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-may-in-atlanta/)[Arts \u0026amp; CultureOctober brings lots of cool things to do in Atlanta, and Halloween is just one. Cool Things To Do in October in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)[Arts \u0026amp; CultureWe\u2019ll admit it can get hot in Atlanta during July, so here are some cool things to do. Cool Things To Do in July in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-july-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/september/"}
```
and got this result:
```
"Atlanta Events September 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* September\r\n\r\n# Atlanta Events September 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices.\r\n\r\nStart DateSep 24, 2024ResetCancelApplyEnd DateSep 26, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 120* Sep 24 2024 6:00 PM Tue Signs of The Swarm Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $42 16 tickets remaining! Tickets\r\n* Sep 24 2024 7:00 PM Tue Kehlani Lakewood Amphitheatre | Capacity: 19000 30315, 2002 Lakewood Avenue Southeast, Atlanta, GA, US Prices from $51 Avg. price ~ $140 Available Tickets: 227 Tickets\r\n* Sep 24 2024 7:20 PM Tue Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $15 Avg. price ~ $96 Available Tickets: 1074 Tickets\r\n* Sep 24 2024 7:30 PM Tue Atlanta United II vs. Inter Miami II Fifth Third Stadium | Capacity: 10200 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Prices from $21 Avg. price ~ $25 6 tickets remaining! Tickets\r\n* Sep 24 2024 7:30 PM Tue Nervosa Boggs Social \u0026 Supply 30310, 1310 White St. SW, Atlanta, GA, US Prices from $73 Avg. price ~ $101 15 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue DPR Coca-Cola Roxy | Capacity: 3600 30339, 800 Battery Ave SE, Atlanta, GA, US Prices from $53 Avg. price ~ $78 21 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Victoria Canal Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $31 Avg. price ~ $36 16 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Juan Fernando Velasco City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $89 Avg. price ~ $141 50 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Sofi Tukker The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $31 Avg. price ~ $80 Available Tickets: 175 Tickets\r\n* Sep 24 2024 8:00 PM Tue Magdalena Bay Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $57 Avg. price ~ $90 18 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Rocco Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $26 Avg. price ~ $98 15 tickets remaining! Tickets\r\n* Sep 25 2024 6:30 PM Wed Lorna Shore The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $34 Avg. price ~ $148 Available Tickets: 110 Tickets\r\n* Sep 25 2024 6:30 PM Wed Sevendust The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $44 Avg. price ~ $51 20 tickets remaining! Tickets\r\n* Sep 25 2024 7:00 PM Wed Creed, 3 Doors Down \u0026 Finger Eleven Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $49 Avg. price ~ $223 Available Tickets: 289 Tickets\r\n* Sep 25 2024 7:20 PM Wed Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $8 Avg. price ~ $62 Available Tickets: 2487 Tickets\r\n* Sep 25 2024 7:30 PM Wed Kings of Leon \u0026 Phantogram State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $10 Avg. price ~ $132 Available Tickets: 978 Tickets\r\n* Sep 25 2024 8:00 PM Wed Gritz and Jelly Butter City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $74 Avg. price ~ $89 8 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed Lasso - Artist Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $58 Avg. price ~ $91 Available Tickets: 64 Tickets\r\n* Sep 25 2024 8:00 PM Wed Shaboozey Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $79 Avg. price ~ $119 27 tickets remaining! Tickets\r\n* Sep 26 2024 6:30 PM Thu Rain City Drive Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $35 Avg. price ~ $41 22 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $94 Avg. price ~ $111 8 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Mico Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $83 29 tickets remaining! Tickets\r\n* Sep 26 2024 7:20 PM Thu Atlanta Braves vs. New York Mets VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $15 Avg. price ~ $150 Available Tickets: 306 Tickets\r\n* Sep 26 2024 7:30 PM Thu Khruangbin Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $51 Avg. price ~ $131 Available Tickets: 296 Tickets\r\n* Sep 26 2024 7:30 PM Thu Disney On Ice Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $41 Avg. price ~ $93 Available Tickets: 1084 Tickets\r\n* Sep 26 2024 8:00 PM Thu The California Honeydrops Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $33 Avg. price ~ $54 25 tickets remaining! Tickets\r\n* Sep 26 2024 8:00 PM Thu Karina Rykman Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $36 Avg. price ~ $96 8 tickets remaining! Tickets\r\n* Sep 26 2024 8:00 PM Thu The Buena Vista Social Orchestra Center Stage Theatre | Capacity: 1050 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Prices from $64 Avg. price ~ $101 Available Tickets: 255 Tickets\r\n* Sep 26 2024 8:00 PM Thu Lawrence The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $29 Avg. price ~ $88 Available Tickets: 108 Tickets\r\n* Sep 26 2024 8:00 PM Thu Wasia Project Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $26 Avg. price ~ $35 16 tickets remaining! Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[Signs of the Swarm Upcoming Events: 2 Hell at The Masquerade - GA Sep 24, 2024 6:00 PM](https://atlanta-ga.events/tickets/signs-of-the-swarm/)[Kehlani Upcoming Events: 2 Lakewood Amphitheatre Sep 24, 2024 7:00 PM](https://atlanta-ga.events/tickets/kehlani/)[New York Mets Upcoming Events: 9 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/new-york-mets/)[Atlanta Braves Upcoming Events: 18 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/atlanta-braves/)[Atlanta United II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/atlanta-united-ii/)[Nervosa Upcoming Events: 2 Boggs Social \u0026 Supply Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/nervosa/)[Inter Miami II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/inter-miami-ii/)[Rocco Upcoming Events: 2 Aisle 5 Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/rocco/)[Victoria Canal Upcoming Events: 2 Vinyl - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/victoria-canal/)[Sofi Tukker Upcoming Events: 2 The Tabernacle - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/sofi-tukker/)[DPR Live Upcoming Events: 2 Coca-Cola Roxy Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/dpr-live/)[Magdalena Bay Upcoming Events: 2 Variety Playhouse Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/magdalena-bay/)[Juan Fernando Velasco Upcoming Events: 2 City Winery - Atlanta Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/juan-fernando-velasco/)[Lorna Shore Upcoming Events: 2 The Tabernacle - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/lorna-shore/)[Sevendust Upcoming Events: 2 The Eastern - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/sevendust/)[Creed Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/creed/)[3 Doors Down Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/3-doors-down/)[Finger Eleven Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/finger-eleven/)[Kings Of Leon Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/kings-of-leon/)[Phantogram Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/phantogram/)[Shaboozey Upcoming Events: 2 Variety Playhouse Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/shaboozey/)[Gritz and Jelly Butter Upcoming Events: 2 City Winery - Atlanta Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)[Lasso - Artist Upcoming Events: 2 Buckhead Theatre Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/lasso-artist/)[Rain City Drive Upcoming Events: 2 Hell at The Masquerade - GA Sep 26, 2024 6:30 PM](https://atlanta-ga.events/tickets/rain-city-drive/)[Lyfe Jennings Upcoming Events: 6 City Winery - Atlanta Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/lyfe-jennings/)[Mico Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/mico/)[Disney On Ice Upcoming Events: 24 Tulsa Expo Square - Pavilion Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/disney-on-ice/)[Khruangbin Upcoming Events: 6 Fabulous Fox Theatre - Atlanta Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/khruangbin/)[The California Honeydrops Upcoming Events: 2 Variety Playhouse Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-california-honeydrops/)[Wasia Project Upcoming Events: 2 Terminal West At King Plow Arts Center Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/wasia-project/)[Jesus Adrian Romero Upcoming Events: 2 Cobb Energy Performing Arts Centre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/jesus-adrian-romero/)[Atlanta Symphony Orchestra Upcoming Events: 6 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)[Nathalie Stutzmann Upcoming Events: 4 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/nathalie-stutzmann/)[Starletta Upcoming Events: 2 Vinyl - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/starletta/)[Lawrence - Band Upcoming Events: 2 The Tabernacle - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/lawrence-band/)[The Buena Vista Social Orchestra Upcoming Events: 2 Center Stage Theatre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-buena-vista-social-orchestra/)[Karina Rykman Upcoming Events: 2 Aisle 5 Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/karina-rykman/)[Red Lace Burlesque Show and Variety Show Upcoming Events: 4 Red Lace Burlesque - Myrtle Beach Sep 27, 2024 6:00 PM](https://atlanta-ga.events/tickets/red-lace-burlesque-show-and-variety-show/)[The Warning Upcoming Events: 2 Hell at The Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-warning/)[Coral Moons Upcoming Events: 2 Altar at Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/coral-moons/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html"}
```
and got this result:
```
"Atlanta Events - September 2024This site uses cookies. Opt out at any time. More details\r\n\r\nHome \u0026rsaquo; Things To Do \u0026rsaquo; September Events\r\n\r\n# Atlanta Events - September 2024\r\n\r\nInterested in a specific type of event? See Atlanta\u0026rsquo;s calendars of upcoming: \u0026ndash; Comedy shows \u0026ndash; Beer and wine festivals \u0026ndash; Music festivals \u0026ndash; Musicals \u0026ndash; Food festivals\r\n\r\nSee also: \u0026ndash; Atlanta\u0026rsquo;s festival calendar \u0026ndash; Atlanta weather in September \u0026ndash; Things to do in Atlanta \u0026ndash; Best time of year to visit Atlanta\r\n\r\n!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.\r\n\r\n## Events From Labor Day Weekend To The Weekend Of September 7-8 2024\r\n\r\n\u0026rarr; See August events\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Wild With Happy, August 16 - September 15 2024Play at Horizon Theatre. More details\r\n\r\n\u0026ndash; Atlanta Black Pride Weekend, August 28 - September 2 2024Annual LGBTQ\u002B festival with male, female and youth events across Atlanta, including parties, block parties, food events, performances, film festival, and more. More details\r\n\r\n\u0026ndash; Dragon Con, August 29 - September 2 2024One of the biggest annual pop culture conventions in the country. More details\r\n\r\n\u0026ndash; Atlanta Black Theatre Festival, August 30 - September 1 2024Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events. More details\r\n\r\n\u0026ndash; The Mountaintop, August 30 - September 22 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Atlanta Hip Hop Day Festival, August 31 - September 1 2024Annual festival of hip hop music at Historic 4th Ward Park. More details\r\n\r\n\u0026ndash; Paw Patrol Live!, August 31 - September 1 2024Children\u0026rsquo;s performance at the Fox Theatre. More details\r\n\r\n\u0026ndash; Monster Jam, August 31 - September 1 2024Monster truck show at the Gas South Arena. More details\r\n\r\n\u0026ndash; Art in the Park, August 31 - September 2 2024Annual Labor Day weekend arts festival in Marietta Square, with fine art market featuring artists from across the country, chalk art, children\u0026rsquo;s craft activities and more. More details\r\n\r\n\u0026ndash; Pure Heat Community Festival, September 1 2024Very popular annual LGBTQ\u002B festival in Piedmont Park as part of Black Pride Weekend, with youth festival, live music and other performances, vendors, food and more. More details\r\n\r\n\u0026ndash; Big Peach Sizzler 10K \u0026amp; 5K, September 2 202410K and 5K run/walk along Peachtree road between Brookhaven/Chamblee and Buckhead, to benefit Miles for Cystic Fibrosis. More details\r\n\r\n\u0026ndash; Prog Power USA, September 4-7 2024Annual progressive and power metal festival. More details\r\n\r\n\u0026ndash; Supreme Court Justice Ketanji Brown Jackson: Lovely One Book Tour, September 5 2024Author talk at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Yellow Daisy Festival, September 5-8 2024Big annual arts and crafts festival at Stone Mount Park, with 400 artists and artisans, live music, festival food, craft demonstrations, beer garden, vintage market and more. More details\r\n\r\n\u0026ndash; Shane Gillis Live, September 6 2024Comedy show at State Farm Arena. More details\r\n\r\n\u0026ndash; Atlanta Home Show, September 6-8 2024Annual fall home improvements trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, September 6-8 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Meredith Willson\u0026rsquo;s The Music Man, September 6-22 2024Musical at the Sandy Springs Performing Arts Center. More details\r\n\r\n\u0026ndash; Beloved Community International Expo, September 7 2024Annual international festival at The King Center, with global foods, performances, art, music, dance and more. More details\r\n\r\n\u0026ndash; Bill Maher: The WTF? Tour, September 7 2024Comedy show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Stuff You Should Know Live, September 7 2024Podcast live show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Dale Elliott Jr: No Hard Feelings, September 7 2024Comedy show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Tacos \u0026amp; Tequilas Marietta Square, September 7 2024Taco and margarita themed bar crawl in downtown Marietta. More details\r\n\r\n\u0026ndash; Coulda Fest, September 7 2024Comedy and hip-hop music festival at State Farm Arena. More details\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Giggly Squad Live: Club Giggly, September 8 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; The Wait Wait Stand-Up Tour, September 8 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n## Events From September 9 To The Weekend Of September 14-15 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Figgmunity, September 9 2024Podcast live show at Terminal West. More details\r\n\r\n\u0026ndash; Wild With Happy, August 16 - September 15 2024Play at Horizon Theatre. More details\r\n\r\n\u0026ndash; The Mountaintop, August 30 - September 22 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Meredith Willson\u0026rsquo;s The Music Man, September 6-22 2024Musical at the Sandy Springs Performing Arts Center. More details\r\n\r\n\u0026ndash; Spider-Man: Across the Spider-Verse Live in Concert, September 11 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; An Evening with Dr Jane Goodall, September 11 2024Talk event at the Fox Theatre. More details\r\n\r\n\u0026ndash; Adam Ray, September 12 2024Comedy show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Andres Lopez: La Pelota de Letras 20 A\u0026ntilde;os, September 12 2024Spanish-language comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Scott Antique Market, September 12-15 2024Monthly antiques market at the Atlanta Expo Center. More details\r\n\r\n\u0026ndash; The Princess Bride in Concert, September 13-14 2024Film screening with live orchestral accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Food \u0026amp; Wine Festival, September 13-15 2024Big annual food festival with a variety of events including tastings, chef talks, and more. More details\r\n\r\n\u0026ndash; Fall Into Rhythm: Atlanta Ballet, September 13-15 2024Ballet performance at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; West Ponce Music Stroll, September 14 2024Free annual music festival along West Ponce de Leon Avenue, Decatur, with outdoor concerts, chalk art, 5K, fitness classes, makers market and more. More details\r\n\r\n\u0026ndash; Sandy Springs Jam Fest, September 14 2024Inaugural festival of local and regional music at The Springs Cinema \u0026amp; Taphouse. More details\r\n\r\n\u0026ndash; Kurtis Conner: The Goodfellow World Tour, September 14 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Jim Jefferies with Mark Normand and Dan Soder, September 14 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Major League Wrestling Presents: Fightland, September 14 2024Wrestling event at Center Stage. More details\r\n\r\n\u0026ndash; Men Are From Mars, Women Are From Venus Live!, September 14 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Sandy Springs Fall Arts and Craft Festival, September 14-15 2024Twice-annual arts festival in downtown Sandy Springs with 100\u002B arts and crafts vendors, local musicians, art activities and more. More details\r\n\r\n\u0026ndash; Out of the Box: Series II, September 14-15, 21-22 2024Ballet performance by Terminus Modern Ballet Theatre at the White Box Theater. More details\r\n\r\n\u0026ndash; Georgia Bridal Show, September 15 2024Wedding trade show at the Gas South District. More details\r\n\r\n\u0026ndash; An Evening with Jamie Lissow, September 15 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; The Sneaker Exit, September 15 2024Footwear trade show at the Gas South Convention Center. More details\r\n\r\n## Events From September 16 To The Weekend Of September 21-22 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; The Mountaintop, August 30 - September 22 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Meredith Willson\u0026rsquo;s The Music Man, September 6-22 2024Musical at the Sandy Springs Performing Arts Center. More details\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Wild \u0026rsquo;N Out Live: The Final Lap, September 19 2024Comedy show at State Farm Arena. More details\r\n\r\n\u0026ndash; Geeked Week 24, September 19 2024Netflix preview event at The Eastern. More details\r\n\r\n\u0026ndash; Georgia Latino International Film Festival, September 19-22 2024Annual festival of Latino/Latina-produced films. More details\r\n\r\n\u0026ndash; Hauntlanta, September 19-22 2024Annual convention of paranormal culture at Lanier Islands, Buford. More details\r\n\r\n\u0026ndash; North Georgia State Fair, September 19-29 2024Annual fair at Jim R Miller Park, Marietta, with carnival rides, animal shows, performances, rodeo, car events and more. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; Kirkwood Wine Stroll, September 20 2024Annual fundraiser of the Kirkwood Business Owners\u0026rsquo; Association, with wine-tastings at 50\u002B neighborhood establishments, demonstrations, live music, and more. More details\r\n\r\n\u0026ndash; Iliza: The Get Ready Tour, September 20 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Eddie Pokropski \u0026ndash; Case 84: Adopted in the USA, September 21 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; Atlanta Greek Festival, September 20-22 2024Annual festival of Greek food and heritage at Annunciation Greek Orthodox Cathedral, with traditional Greek foods and food demonstrations, cathedral tours, live music and dancing, performances and more. More details\r\n\r\n\u0026ndash; Little Shop of Horrors: An Art Farm at Serenbe and Matt Logan Production, September 20-29 2024Musical under the tent in the Wildflower Meadow at Serenbe, Chattahoochee Hills. More details\r\n\r\n\u0026ndash; Porches and Pies, September 21 2024Annual pie-themed food festival in Adair Park, with pie tastings and food trucks, vendor market, live entertainment, beer and cocktail garden, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Pop-In For Family Fun: Books, Books, Books, September 21 2024Monthly free family day at the Marietta History Center. More details\r\n\r\n\u0026ndash; HBCU AccessFest, September 21 2024Free college and career fair at Woodruff Arts Center, with representatives from Historically Black Colleges and Universities from across the country. More details\r\n\r\n\u0026ndash; Million Meal Pack, September 21 2024Charity event at State Farm Arena at which volunteers pack meals for local people experiencing food insecurity. More details\r\n\r\n\u0026ndash; Brett Goldstein: The Second Best Night Of Your Life, September 21 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Sippin\u0026rsquo; Safari, September 21 2024After-hours wine festival at Zoo Atlanta, with wine tasting and live music amidst the animal displays. More details\r\n\r\n\u0026ndash; Out of the Box: Series II, September 14-15, 21-22 2024Ballet performance by Terminus Modern Ballet Theatre at the White Box Theater. More details\r\n\r\n\u0026ndash; Buckhead Arts Festival, September 21-22 2024Annual outdoor fine arts festival in Buckhead, with 100\u002B artists working in a variety of media, art demonstrations and live music. More details\r\n\r\n\u0026ndash; JapanFest, September 21-22 2024Annual festival of Japanese culture at the Gas South Convention Center, with Japanese food from local restaurants and food trucks, live performances, vendors and more. More details\r\n\r\n\u0026ndash; Peachtree Corners Festival, September 21-22 2024Annual fall community festival at Peachtree Corners Town Green, with arts and crafts, classic car show, live music, food and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Matteo Lane: Can\u0026rsquo;t Stop Talking, September 22 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; My Brother, My Brother \u0026amp; Me, September 22 2024Podcast live show at Atlanta Symphony Hall. More details\r\n\r\n## Events From September 23 To The Weekend Of September 28-29 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; North Georgia State Fair, September 19-29 2024Annual fair at Jim R Miller Park, Marietta, with carnival rides, animal shows, performances, rodeo, car events and more. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; Little Shop of Horrors: An Art Farm at Serenbe and Matt Logan Production, September 20-29 2024Musical under the tent in the Wildflower Meadow at Serenbe, Chattahoochee Hills. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney On Ice, September 26-29 2024Family ice-skating performance at the Gas South Arena. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; Aqua Vino, September 27 2024Annual wine tasting event at the Georgia Aquarium to benefit its research and conservation initiatives, with unlimited wine samples, small bites and live entertainment amidst the sea life exhibits. More details\r\n\r\n\u0026ndash; Sweet Auburn Music Fest, September 27-29 2024Popular annual R\u0026amp;B and gospel music street festival in the Sweet Auburn district, with international food, art and other vendors, local businesses and more. More details\r\n\r\n\u0026ndash; Joelanta, September 27-29 2024Action figure and collectibles convention at the Gas South Convention Center. More details\r\n\r\n\u0026ndash; East Atlanta Strut, September 28 2024Annual neighborhood festival in East Atlanta, with parade, contests, marketplaces, music and more. More details\r\n\r\n\u0026ndash; Yappy Hour, September 28 2024Pet-friendly event series at The Hamilton Hotel Alpharetta, with pet adoptions and specialty cocktails. More details\r\n\r\n\u0026ndash; AIDS Walk Atlanta Music Festival \u0026amp; 5K Run, September 28 20245K run/walk and music festival in Piedmont Park to benefit AIDS services in Atlanta. More details\r\n\r\n\u0026ndash; French Market Atlanta, September 28 2024French-style outdoor market in Buckhead Village District, with French food, crafts and live music. More details\r\n\r\n\u0026ndash; Fine Wine Festival, September 28 2024Wine festival at the Home Depot Backyard, with wine tastings, R\u0026amp;B music and more. More details\r\n\r\n\u0026ndash; Trae Crowder, September 28 2024Comedy show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Ghost Files Live!, September 28 2024Paranormal investigation live show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Riverfest, September 28-29 2024Annual juried arts and crafts festival in Etowah River Park, Canton, with 150\u002B artists and artisans from across the southeast, local food trucks and musicians, children\u0026rsquo;s activities, football screenings and more. More details\r\n\r\n\u0026ndash; Roswell Wine Festival, September 28-29 2024Annual wine festival in downtown Roswell, with wine tastings, food from local restaurants and live music. More details\r\n\r\n\u0026ndash; Duluth Fall Festival, September 28-29 2024Popular fall neighborhood festival in downtown Duluth, with parade, 5K race, concert and other entertainment, arts and crafts, food and more. More details\r\n\r\n\u0026ndash; Everyday Heroes, September 28-29 2024Dance performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; Atlanta Comic Convention, September 29 2024Comic book and collectibles convention at the Marriott Hotel - Century Center. More details\r\n\r\n## Events From September 30 To The Weekend Of October 5-6 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; The Moth Atlanta GrandSLAM, October 1 2024Storytelling competition at Center Stage. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Kumail Nanjiani: Doing This Again, October 4 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Stand Up Comedy at the Clay! Hosted by Yoshee, October 4 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; ExpoContratista, October 4-5 2024Hispanic construction trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; How to Train Your Dragon in Concert with the ASO, October 4-5 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Horror Film Festival, October 4-6 2024Annual festival of horror film at Limeliight Theater. More details\r\n\r\n\u0026ndash; DreamHack Atlanta, October 4-6 2024Annual esports convention at the Georgia World Congress Center. More details\r\n\r\n\u0026ndash; Atlanta Sci-Fi Film Festival, October 4-6 2024Annual science fiction film festival in Peachtree Corners. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, October 4-6 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; Wings \u0026amp; Whitewalls, October 5 2024Fall car show at the Aviation History \u0026amp; Technology Center, Marietta, with vintage aircraft, cars and more. More details\r\n\r\n\u0026ndash; Taste of Tucker, October 5 2024Annual food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music and more. More details\r\n\r\n\u0026ndash; EGGtoberfest, October 5 2024Barbecue festival at Coolray Field, Lawrenceville, with cook-off and cooking demonstrations, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; WWE Bad Blood, October 5 2024Wrestling event at State Farm Arena. More details\r\n\r\n\u0026ndash; Festival on Ponce, October 5-6 2024Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; Candler Park Fall Fest, October 5-6 2024Popular annual festival in Candler Park and neighborhood, with live music, art, food, rides, games and more. More details\r\n\r\n\u0026ndash; Atlanta Arab Festival, October 5-6 2024Annual festival of Arabic food and culture at the Alif Institute, with traditional dishes from Arabic countries, live performances, vendors, exhibits, workshops, games and more. More details\r\n\r\n\u0026ndash; Aladdin, October 5-6 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026rarr; See October events\r\n\r\n\u0026copy; Melanie K Jones 2024\r\n\r\n* Privacy\r\n* Terms\r\n* Contact\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://discoveratlanta.com/events/main/"}
```
and got this result:
```
"Explore Atlanta Events for Today, This Week or Weekend[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading[](#events-listing-search)# Events\r\n\r\n## Unleash the fun!\r\n\r\nStay up to date on things to do and events in Atlanta today, this week and this weekend. Whether you\u2019re into live music, food and wine, sports or family-friendly entertainment, Atlanta events offer nonstop fun for everyone.\u0026nbsp;\r\n\r\nSearch the Atlanta events calendar below or explore our annual festivals and seasonal events taking place throughout the year. If you\u2019re in ATL now, make sure you see what\u2019s happening today, this week and find things to do this weekend so you don\u2019t miss any one-of-a-kind Atlanta events.\r\n\r\nVenues and event organizers, submit events here.\r\n\r\n[Fall Festivals](https://discoveratlanta.com/events/festivals/fall/)[READ MORE](https://discoveratlanta.com/events/festivals/fall/)[Upcoming Events](https://discoveratlanta.com/events/all/?date_sort=today)[Find Out More](https://discoveratlanta.com/events/all/?date_sort=today)[Festivals \u0026amp; Annual Events](https://discoveratlanta.com/events/festivals/)[LEARN MORE](https://discoveratlanta.com/events/festivals/)[Things to do In Atlanta this Weekend](https://discoveratlanta.com/stories/things-to-do/things-to-do-in-atlanta-this-weekend/)[LEARN MORE](https://discoveratlanta.com/stories/things-to-do/things-to-do-in-atlanta-this-weekend/)[Plan An Epic Road Trip To Atlanta](https://discoveratlanta.com/things-to-do/how-to-plan-an-epic-road-trip-to-atlanta/)[Make Your Plans](https://discoveratlanta.com/things-to-do/how-to-plan-an-epic-road-trip-to-atlanta/)## Events in Atlanta This Week\r\n\r\nEvents provide a glimpse into the ATL experience like nothing else, warmly welcoming visitors to the city year-round. Discover events in Atlanta.\r\n\r\n[See more events](/events/all)[Walt DisneyxUNCF Scholar Program Now Open!Education \u0026amp; Lectures9/24/24 \u2013 10/1/24](https://discoveratlanta.com/event/detail/walt-disneyxuncf-scholar-program-now-open/)[Edgewood Comedy Night at Georgia Beer GardenComedy9/24/24](https://discoveratlanta.com/event/detail/edgewood-comedy-night-at-georgia-beer-garden/)[Oktoberfest Tasting ExperienceDrinks9/24/24](https://discoveratlanta.com/event/detail/oktoberfest-tasting-experience/)[Oktoberfest Crafts \u0026#038; Romcom NightOther Activities9/25/24](https://discoveratlanta.com/event/detail/oktoberfest-crafts-romcom-night/)[Disney on Ice at Gas South ArenaFamily \u0026amp; Kids9/26/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/disney-on-ice-at-gas-south-arena/)[Centennial Yards Presents Friday VibeNeighborhood9/27/24](https://discoveratlanta.com/event/detail/centennial-yards-presents-friday-vibe/)[Rival Entertainment Presents an Evening with CAKEMusic \u0026amp; Concerts9/28/24](https://discoveratlanta.com/event/detail/rival-entertainment-presents-an-evening-with-cake/)[Ethel\u2019s Birthday Jam Session at Elsewhere BrewingMusic \u0026amp; Concerts9/29/24](https://discoveratlanta.com/event/detail/ethel-s-birthday-jam-session-at-elsewhere-brewing/)[A Sunset Cruise aboard The ExperienceOutdoors \u0026amp; Recreation9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/a-sunset-cruise-aboard-the-experience/)[See more events](/events/all)## Events in Atlanta This Weekend\r\n\r\nEvents provide a glimpse into the ATL experience like nothing else, warmly welcoming visitors to the city year-round. Discover events in Atlanta.\r\n\r\n[]()[Walt DisneyxUNCF Scholar Program Now Open!Education \u0026amp; Lectures9/24/24 \u2013 10/1/24](https://discoveratlanta.com/event/detail/walt-disneyxuncf-scholar-program-now-open/)[Edgewood Comedy Night at Georgia Beer GardenComedy9/24/24](https://discoveratlanta.com/event/detail/edgewood-comedy-night-at-georgia-beer-garden/)[KultureCity Fundraiser at Lazy BettyFundraiser9/24/24](https://discoveratlanta.com/event/detail/kulturecity-fundraiser-at-lazy-betty/)[Oktoberfest Tasting ExperienceDrinks9/24/24](https://discoveratlanta.com/event/detail/oktoberfest-tasting-experience/)[A Sunset Cruise aboard The ExperienceOutdoors \u0026amp; Recreation9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/a-sunset-cruise-aboard-the-experience/)[Daddy Long LegsTheater \u0026amp; Performing Arts9/24/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/daddy-long-legs/)[](https://discoveratlanta.com/event/detail/walt-disneyxuncf-scholar-program-now-open/)### Walt DisneyxUNCF Scholar Program Now Open!\r\n\r\nEducation \u0026amp; Lectures\r\n\r\n9/24/24 \u2013 10/1/24\r\n\r\n[](https://discoveratlanta.com/event/detail/edgewood-comedy-night-at-georgia-beer-garden/)### Edgewood Comedy Night at Georgia Beer Garden\r\n\r\nComedy\r\n\r\n9/24/24\r\n\r\n[](https://discoveratlanta.com/event/detail/kulturecity-fundraiser-at-lazy-betty/)### KultureCity Fundraiser at Lazy Betty\r\n\r\nFundraiser\r\n\r\n9/24/24\r\n\r\n[](https://discoveratlanta.com/event/detail/oktoberfest-tasting-experience/)### Oktoberfest Tasting Experience\r\n\r\nDrinks\r\n\r\n9/24/24\r\n\r\n[](https://discoveratlanta.com/event/detail/a-sunset-cruise-aboard-the-experience/)### A Sunset Cruise aboard The Experience\r\n\r\nOutdoors \u0026amp; Recreation\r\n\r\n9/24/24 \u2013 9/28/24\r\n\r\n[](https://discoveratlanta.com/event/detail/daddy-long-legs/)### Daddy Long Legs\r\n\r\nTheater \u0026amp; Performing Arts\r\n\r\n9/24/24 \u2013 10/6/24\r\n\r\n## Upcoming Atlanta events\r\n\r\n[EXPLORE MORE EVENTS](https://discoveratlanta.com/events/all/)SearchSubmit keyword searchFiltersCategoryallAutomotiveBenefitComedyDay CampsDiningDrinksEducation \u0026amp; LecturesFamily \u0026amp; KidsFashionFestivalsFilmFitness \u0026amp; HealthFundraiserGamingHolidayHoliday Pop-upHome ToursLGBTLiteraryMental HealthMuseums \u0026amp; AttractionsMusic \u0026amp; ConcertsNeighborhoodNetworkingNightlifeOther ActivitiesOutdoors \u0026amp; RecreationPetsShoppingSpanish-speaking/LatinSpecial EventsSpiritualitySportsTheater \u0026amp; Performing ArtsVirtual \u0026amp; OnlineVisual Arts \u0026amp; GalleriesWatch PartyYear of the YouthShow meAllEditor\u2019s pickWhat\u2019s hot?Clear FiltersCalendarShowing1\u201315of500No events available. Try clearing filters and searching again.\r\n\r\n[College Football Hall of Fame9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/college-football-hall-of-fame/)[Pumpkin Festival9/24/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/pumpkin-festival/)[Sweet Auburn Music Fest9/28/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/sweet-auburn-music-fest/)[Scarecrows in the GardenMultiple days until 11/03/2024](https://discoveratlanta.com/event/detail/scarecrows-in-the-garden/)[Georgia Latino International Film Festival9/26/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/georgia-latino-international-film-festival/)[3rd Annual Baldie Con9/27/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/3rd-annual-baldie-con/)[Atlanta Food Staples Wine Dinner9/27/24](https://discoveratlanta.com/event/detail/atlanta-food-staples-wine-dinner/)[Westside Beer, Bourbon \u0026#038; Bites Fest9/28/24](https://discoveratlanta.com/event/detail/westside-beer-bourbon-bites-fest/)[Atlanta Art Week9/30/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/atlanta-art-week/)[Games Week Georgia10/1/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/games-week-georgia/)[DreamHack Atlanta10/4/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/dreamhack-atlanta/)[Haunted Seas10/4/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/haunted-seas/)[Candler Park Fall Festival10/5/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/candler-park-fall-festival/)[Little 5 Points Halloween Festival \u0026#038; Parade10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/little-5-points-halloween-festival-parade/)[Stone Mountain Scottish Festival and Highland Games10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/stone-mountain-scottish-festival-and-highland-games/)[Goblins in the Garden10/20/24](https://discoveratlanta.com/event/detail/goblins-in-the-garden/)[Cabbagetown Chomp \u0026#038; Stomp11/2/24](https://discoveratlanta.com/event/detail/cabbagetown-chomp-stomp/)[Atlanta Chili Cookoff11/16/24](https://discoveratlanta.com/event/detail/atlanta-chili-cookoff/)[Invesco QQQ Thanksgiving Day Half Marathon11/28/24](https://discoveratlanta.com/event/detail/invesco-qqq-thanksgiving-day-half-marathon/)[The Horizon of Khufu9/24/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/the-horizon-of-khufu/)[Usher Performing in AtlantaMultiple dates until 10/20/2024](https://discoveratlanta.com/event/detail/usher-performing-in-atlanta/)[Atlanta History Center9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/atlanta-history-center/)[Georgia Aquarium9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/georgia-aquarium/)[La Boheme9/24/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/la-boheme/)[Pancho Rabbit and the Coyote9/24/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/pancho-rabbit-and-the-coyote/)[RENT \u2013 The Musical9/24/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/rent-the-musical/)[Elevate AtlantaEvery weekend until 10/13/2024](https://discoveratlanta.com/event/detail/elevate-atlanta/)[Little Shop of Horrors9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/little-shop-of-horrors/)[Placita Latina Exhibition9/24/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/placita-latina-exhibition/)[Walt DisneyxUNCF Scholar Program Now Open!9/24/24 \u2013 10/1/24](https://discoveratlanta.com/event/detail/walt-disneyxuncf-scholar-program-now-open/)[Oktoberfest at Der BiergartenEvery Saturday until 10/26/2024](https://discoveratlanta.com/event/detail/oktoberfest-at-der-biergarten/)[Sangria n\u2019 Scents DIY Candle Workshop9/24/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/sangria-n-scents-diy-candle-workshop/)[Indulge9/22/24 \u2013 9/24/24](https://discoveratlanta.com/event/detail/indulge/)[Bulla Atlanta 7th Anniversary9/24/24](https://discoveratlanta.com/event/detail/bulla-atlanta-7th-anniversary/)[DISCOVER ADAC9/24/24 \u2013 9/26/24](https://discoveratlanta.com/event/detail/discover-adac/)[EBONY HBCU Yard Fest9/24/24](https://discoveratlanta.com/event/detail/ebony-hbcu-yard-fest/)[Edgewood Comedy Night at Georgia Beer Garden9/24/24](https://discoveratlanta.com/event/detail/edgewood-comedy-night-at-georgia-beer-garden/)[F.A.C.E. Fulton County: ATLSFW9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-atlsfw/)[Hey there! Connections9/24/24](https://discoveratlanta.com/event/detail/hey-there-connections/)[KultureCity Fundraiser at Lazy Betty9/24/24](https://discoveratlanta.com/event/detail/kulturecity-fundraiser-at-lazy-betty/)[Meka James Book Launch9/24/24](https://discoveratlanta.com/event/detail/meka-james-book-launch/)[Oktoberfest Tasting Experience9/24/24](https://discoveratlanta.com/event/detail/oktoberfest-tasting-experience/)[September Networking Social9/24/24](https://discoveratlanta.com/event/detail/september-networking-social/)[\u201CAt Home in France with Timothy Corrigan\u201D at ADAC9/24/24](https://discoveratlanta.com/event/detail/at-home-in-france-with-timothy-corrigan-at-adac/)[\u201CDesigning with a Botanical Accent\u201D at ADAC9/24/24](https://discoveratlanta.com/event/detail/designing-with-a-botanical-accent-at-adac/)[\u201CThe Power of Teamwork in Residential Design\u201D ADAC9/24/24](https://discoveratlanta.com/event/detail/the-power-of-teamwork-in-residential-design-adac/)[BEAT: Steve Vai, Adrian Belew, Tony Levin, Danny Carey9/25/24](https://discoveratlanta.com/event/detail/beat-steve-vai-adrian-belew-tony-levin-danny-carey/)[Community Movie Night presented by Disney on Ice9/25/24](https://discoveratlanta.com/event/detail/community-movie-night-presented-by-disney-on-ice/)[Curated Conversations \u2013 Intro to Wellness9/25/24](https://discoveratlanta.com/event/detail/curated-conversations-intro-to-wellness/)[Kings of Leon9/25/24](https://discoveratlanta.com/event/detail/kings-of-leon/)[Leanne Morgan \u2013 What in the World?! \u2013 Book Event9/25/24](https://discoveratlanta.com/event/detail/leanne-morgan-what-in-the-world-book-event/)[Los Bravos Night9/25/24](https://discoveratlanta.com/event/detail/los-bravos-night/)[Make @ MODA: Beats \u0026#038; Bricks9/25/24](https://discoveratlanta.com/event/detail/make-moda-beats-bricks/)[Oktoberfest Crafts \u0026#038; Romcom Night9/25/24](https://discoveratlanta.com/event/detail/oktoberfest-crafts-romcom-night/)[Bradley Tusk Talks about Vote With Your Phone9/26/24](https://discoveratlanta.com/event/detail/bradley-tusk-talks-about-vote-with-your-phone/)[Disney on Ice at Gas South Arena9/26/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/disney-on-ice-at-gas-south-arena/)[Fall Native Plant Sale9/26/24](https://discoveratlanta.com/event/detail/fall-native-plant-sale/)[Geology Walk and Talk9/26/24](https://discoveratlanta.com/event/detail/geology-walk-and-talk/)[Les Dames d\u2019 Escoffier present \u201CPerfect Pairings\u201D9/26/24](https://discoveratlanta.com/event/detail/les-dames-d-escoffier-present-perfect-pairings/)[Make It Last Forever \u2013 Keith Sweat, Pretty Ricky9/26/24](https://discoveratlanta.com/event/detail/make-it-last-forever-keith-sweat-pretty-ricky/)[New South Kitchen\u2019s Wine Dinner Series9/26/24](https://discoveratlanta.com/event/detail/new-south-kitchen-s-wine-dinner-series/)[Piedmont Park Celebrates Park\u2019s 120th Birthday9/26/24](https://discoveratlanta.com/event/detail/piedmont-park-celebrates-park-s-120th-birthday/)[Piedmont Park Conservancy 35th Anniversary9/26/24](https://discoveratlanta.com/event/detail/piedmont-park-conservancy-35th-anniversary/)[Southeast Community Health Worker Network Summit9/26/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/southeast-community-health-worker-network-summit-2/)[\u201CTHE MARK OF ZORRO\u201D (1920) w/ organist Ken Double9/26/24](https://discoveratlanta.com/event/detail/the-mark-of-zorro-1920-w-organist-ken-double/)[1259 Burlesque Show in Atlanta9/27/24](https://discoveratlanta.com/event/detail/1259-burlesque-show-in-atlanta/)[Arte Agave Atlanta9/27/24](https://discoveratlanta.com/event/detail/arte-agave-atlanta/)[Atlanta Tattoo Festival9/27/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/atlanta-tattoo-festival/)[Autumn Art Market \u0026#038; Pop-Up Party at Snack Bar9/27/24](https://discoveratlanta.com/event/detail/autumn-art-market-pop-up-party-at-snack-bar/)[BUNT Live9/27/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/bunt-live/)[Candlelight: Tribute to Beyonc\u00E99/27/24](https://discoveratlanta.com/event/detail/candlelight-tribute-to-beyonc/)[Centennial Yards Presents Friday Vibe9/27/24](https://discoveratlanta.com/event/detail/centennial-yards-presents-friday-vibe/)[Chromeo \u0026#038; The Midnight9/27/24](https://discoveratlanta.com/event/detail/chromeo-the-midnight/)[Howls Moving Castle screening at Aurora Cineplex9/27/24 \u2013 10/3/24](https://discoveratlanta.com/event/detail/howls-moving-castle-screening-at-aurora-cineplex/)[Movies at The Strand: Tootsie (1982)9/27/24](https://discoveratlanta.com/event/detail/movies-at-the-strand-tootsie-1982/)[SEPHORia 20249/27/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/sephoria-2024/)[THE REFLEX PERFORMING LIVE on FTE\u2019s BACK PORCH!9/27/24](https://discoveratlanta.com/event/detail/the-reflex-performing-live-on-fte-s-back-porch/)[Unleash Your Inner Worth: Women\u2019s Summit9/27/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/unleash-your-inner-worth-women-s-summit/)[9th Annual Refuge Coffee Run: 5k and Mile Fun Run9/28/24](https://discoveratlanta.com/event/detail/9th-annual-refuge-coffee-run-5k-and-mile-fun-run/)[AIDS Walk Atlanta Music Festival \u0026#038; 5K Run9/28/24](https://discoveratlanta.com/event/detail/aids-walk-atlanta-music-festival-5k-run/)[AMADOR Fest9/28/24](https://discoveratlanta.com/event/detail/amador-fest/)[ATL Women\u2019s Polo Series9/28/24](https://discoveratlanta.com/event/detail/atl-women-s-polo-series/)[Afro Street Market Festival9/28/24](https://discoveratlanta.com/event/detail/afro-street-market-festival/)[Afro Street Market Festival: Celebrate Culture9/28/24](https://discoveratlanta.com/event/detail/afro-street-market-festival-celebrate-culture/)[An Evening with CAKE9/28/24](https://discoveratlanta.com/event/detail/an-evening-with-cake/)[Autism Speaks WALK9/28/24](https://discoveratlanta.com/event/detail/autism-speaks-walk/)[Carrollton Pride Festival 20249/28/24](https://discoveratlanta.com/event/detail/carrollton-pride-festival-2024/)[Divinely Inspired Victoriously Aware High Tea9/28/24](https://discoveratlanta.com/event/detail/divinely-inspired-victoriously-aware-high-tea/)[East Atlanta Strut Dog Adoption Event9/28/24](https://discoveratlanta.com/event/detail/east-atlanta-strut-dog-adoption-event/)[Exercise Your Faith 5k Walk/Run9/28/24](https://discoveratlanta.com/event/detail/exercise-your-faith-5k-walk-run/)[Fall KickBack ~ Block Party9/28/24](https://discoveratlanta.com/event/detail/fall-kickback-block-party/)[French Market Atlanta9/28/24](https://discoveratlanta.com/event/detail/french-market-atlanta/)[Georgia Southern Eagles at Georgia State Panthers Football9/28/24](https://discoveratlanta.com/event/detail/georgia-southern-eagles-at-georgia-state-panthers-football/)[Give Sanctuary Festival9/28/24](https://discoveratlanta.com/event/detail/give-sanctuary-festival/)[Half-Day Wine Tastings in the North Georgia9/28/24](https://discoveratlanta.com/event/detail/half-day-wine-tastings-in-the-north-georgia/)[JC100 Film Festival9/28/24](https://discoveratlanta.com/event/detail/jc100-film-festival/)[JC100 Film Festival-President Carter\u2019s 100th Bday9/28/24](https://discoveratlanta.com/event/detail/jc100-film-festival-president-carter-s-100th-bday/)[Jigsaw Dating\u00AE : Atlanta Coffee Shop Speed Dating9/28/24](https://discoveratlanta.com/event/detail/jigsaw-dating-atlanta-coffee-shop-speed-dating/)[Members Mart at The Overline9/28/24](https://discoveratlanta.com/event/detail/members-mart-at-the-overline/)[PERC Coffee Dog Adoption Event9/28/24](https://discoveratlanta.com/event/detail/perc-coffee-dog-adoption-event/)[Pixel Putt \u2013 A Mini Golf Tournament at CMoA9/28/24](https://discoveratlanta.com/event/detail/pixel-putt-a-mini-golf-tournament-at-cmoa/)[Plant-Oberfest: Plants \u0026#038; Oktoberfest at 6 Bridges9/28/24](https://discoveratlanta.com/event/detail/plant-oberfest-plants-oktoberfest-at-6-bridges/)[Rival Entertainment Presents an Evening with CAKE9/28/24](https://discoveratlanta.com/event/detail/rival-entertainment-presents-an-evening-with-cake/)[Roswell Wine FestivalMultiple dates until 09/29/2024](https://discoveratlanta.com/event/detail/roswell-wine-festival/)[Sangria n\u2019 Skincare DIY Facial Serum Workshop9/28/24](https://discoveratlanta.com/event/detail/sangria-n-skincare-diy-facial-serum-workshop/)[The Four Horsemen \u2013 Album Quality Metallica Tribut9/28/24](https://discoveratlanta.com/event/detail/the-four-horsemen-album-quality-metallica-tribut/)[The Original Atlanta Korean Festival 20249/28/24](https://discoveratlanta.com/event/detail/the-original-atlanta-korean-festival-2024/)[Unleash Your Inner Worth 1st Annual Black Tie Gala9/28/24](https://discoveratlanta.com/event/detail/unleash-your-inner-worth-1st-annual-black-tie-gala/)[Unleash Your Inner Worth 1st Annual Black-Tie Gala9/28/24](https://discoveratlanta.com/event/detail/unleash-your-inner-worth-1st-annual-black-tie-gala-2/)[Westside Chattahoochee Paddle 20249/28/24](https://discoveratlanta.com/event/detail/westside-chattahoochee-paddle-2024/)[ELEVATED RHYTHMS9/29/24](https://discoveratlanta.com/event/detail/elevated-rhythms/)[Ethel\u2019s Birthday Jam Session at Elsewhere Brewing9/29/24](https://discoveratlanta.com/event/detail/ethel-s-birthday-jam-session-at-elsewhere-brewing/)[FACEs of Fashion: A Celestial Experience9/29/24](https://discoveratlanta.com/event/detail/faces-of-fashion-a-celestial-experience/)[Georgia Plant Swap \u002B Sale Peachy Autumn9/29/24](https://discoveratlanta.com/event/detail/georgia-plant-swap-sale-peachy-autumn/)[Silent Film at The Strand: College (1927)9/29/24](https://discoveratlanta.com/event/detail/silent-film-at-the-strand-college-1927/)[THE FACEs OF FASHION:9/29/24](https://discoveratlanta.com/event/detail/the-faces-of-fashion/)[Tea Party Socials9/29/24](https://discoveratlanta.com/event/detail/tea-party-socials/)[Village Nanny Community Brand Activation9/29/24](https://discoveratlanta.com/event/detail/village-nanny-community-brand-activation/)[Corps De Ballet 26th Fashion Show \u0026#038; Luncheon9/30/24](https://discoveratlanta.com/event/detail/corps-de-ballet-26th-fashion-show-luncheon/)[F.A.C.E. Fulton County: Behind the Seams9/30/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-behind-the-seams/)[F.A.C.E. Fulton County: Fulton County NAP Exhibit9/30/24 \u2013 10/25/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-fulton-county-nap-exhibit/)[Fletcher \u2013 In Search Of The Antidote Tour9/30/24](https://discoveratlanta.com/event/detail/fletcher-in-search-of-the-antidote-tour/)[Military Influencer Conference9/30/24 \u2013 10/3/24](https://discoveratlanta.com/event/detail/military-influencer-conference/)[Tequila Dinner Featuring Pedro Furtivo9/30/24](https://discoveratlanta.com/event/detail/tequila-dinner-featuring-pedro-furtivo/)[A Haunted Lake Lanier Experience10/1/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/a-haunted-lake-lanier-experience/)[Brooks Nielsen10/1/24](https://discoveratlanta.com/event/detail/brooks-nielsen/)[Cue \u0026#038; Cocktails, Presented by Taste of Atlanta10/1/24](https://discoveratlanta.com/event/detail/cue-cocktails-presented-by-taste-of-atlanta/)[Dine With Gratitude \u0026#038; Support Giving Kitchen10/1/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/dine-with-gratitude-support-giving-kitchen/)[Games Week Georgia 202410/1/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/games-week-georgia-2024/)[InnovATL202410/1/24 \u2013 10/19/24](https://discoveratlanta.com/event/detail/innovatl2024/)[The Moth GrandSLAM: FIRED UP10/1/24](https://discoveratlanta.com/event/detail/the-moth-grandslam-fired-up/)[The Texas Chainsaw Massacre (1974)Multiple dates until 10/02/2024](https://discoveratlanta.com/event/detail/the-texas-chainsaw-massacre-1974/)[Atlanta Fashion Week 202410/2/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/atlanta-fashion-week-2024/)[CF Montr\u00E9al at Atlanta United10/2/24](https://discoveratlanta.com/event/detail/cf-montral-at-atlanta-united/)[Curated Conversations: Finding Work-Life Balance10/2/24](https://discoveratlanta.com/event/detail/curated-conversations-finding-work-life-balance/)[A Third Way10/3/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/a-third-way/)[Atlanta Art Fair10/3/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/atlanta-art-fair/)[Charli XCX and Troye Sivan10/3/24](https://discoveratlanta.com/event/detail/charli-xcx-and-troye-sivan/)[Every Girl Shines Mentor Mixer10/3/24](https://discoveratlanta.com/event/detail/every-girl-shines-mentor-mixer/)[F.A.C.E. Fulton County: Atlanta Fashion Week10/3/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-atlanta-fashion-week/)[Four Scythes Haunted Attraction10/3/24 \u2013 10/26/24](https://discoveratlanta.com/event/detail/four-scythes-haunted-attraction/)[Mud by Theater Emory10/3/24 \u2013 10/12/24](https://discoveratlanta.com/event/detail/mud-by-theater-emory/)[Navigating Youth Sports Today As Parents (Free)10/3/24](https://discoveratlanta.com/event/detail/navigating-youth-sports-today-as-parents-free/)[Scary Movies at The Strand: Scream (1996)10/3/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-scream-1996/)[Atlanta SCI-FI Film Festival10/4/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/atlanta-sci-fi-film-festival/)[Conan Gray10/4/24](https://discoveratlanta.com/event/detail/conan-gray/)[Elizabeth Gilbert BIG MAGIC Creativity Retreat10/4/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/elizabeth-gilbert-big-magic-creativity-retreat/)[Expo Contratista10/4/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/expo-contratista/)[How To Train Your Dragon: Live In Concert10/4/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/how-to-train-your-dragon-live-in-concert/)[LAYTON GIORDANI10/4/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/layton-giordani/)[Luxury Date Night with Live Music by Trey Daniels10/4/24](https://discoveratlanta.com/event/detail/luxury-date-night-with-live-music-by-trey-daniels/)[SCAD Presents: aja monet10/4/24](https://discoveratlanta.com/event/detail/scad-presents-aja-monet/)[Saxsquatch Rave10/4/24](https://discoveratlanta.com/event/detail/saxsquatch-rave/)[Signature Sounds at The Waldorf: Michelle Winters10/4/24](https://discoveratlanta.com/event/detail/signature-sounds-at-the-waldorf-michelle-winters/)[The Purple Madness \u2013 Prince Tribute10/4/24](https://discoveratlanta.com/event/detail/the-purple-madness-prince-tribute/)[20th Annual NAMIWalks Georgia10/5/24](https://discoveratlanta.com/event/detail/20th-annual-namiwalks-georgia/)[6th Annual Grove Park Community 5K10/5/24](https://discoveratlanta.com/event/detail/6th-annual-grove-park-community-5k/)[Atlanta Light The Night10/5/24](https://discoveratlanta.com/event/detail/atlanta-light-the-night/)[Breast Cancer Line Dance Fundraiser 202410/5/24](https://discoveratlanta.com/event/detail/breast-cancer-line-dance-fundraiser-2024/)[Classical K-pop Concert10/5/24](https://discoveratlanta.com/event/detail/classical-k-pop-concert/)[Cops and Cones Motor Skills Course10/5/24](https://discoveratlanta.com/event/detail/cops-and-cones-motor-skills-course/)[EVECE Walk For Life 5k Walk/Run10/5/24](https://discoveratlanta.com/event/detail/evece-walk-for-life-5k-walk-run/)[F.A.C.E. Fulton County: Les Farfadais CELESTIAL10/5/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-les-farfadais-celestial/)[Fall Festival on PonceMultiple dates until 10/06/2024](https://discoveratlanta.com/event/detail/fall-festival-on-ponce/)[Fela! Under the Stars10/5/24](https://discoveratlanta.com/event/detail/fela-under-the-stars/)[Georgia Tasting Festival 202410/5/24](https://discoveratlanta.com/event/detail/georgia-tasting-festival-2024/)[Georgia Tech Football vs Duke10/5/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-duke/)[My Plus Size Wedding Expo10/5/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/my-plus-size-wedding-expo/)[Punch Card Machine Demo @ CMoA, October10/5/24](https://discoveratlanta.com/event/detail/punch-card-machine-demo-cmoa-october/)[Rise and Dine with the Animals: GorillasMultiple dates until 10/12/2024](https://discoveratlanta.com/event/detail/rise-and-dine-with-the-animals-gorillas/)[Roberta Lea Live10/5/24](https://discoveratlanta.com/event/detail/roberta-lea-live/)[Strand Ole Opry: Fa-BOO-lous Halloween Hits10/5/24](https://discoveratlanta.com/event/detail/strand-ole-opry-fa-boo-lous-halloween-hits/)[Cirque Kalabante \u2013 Afrique en Cirque10/6/24](https://discoveratlanta.com/event/detail/cirque-kalabante-afrique-en-cirque/)[Dean Owens Live10/6/24](https://discoveratlanta.com/event/detail/dean-owens-live/)[Fall SCRAPFest: a comedy fundraiser \u0026#038; craft fair10/6/24](https://discoveratlanta.com/event/detail/fall-scrapfest-a-comedy-fundraiser-craft-fair/)[Flower Power: A Fall Flower-Arranging Workshop10/6/24](https://discoveratlanta.com/event/detail/flower-power-a-fall-flower-arranging-workshop/)[Halloween Brick-or-Treat Monster Party10/6/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/halloween-brick-or-treat-monster-party/)[Jazz at The Strand: The Music of Louis Armstrong10/6/24](https://discoveratlanta.com/event/detail/jazz-at-the-strand-the-music-of-louis-armstrong/)[Maize \u0026#038; Mingle10/6/24](https://discoveratlanta.com/event/detail/maize-mingle/)[OYE Fest10/6/24](https://discoveratlanta.com/event/detail/oye-fest/)[Oktoberfest at Chateau Elan10/6/24](https://discoveratlanta.com/event/detail/oktoberfest-at-chateau-elan/)[Side by Side by Side: A Sondheim Celebration10/6/24](https://discoveratlanta.com/event/detail/side-by-side-by-side-a-sondheim-celebration/)[Sip \u0026#038; Style: A Fall Fashion \u0026#038; Wine Showcase10/6/24](https://discoveratlanta.com/event/detail/sip-style-a-fall-fashion-wine-showcase/)[Sunday Jazz Brunch at One Flew South BeltLine10/6/24](https://discoveratlanta.com/event/detail/sunday-jazz-brunch-at-one-flew-south-beltline/)[Association for Talent Development\u2019s OrgDev2410/7/24 \u2013 10/9/24](https://discoveratlanta.com/event/detail/association-for-talent-development-s-orgdev24/)[Avant South10/7/24](https://discoveratlanta.com/event/detail/avant-south/)[F.A.C.E. Fulton County: DIY Workshop Trendy Totes10/7/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-diy-workshop-trendy-totes/)[F.A.C.E. Fulton County: Workshop Trendy Totes10/7/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-workshop-trendy-totes/)[Nygren Placemaking Conference10/7/24 \u2013 10/9/24](https://discoveratlanta.com/event/detail/nygren-placemaking-conference/)[Open House: Focused Community Strategies10/7/24 \u2013 10/8/24](https://discoveratlanta.com/event/detail/open-house-focused-community-strategies/)[Venture Atlanta 202410/8/24 \u2013 10/9/24](https://discoveratlanta.com/event/detail/venture-atlanta-2024/)[Atlanta miniFEST10/9/24 \u2013 10/10/24](https://discoveratlanta.com/event/detail/atlanta-minifest/)[Homeshake10/9/24](https://discoveratlanta.com/event/detail/homeshake/)[Inclusive Health Equity Conference at Agnes Scott10/9/24](https://discoveratlanta.com/event/detail/inclusive-health-equity-conference-at-agnes-scott/)[Walter Parks \u0026#038; The Unlawful Assembly10/9/24](https://discoveratlanta.com/event/detail/walter-parks-the-unlawful-assembly/)[ART \u0026#038; SURVIVAL: Honoring Indigenous People\u2019s Day10/10/24](https://discoveratlanta.com/event/detail/art-survival-honoring-indigenous-people-s-day/)[Atlanta UserCon10/10/24](https://discoveratlanta.com/event/detail/atlanta-usercon/)[Scary Movies at The Strand: Young Frankenstein10/10/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-young-frankenstein/)[2024 Georgia Aquarium Pride Kickoff10/11/24](https://discoveratlanta.com/event/detail/2024-georgia-aquarium-pride-kickoff/)[Aerosmith: PEACE OUT The Farewell Tour with The Black Crowes10/11/24](https://discoveratlanta.com/event/detail/aerosmith-peace-out-the-farewell-tour-with-the-black-crowes/)[Akoni Eyewear \u0026#038; Sunglasses \u2013 Private Event10/11/24 \u2013 10/12/24](https://discoveratlanta.com/event/detail/akoni-eyewear-sunglasses-private-event/)[Dana and Alden10/11/24](https://discoveratlanta.com/event/detail/dana-and-alden/)[F.A.C.E. Fulton County: ATL Fashion \u0026#038; Polo Classic10/11/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-atl-fashion-polo-classic/)[FashionFest 202410/11/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/fashionfest-2024/)[No Words ATL Instrumental Music Festival10/11/24 \u2013 10/14/24](https://discoveratlanta.com/event/detail/no-words-atl-instrumental-music-festival/)[NoWordsATL3.0 Traveling Instrumental Music Fest10/11/24 \u2013 10/14/24](https://discoveratlanta.com/event/detail/nowordsatl3-0-traveling-instrumental-music-fest/)[Nurko United Tour10/11/24 \u2013 10/12/24](https://discoveratlanta.com/event/detail/nurko-united-tour/)[Scary Movies at The Strand: Poltergeist (1982)10/11/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-poltergeist-1982/)[Seven Days Band- A Tribute to Sting and The Police10/11/24](https://discoveratlanta.com/event/detail/seven-days-band-a-tribute-to-sting-and-the-police/)[BIG GAY BRUNCH Returns to Park Tavern10/12/24](https://discoveratlanta.com/event/detail/big-gay-brunch-returns-to-park-tavern/)[Cherokee Tales10/12/24](https://discoveratlanta.com/event/detail/cherokee-tales/)[Old Fourth Ward Arts FestivalMultiple dates until 10/13/2024](https://discoveratlanta.com/event/detail/old-fourth-ward-arts-festival/)[Spooky Mill10/12/24](https://discoveratlanta.com/event/detail/spooky-mill/)[Spooky Spirits 5K10/12/24](https://discoveratlanta.com/event/detail/spooky-spirits-5k/)[Taste \u0026#038; Brews Fall Festival10/12/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/taste-brews-fall-festival/)[Creative Caviar: A Unique Caviar Pairing Workshop10/13/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/creative-caviar-a-unique-caviar-pairing-workshop/)[Creed Fisher10/13/24](https://discoveratlanta.com/event/detail/creed-fisher/)[Halloween Spooktacular10/13/24](https://discoveratlanta.com/event/detail/halloween-spooktacular/)[Jeff Lynne\u2019s ELO10/13/24](https://discoveratlanta.com/event/detail/jeff-lynne-s-elo/)[Romeo and Juliet10/13/24](https://discoveratlanta.com/event/detail/romeo-and-juliet/)[26th Annual Special Needs Cobb Golf Invitational10/14/24](https://discoveratlanta.com/event/detail/26th-annual-special-needs-cobb-golf-invitational/)[World Commerce and Contracting Americas Summit10/14/24 \u2013 10/15/24](https://discoveratlanta.com/event/detail/world-commerce-and-contracting-americas-summit/)[QS Discover for Masters and MBA in Atlanta10/15/24](https://discoveratlanta.com/event/detail/qs-discover-for-masters-and-mba-in-atlanta/)[Feed Your Senses Concert: Nikki Speake/Anna Caller10/16/24](https://discoveratlanta.com/event/detail/feed-your-senses-concert-nikki-speake-anna-caller/)[I Know What You Did Last Summer (1997)10/17/24](https://discoveratlanta.com/event/detail/i-know-what-you-did-last-summer-1997/)[Midtown Movie Night: Nightmare Before Christmas10/17/24](https://discoveratlanta.com/event/detail/midtown-movie-night-nightmare-before-christmas/)[Planned PEThood of Georgia Murder Mystery Dinner10/17/24](https://discoveratlanta.com/event/detail/planned-pethood-of-georgia-murder-mystery-dinner/)[ShaoConMultiple dates until 10/20/2024](https://discoveratlanta.com/event/detail/shaocon/)[1 Night with Diplo at Underground Atlanta on 10/1810/18/24](https://discoveratlanta.com/event/detail/1-night-with-diplo-at-underground-atlanta-on-10-18/)[Comedy at The Strand: The Mary Mack Comedy Show10/18/24](https://discoveratlanta.com/event/detail/comedy-at-the-strand-the-mary-mack-comedy-show/)[Jesse Lynn Madera and Jill Sobule Live10/18/24](https://discoveratlanta.com/event/detail/jesse-lynn-madera-and-jill-sobule-live/)[Jessica Kirson10/18/24](https://discoveratlanta.com/event/detail/jessica-kirson/)[Jill Sobule And Jesse Lynn Madera10/18/24 \u2013 10/19/24](https://discoveratlanta.com/event/detail/jill-sobule-and-jesse-lynn-madera/)[Jos\u00E9 Gonz\u00E1lez10/18/24](https://discoveratlanta.com/event/detail/jos-gonzalez/)[PeriCon(ATL) Menopause Transition Summit10/18/24](https://discoveratlanta.com/event/detail/pericon-atl-menopause-transition-summit/)[25th Annual Trees Atlanta Tree Sale10/19/24](https://discoveratlanta.com/event/detail/25th-annual-trees-atlanta-tree-sale/)[Candlelight: A Haunted Evening of Halloween10/19/24](https://discoveratlanta.com/event/detail/candlelight-a-haunted-evening-of-halloween/)[Cars \u0026#038; \u2018Q for the Cause10/19/24](https://discoveratlanta.com/event/detail/cars-q-for-the-cause/)[Georgia Tech Football vs Notre Dame10/19/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-notre-dame/)[Jackson County Lane10/19/24](https://discoveratlanta.com/event/detail/jackson-county-lane/)[John Vincent III: The Roadtrip10/19/24](https://discoveratlanta.com/event/detail/john-vincent-iii-the-roadtrip/)[Macon Beer Fest10/19/24](https://discoveratlanta.com/event/detail/macon-beer-fest/)[NBAF Celebration Under the Stars10/19/24](https://discoveratlanta.com/event/detail/nbaf-celebration-under-the-stars/)[Paws and Pastries10/19/24](https://discoveratlanta.com/event/detail/paws-and-pastries/)[Pine Lake Lakefest10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/pine-lake-lakefest/)[Pride Showcase10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/pride-showcase/)[Race for Rest 202410/19/24](https://discoveratlanta.com/event/detail/race-for-rest-2024/)[Safe and Secure Online \u2013 Cybersecurity for Kids10/19/24](https://discoveratlanta.com/event/detail/safe-and-secure-online-cybersecurity-for-kids/)[Sweet Auburn Barbecue Watch Party10/19/24](https://discoveratlanta.com/event/detail/sweet-auburn-barbecue-watch-party/)[Talking Tech: Det. Zack Kowalske \u2013 Astroforenics10/19/24](https://discoveratlanta.com/event/detail/talking-tech-det-zack-kowalske-astroforenics/)[To The Limit: An Eagles Tribute10/19/24](https://discoveratlanta.com/event/detail/to-the-limit-an-eagles-tribute/)[Tower of Power10/19/24](https://discoveratlanta.com/event/detail/tower-of-power/)[Dr. Jekyll and Mr. Hyde (1920)10/20/24](https://discoveratlanta.com/event/detail/dr-jekyll-and-mr-hyde-1920/)[Experience Castleberry Hill: Loft Tour \u0026#038; Wine Tast10/20/24](https://discoveratlanta.com/event/detail/experience-castleberry-hill-loft-tour-wine-tast/)[Litner\u002BDeganian Fall Family Fun Fest Dog Adoption10/20/24](https://discoveratlanta.com/event/detail/litner-deganian-fall-family-fun-fest-dog-adoption/)[Sip \u0026#038; Scent: A Halloween Candle-Making Workshop10/20/24](https://discoveratlanta.com/event/detail/sip-scent-a-halloween-candle-making-workshop/)[The Atlanta Symphony Orchestra at Spivey Hall10/20/24](https://discoveratlanta.com/event/detail/the-atlanta-symphony-orchestra-at-spivey-hall/)[The Wilson Springs Hotel w/ Conrad Moore10/20/24](https://discoveratlanta.com/event/detail/the-wilson-springs-hotel-w-conrad-moore/)[DJ Shadow10/21/24](https://discoveratlanta.com/event/detail/dj-shadow/)[PINK\u2019s Global 20th Fall Women\u2019s Empowerment Event10/22/24](https://discoveratlanta.com/event/detail/pink-s-global-20th-fall-women-s-empowerment-event/)[The Headless Horseman of Silly Hollow10/23/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/the-headless-horseman-of-silly-hollow/)[ATLANTA BUSINESS OWNERS POP-UP MIXER10/24/24](https://discoveratlanta.com/event/detail/atlanta-business-owners-pop-up-mixer/)[An Evening with Witches10/24/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/an-evening-with-witches/)[Candlelight: Horror Movie Soundtracks10/24/24](https://discoveratlanta.com/event/detail/candlelight-horror-movie-soundtracks/)[Christopher Mosley \u2013 Exchange: Behind The Music10/24/24](https://discoveratlanta.com/event/detail/christopher-mosley-exchange-behind-the-music/)[HAIRSPRAY10/24/24 \u2013 11/9/24](https://discoveratlanta.com/event/detail/hairspray/)[Scary Movies at The Strand: Friday The 13th (1980)10/24/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-friday-the-13th-1980/)[Candlelight: A Haunted Evening of Halloween Classi10/25/24](https://discoveratlanta.com/event/detail/candlelight-a-haunted-evening-of-halloween-classi/)[Footwork \u0026#038; Foodtrucks Trunk or Treat10/25/24](https://discoveratlanta.com/event/detail/footwork-foodtrucks-trunk-or-treat/)[Halloween Chocolate \u0026#038; Wine Pairing10/25/24](https://discoveratlanta.com/event/detail/halloween-chocolate-wine-pairing/)[Nero Simon \u0026#038; The Sunsetters W/ The Explorers Club10/25/24](https://discoveratlanta.com/event/detail/nero-simon-the-sunsetters-w-the-explorers-club/)[The Rocky Horror Picture Show (1975)10/25/24 \u2013 10/26/24](https://discoveratlanta.com/event/detail/the-rocky-horror-picture-show-1975/)[18th PF HALLOWEEN PARTY \u0026#038; COSTUME CONTEST10/26/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/18th-pf-halloween-party-costume-contest/)[2024 Komen Georgia MORE THAN PINK Walk10/26/24](https://discoveratlanta.com/event/detail/2024-komen-georgia-more-than-pink-walk/)[Boris10/26/24](https://discoveratlanta.com/event/detail/boris/)[DEHD10/26/24](https://discoveratlanta.com/event/detail/dehd/)[Dinosaur Trick-or-Treat10/26/24](https://discoveratlanta.com/event/detail/dinosaur-trick-or-treat/)[Fall Country Affair10/26/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/fall-country-affair/)[Good Mews Fundraising Gala10/26/24](https://discoveratlanta.com/event/detail/good-mews-fundraising-gala/)[Halloween Magic Show10/26/24](https://discoveratlanta.com/event/detail/halloween-magic-show/)[INTERNATIONAL LIFE CHANGERS AWARDS \u0026#038; FUNDRAISER10/26/24](https://discoveratlanta.com/event/detail/international-life-changers-awards-fundraiser/)[JAZZ in the PARK ATL10/26/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/jazz-in-the-park-atl/)[Jelly Roll10/26/24](https://discoveratlanta.com/event/detail/jelly-roll/)[Lightning 5K/10K10/26/24](https://discoveratlanta.com/event/detail/lightning-5k-10k/)[Marietta PAL Fall Festival Family Fun House10/26/24](https://discoveratlanta.com/event/detail/marietta-pal-fall-festival-family-fun-house/)[Monster Mash10/26/24](https://discoveratlanta.com/event/detail/monster-mash/)[Official Atlanta Halloween Bar Crawl10/26/24](https://discoveratlanta.com/event/detail/official-atlanta-halloween-bar-crawl/)[Saje Grammy-Winning Female Jazz Vocal Quartet10/26/24](https://discoveratlanta.com/event/detail/saje-grammy-winning-female-jazz-vocal-quartet/)[Scary Movies at The Strand: Hocus Pocus (1991)10/26/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-hocus-pocus-1991/)[Stone Mountain Chorus \u2013 Back to Barbershop!10/26/24](https://discoveratlanta.com/event/detail/stone-mountain-chorus-back-to-barbershop/)[Tales From the T!t Halloween Burlesque Show10/26/24](https://discoveratlanta.com/event/detail/tales-from-the-t-t-halloween-burlesque-show/)[The Official Halloween Bar Crawl \u2013 Atlanta10/26/24](https://discoveratlanta.com/event/detail/the-official-halloween-bar-crawl-atlanta/)[2nd Annual Festa Dell\u2019Uva10/27/24](https://discoveratlanta.com/event/detail/2nd-annual-festa-dell-uva/)[2nd Annual Festa Dell\u2019uva at Pricci10/27/24](https://discoveratlanta.com/event/detail/2nd-annual-festa-dell-uva-at-pricci/)[Adam W. Sadberry, flute and Nathan Cheung, piano10/27/24](https://discoveratlanta.com/event/detail/adam-w-sadberry-flute-and-nathan-cheung-piano/)[Dylan Gossett \u2013 The Back 40 Tour10/27/24 \u2013 10/28/24](https://discoveratlanta.com/event/detail/dylan-gossett-the-back-40-tour/)[Halloween Kids Chocolate Making Event10/27/24](https://discoveratlanta.com/event/detail/halloween-kids-chocolate-making-event/)[METALCON10/30/24 \u2013 11/1/24](https://discoveratlanta.com/event/detail/metalcon/)[21st Annual Rome International Film Festival10/31/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/21st-annual-rome-international-film-festival/)[Candlelight: A Haunted Evening of Halloween10/31/24](https://discoveratlanta.com/event/detail/candlelight-a-haunted-evening-of-halloween-2/)[Nightmare On Elm Street (1984)10/31/24](https://discoveratlanta.com/event/detail/nightmare-on-elm-street-1984/)[ShakesBEER!10/31/24](https://discoveratlanta.com/event/detail/shakesbeer/)[The 4th Annual Vampire Ball10/31/24](https://discoveratlanta.com/event/detail/the-4th-annual-vampire-ball/)[Trick or Drink Atlanta Halloween Bar Crawl10/31/24](https://discoveratlanta.com/event/detail/trick-or-drink-atlanta-halloween-bar-crawl/)[Chayanne11/1/24](https://discoveratlanta.com/event/detail/chayanne/)[David Briggs: The Phantom of the Opera (1925)11/1/24](https://discoveratlanta.com/event/detail/david-briggs-the-phantom-of-the-opera-1925/)[Ghostbusters in Concert with the ASO11/1/24 \u2013 11/2/24](https://discoveratlanta.com/event/detail/ghostbusters-in-concert-with-the-aso/)[Ghostbusters: Live In Concert11/1/24 \u2013 11/2/24](https://discoveratlanta.com/event/detail/ghostbusters-live-in-concert/)[We CARE Vet Fair11/1/24](https://discoveratlanta.com/event/detail/we-care-vet-fair/)[Atlanta Whiskey Riot 202411/2/24](https://discoveratlanta.com/event/detail/atlanta-whiskey-riot-2024/)[Atlanta Whiskey Riot 2024 at Southern Exchange Ballroom (The Cellar)11/2/24](https://discoveratlanta.com/event/detail/atlanta-whiskey-riot-2024-at-southern-exchange-ballroom-the-cellar/)[Atlanta Women\u2019s Expo11/2/24](https://discoveratlanta.com/event/detail/atlanta-women-s-expo/)[Billie Eilish11/2/24 \u2013 11/4/24](https://discoveratlanta.com/event/detail/billie-eilish/)[Dino-Light11/2/24](https://discoveratlanta.com/event/detail/dino-light/)[Garden \u0026#038; Gun\u2019s 12th Annual Shoot11/2/24](https://discoveratlanta.com/event/detail/garden-gun-s-12th-annual-shoot/)[Paws \u0026#038; Pepper Chili Cook-Off11/2/24](https://discoveratlanta.com/event/detail/paws-pepper-chili-cook-off/)[Phenomenal Women\u2019s Health11/2/24](https://discoveratlanta.com/event/detail/phenomenal-women-s-health/)[Punch Card Machine Demo @ CMoA, November11/2/24](https://discoveratlanta.com/event/detail/punch-card-machine-demo-cmoa-november/)[Radney Foster, An Acoustic Evening; Opener11/2/24](https://discoveratlanta.com/event/detail/radney-foster-an-acoustic-evening-opener/)[Suwanee Wine Fest11/2/24](https://discoveratlanta.com/event/detail/suwanee-wine-fest/)[Dallas Cowboys at Atlanta Falcons11/3/24](https://discoveratlanta.com/event/detail/dallas-cowboys-at-atlanta-falcons/)[Nature Walk with Chattahoochee National Conservanc11/3/24](https://discoveratlanta.com/event/detail/nature-walk-with-chattahoochee-national-conservanc/)[Caloncho11/7/24](https://discoveratlanta.com/event/detail/caloncho/)[Escape to Margaritaville11/7/24 \u2013 11/16/24](https://discoveratlanta.com/event/detail/escape-to-margaritaville/)[Highlands Food \u0026#038; Wine Festival11/7/24 \u2013 11/10/24](https://discoveratlanta.com/event/detail/highlands-food-wine-festival/)[Reins of Hope11/7/24](https://discoveratlanta.com/event/detail/reins-of-hope/)[Black Women In Clinical Research Conference11/8/24 \u2013 11/10/24](https://discoveratlanta.com/event/detail/black-women-in-clinical-research-conference/)[IrishFest Atlanta 202411/8/24 \u2013 11/10/24](https://discoveratlanta.com/event/detail/irishfest-atlanta-2024-2/)[Mom Talks \u2013 Atlanta11/8/24](https://discoveratlanta.com/event/detail/mom-talks-atlanta/)[Beyond Limits Ride11/9/24](https://discoveratlanta.com/event/detail/beyond-limits-ride/)[Decatur Wine Festival11/9/24](https://discoveratlanta.com/event/detail/decatur-wine-festival/)[Georgia Tech Football vs Miami11/9/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-miami/)[JUNNY: The Dopamine Tour w/ DAUL11/9/24](https://discoveratlanta.com/event/detail/junny-the-dopamine-tour-w-daul/)[Magnificent Marimba11/9/24](https://discoveratlanta.com/event/detail/magnificent-marimba/)[Champions Classic (Kansas Jayhawks vs. Michigan State Spartans, Kentucky Wildcats vs. Duke Blue Devils)11/13/24](https://discoveratlanta.com/event/detail/champions-classic-kansas-jayhawks-vs-michigan-state-spartans-kentucky-wildcats-vs-duke-blue-devils/)[P!NK Concert at State Farm Arena11/14/24](https://discoveratlanta.com/event/detail/p-nk-concert-at-state-farm-arena/)[NPCF Fashion Funds the Cure11/15/24](https://discoveratlanta.com/event/detail/npcf-fashion-funds-the-cure/)[The Tim Shleton Syndicate11/15/24](https://discoveratlanta.com/event/detail/the-tim-shleton-syndicate/)[We Are The World Festival11/15/24 \u2013 11/17/24](https://discoveratlanta.com/event/detail/we-are-the-world-festival/)[Arkansas State Red Wolves at Georgia State Panthers Football11/16/24](https://discoveratlanta.com/event/detail/arkansas-state-red-wolves-at-georgia-state-panthers-football/)[Bohemian Rhapsody11/16/24](https://discoveratlanta.com/event/detail/bohemian-rhapsody/)[Devastation On The Nation Tour 2024.11/16/24](https://discoveratlanta.com/event/detail/devastation-on-the-nation-tour-2024/)[Locals \u0026#038; Legends11/16/24](https://discoveratlanta.com/event/detail/locals-legends/)[The All Black Experience11/16/24 \u2013 11/17/24](https://discoveratlanta.com/event/detail/the-all-black-experience/)[Uncommon Bizarre Historic Fourth WardMultiple dates until 11/17/2024](https://discoveratlanta.com/event/detail/uncommon-bizarre-historic-fourth-ward/)[Coat Check \u2013 Morgan Jay \u2013 Not a Concert Ticket**11/17/24](https://discoveratlanta.com/event/detail/coat-check-morgan-jay-not-a-concert-ticket/)[Great Atlanta Dog Show @ Wild Heaven Beer WEST END11/17/24](https://discoveratlanta.com/event/detail/great-atlanta-dog-show-wild-heaven-beer-west-end/)[52nd NABSE Conference-Empowering Black Excellence11/20/24 \u2013 11/24/24](https://discoveratlanta.com/event/detail/52nd-nabse-conference-empowering-black-excellence/)[Feed Your Senses Lunchtime Concert: Chamber Cartel11/20/24](https://discoveratlanta.com/event/detail/feed-your-senses-lunchtime-concert-chamber-cartel/)[Georgia Tech Football vs NC State11/21/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-nc-state/)[ISM TEST EVENT11/21/24](https://discoveratlanta.com/event/detail/ism-test-event/)[39th Annual Cobb County GMS Show11/22/24 \u2013 11/24/24](https://discoveratlanta.com/event/detail/39th-annual-cobb-county-gms-show/)[Adrianne Lenker11/22/24](https://discoveratlanta.com/event/detail/adrianne-lenker/)[Shawn Phillips Live11/22/24](https://discoveratlanta.com/event/detail/shawn-phillips-live/)[Becky Bullers Band XMas Show11/23/24](https://discoveratlanta.com/event/detail/becky-bullers-band-xmas-show/)[Exhale: An Evening of Sound Meditation and Music11/23/24](https://discoveratlanta.com/event/detail/exhale-an-evening-of-sound-meditation-and-music/)[Animals As Leaders11/24/24](https://discoveratlanta.com/event/detail/animals-as-leaders/)[Holiday Photos with Santa11/24/24](https://discoveratlanta.com/event/detail/holiday-photos-with-santa/)[Grand Kyiv Ballet Presents the Magic of Nutcracker11/27/24](https://discoveratlanta.com/event/detail/grand-kyiv-ballet-presents-the-magic-of-nutcracker/)[Atlanta Turkey Trot11/28/24](https://discoveratlanta.com/event/detail/atlanta-turkey-trot/)[Cirque Du Soleil Twas The Night Before11/30/24 \u2013 12/15/24](https://discoveratlanta.com/event/detail/cirque-du-soleil-twas-the-night-before/)[Coastal Carolina Chanticleers at Georgia State Panthers Football11/30/24](https://discoveratlanta.com/event/detail/coastal-carolina-chanticleers-at-georgia-state-panthers-football/)[Frozen in Concert with the ASO11/30/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/frozen-in-concert-with-the-aso/)[Gift of the Magi 2.011/30/24 \u2013 12/8/24](https://discoveratlanta.com/event/detail/gift-of-the-magi-2-0/)[American Aquarium12/1/24](https://discoveratlanta.com/event/detail/american-aquarium/)[Coat Check \u2013 Ocie Elliott \u2013 Not a Concert Ticket12/1/24](https://discoveratlanta.com/event/detail/coat-check-ocie-elliott-not-a-concert-ticket/)[Frozen: Live In Concert12/1/24](https://discoveratlanta.com/event/detail/frozen-live-in-concert/)[Creed12/5/24](https://discoveratlanta.com/event/detail/creed/)[Kathy Griffin: My Life On The PTSD-List12/7/24](https://discoveratlanta.com/event/detail/kathy-griffin-my-life-on-the-ptsd-list/)[\u2019Twas the Night Before Christmas12/7/24](https://discoveratlanta.com/event/detail/twas-the-night-before-christmas/)[27th Annual Holiday Gala by GSU School of Music12/8/24](https://discoveratlanta.com/event/detail/27th-annual-holiday-gala-by-gsu-school-of-music/)[FINAL FANTASY VII REBIRTH Orchestra World Tour12/8/24](https://discoveratlanta.com/event/detail/final-fantasy-vii-rebirth-orchestra-world-tour/)[ISMO: Watch Your Language Tour12/8/24](https://discoveratlanta.com/event/detail/ismo-watch-your-language-tour/)[Twas the Night Before by Cirque du Soleil12/8/24](https://discoveratlanta.com/event/detail/twas-the-night-before-by-cirque-du-soleil/)[356 Porsche Dinner Series // Edible Engineering12/13/24](https://discoveratlanta.com/event/detail/356-porsche-dinner-series-edible-engineering/)[Chloe Agnew And The Brave Collide Christmas Show12/14/24](https://discoveratlanta.com/event/detail/chloe-agnew-and-the-brave-collide-christmas-show/)[Holidays With the Johns Creek Symphony Orchestra12/14/24](https://discoveratlanta.com/event/detail/holidays-with-the-johns-creek-symphony-orchestra/)[North Pole Fa-La-La-La FolliesMultiple dates until 12/14/2024](https://discoveratlanta.com/event/detail/north-pole-fa-la-la-la-follies/)[Plush Romance: V Day Luxury Picnic at Piedmont12/14/24 \u2013 12/17/24](https://discoveratlanta.com/event/detail/plush-romance-v-day-luxury-picnic-at-piedmont/)[Roswell Ugly Sweater Bar Crawl: 4th Annual12/14/24](https://discoveratlanta.com/event/detail/roswell-ugly-sweater-bar-crawl-4th-annual/)[CHUU12/16/24](https://discoveratlanta.com/event/detail/chuu/)[Feed Your Senses Lunch Concert: GSU Jazz Combo12/18/24](https://discoveratlanta.com/event/detail/feed-your-senses-lunch-concert-gsu-jazz-combo/)[Blue Dogs12/27/24](https://discoveratlanta.com/event/detail/blue-dogs/)[Carolina Panthers at Atlanta Falcons1/5/25](https://discoveratlanta.com/event/detail/carolina-panthers-at-atlanta-falcons/)[Squirrel Stole My Underpants1/8/25 \u2013 1/19/25](https://discoveratlanta.com/event/detail/squirrel-stole-my-underpants/)[\u0026#038; Juliet1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/juliet/)[2025 DSC Convention \u0026#038; Sporting Expo1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/2025-dsc-convention-sporting-expo/)[College Football Playoff National Championship1/21/25](https://discoveratlanta.com/event/detail/college-football-playoff-national-championship/)[FSR Presents: Life Be Lifin\u2019 Starring Monet X Change1/24/25](https://discoveratlanta.com/event/detail/fsr-presents-life-be-lifin-starring-monet-x-change/)[First Arab Elbow Course- The Traumatic1/24/25 \u2013 1/26/25](https://discoveratlanta.com/event/detail/first-arab-elbow-course-the-traumatic/)[Heroes Among Us: A Musical Tribute1/25/25](https://discoveratlanta.com/event/detail/heroes-among-us-a-musical-tribute/)[Lunar New Year Celebration1/25/25](https://discoveratlanta.com/event/detail/lunar-new-year-celebration/)[Clue The Musical1/30/25 \u2013 2/1/25](https://discoveratlanta.com/event/detail/clue-the-musical/)[FLOTAX test2/1/25](https://discoveratlanta.com/event/detail/flotax-test/)[Hot Chocolate Run Atlanta2/2/25](https://discoveratlanta.com/event/detail/hot-chocolate-run-atlanta/)[FREE Movie Night: Frozen2/8/25](https://discoveratlanta.com/event/detail/free-movie-night-frozen/)[A Beautiful Noise: The Neil Diamond Musical2/9/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/a-beautiful-noise-the-neil-diamond-musical/)[Martin Amini2/14/25](https://discoveratlanta.com/event/detail/martin-amini/)[African-American Folktales2/15/25](https://discoveratlanta.com/event/detail/african-american-folktales/)[Trailblazers2/15/25](https://discoveratlanta.com/event/detail/trailblazers/)[Suwanee Beer Fest Beer Chaser 5K3/1/25](https://discoveratlanta.com/event/detail/suwanee-beer-fest-beer-chaser-5k/)[Dan Forrest\u2019s Newest Work \u201CCreation\u201D3/2/25](https://discoveratlanta.com/event/detail/dan-forrest-s-newest-work-creation/)[Galapagos George, The Little Tortoise that Could3/5/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/galapagos-george-the-little-tortoise-that-could/)[Blacktop Playground: Improv for Kids3/8/25](https://discoveratlanta.com/event/detail/blacktop-playground-improv-for-kids/)[PACK EXPO 20253/10/25 \u2013 3/12/25](https://discoveratlanta.com/event/detail/pack-expo-2025/)[Milo the Magnificent3/19/25 \u2013 3/30/25](https://discoveratlanta.com/event/detail/milo-the-magnificent/)[Debussy, Ravel, \u0026#038; Sibelius: A Birthday Celebration3/29/25](https://discoveratlanta.com/event/detail/debussy-ravel-sibelius-a-birthday-celebration/)[It\u2019s Electric!3/29/25](https://discoveratlanta.com/event/detail/it-s-electric/)[Spring Break Magic Show4/9/25](https://discoveratlanta.com/event/detail/spring-break-magic-show/)[Dr. Dinosaur!4/19/25](https://discoveratlanta.com/event/detail/dr-dinosaur/)[Fairy God Doctor5/3/25](https://discoveratlanta.com/event/detail/fairy-god-doctor/)[May the 4th Be With You: Sun, Moon, Air, \u0026#038; Space5/4/25](https://discoveratlanta.com/event/detail/may-the-4th-be-with-you-sun-moon-air-space/)[The Firebird5/17/25](https://discoveratlanta.com/event/detail/the-firebird/)[Oral History Association 2025 Annual Meeting10/14/25 \u2013 10/19/25](https://discoveratlanta.com/event/detail/oral-history-association-2025-annual-meeting/)[9.99 Burger Night at Elsewhere9/24/24 \u2013 9/26/24](https://discoveratlanta.com/event/detail/9-99-burger-night-at-elsewhere/)[Johnnie \u0026#038; Clyde\u2019s Pop Up at Elsewhere Brewing9/2/24 \u2013 9/24/24](https://discoveratlanta.com/event/detail/johnnie-clyde-s-pop-up-at-elsewhere-brewing/)[Center for Puppetry Arts9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/center-for-puppetry-arts/)[Children\u2019s Museum of AtlantaEvery Friday until 09/30/2024](https://discoveratlanta.com/event/detail/children-s-museum-of-atlanta/)[David J. Sencer CDC Museum9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/david-j-sencer-cdc-museum/)[Fernbank Museum9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/fernbank-museum/)[High Museum of ArtMultiple dates until 09/30/2024](https://discoveratlanta.com/event/detail/high-museum-of-art/)[Jimmy Carter Presidential Library and Museum9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/jimmy-carter-presidential-library-and-museum/)[The Breman9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/the-breman/)[World of Coca-Cola9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/world-of-coca-cola/)[A Sunset Cruise aboard The Experience9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/a-sunset-cruise-aboard-the-experience/)[First Responder Appreciation Month9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/first-responder-appreciation-month/)[Geoffrey Chaucer\u2019s \u201CThe Canterbury Tales\u201D9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/geoffrey-chaucer-s-the-canterbury-tales/)[A Raisin In The Sun9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/a-raisin-in-the-sun/)[Atlanta Full-Day Wine Country Wine Tastings9/24/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/atlanta-full-day-wine-country-wine-tastings-2/)[Daddy Long Legs9/24/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/daddy-long-legs/)[Queer QuizMultiple dates until 10/21/2024](https://discoveratlanta.com/event/detail/queer-quiz/)[Food and Street Art Tour of the Atlanta BeltLineEvery Saturday until 11/16/2024](https://discoveratlanta.com/event/detail/food-and-street-art-tour-of-the-atlanta-beltline/)[History with Chutzpah9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/history-with-chutzpah/)[Black Child Book Fair10/19/24](https://discoveratlanta.com/event/detail/black-child-book-fair/)[Eatonton Mistletoe Market11/9/24](https://discoveratlanta.com/event/detail/eatonton-mistletoe-market/)[Barks and Bites: Doggie Crawl on the BeltLineMultiple dates until 11/17/2024](https://discoveratlanta.com/event/detail/barks-and-bites-doggie-crawl-on-the-beltline/)[Best Artist Ever9/24/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/best-artist-ever/)[Christmas with The Lanier Chamber Singers9/24/24 \u2013 12/6/24](https://discoveratlanta.com/event/detail/christmas-with-the-lanier-chamber-singers/)[Holiday Juggling Show9/24/24 \u2013 11/16/24](https://discoveratlanta.com/event/detail/holiday-juggling-show/)[NUTCRACKER! Magical Christmas Ballet9/24/24 \u2013 12/20/24](https://discoveratlanta.com/event/detail/nutcracker-magical-christmas-ballet/)[Candlelight Concerts 20249/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/candlelight-concerts-2024/)[WQEB 97.3 DINE \u0026#038; DONATE, a Food \u0026#038; Clothing Drive9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/wqeb-97-3-dine-donate-a-food-clothing-drive/)[Adopt Any Senior Pet for FREE in 20249/24/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/adopt-any-senior-pet-for-free-in-2024/)[Bottomless BrunchEvery Sunday until 12/29/2024](https://discoveratlanta.com/event/detail/bottomless-brunch/)[(ad)Vantage Point9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/ad-vantage-point/)[Next Level Boys Academy Mentoring9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/next-level-boys-academy-mentoring/)[UrbanTree After Dark: A Disco Pop-Up Bar9/24/24 \u2013 10/4/24](https://discoveratlanta.com/event/detail/urbantree-after-dark-a-disco-pop-up-bar/)[Free Self-guided Street Art Walking Tours9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/free-self-guided-street-art-walking-tours/)[Midnight Basketball League: Season 6Multiple days until 09/30/2024](https://discoveratlanta.com/event/detail/midnight-basketball-league-season-6/)[Wing \u0026#038; Rock Fest9/24/24 \u2013 3/23/25](https://discoveratlanta.com/event/detail/wing-rock-fest/)[Yoga at Van Gogh: The Immersive Experience9/24/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/yoga-at-van-gogh-the-immersive-experience-2/)[Punchline Comedy Night9/24/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/punchline-comedy-night/)[Punch Card Machine Demo @ CMoA9/24/24 \u2013 12/7/24](https://discoveratlanta.com/event/detail/punch-card-machine-demo-cmoa/)[Atlanta Comic Convention9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/atlanta-comic-convention/)[Broad Street Boardwalk Wednesday Lunchtime SeriesEvery Wednesday until 10/02/2024](https://discoveratlanta.com/event/detail/broad-street-boardwalk-wednesday-lunchtime-series/)[Sip \u0026#038; Scent: A Candle-Making Workshop9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/sip-scent-a-candle-making-workshop-2/)[Electronics Recycling Drive9/24/24 \u2013 11/2/24](https://discoveratlanta.com/event/detail/electronics-recycling-drive-2/)[Free Yoga Classes on the Broad Street BoardwalkEvery Saturday until 09/28/2024](https://discoveratlanta.com/event/detail/free-yoga-classes-on-the-broad-street-boardwalk/)[Protect Yo Energy Con9/24/24 \u2013 11/9/24](https://discoveratlanta.com/event/detail/protect-yo-energy-con/)[Weekend Creekwater Alpaca Farm Barn Tour9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/weekend-creekwater-alpaca-farm-barn-tour-2/)[Pick Your Perks Group Promotion9/24/24 \u2013 5/31/25](https://discoveratlanta.com/event/detail/pick-your-perks-group-promotion/)[Create Your Own Scented Candle!9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/create-your-own-scented-candle/)[Diva Royale Drag Queen Show9/24/24 \u2013 12/14/24](https://discoveratlanta.com/event/detail/diva-royale-drag-queen-show/)[Little Acorns9/24/24 \u2013 10/22/24](https://discoveratlanta.com/event/detail/little-acorns/)[Jazzy Thursdays and 1/2 off Bottles of WineEvery Thursday until 12/26/2024](https://discoveratlanta.com/event/detail/jazzy-thursdays-and-1-2-off-bottles-of-wine/)[Sunset Sessions Presents DJ Mike Webster \u0026#038; DJ Xzac9/24/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/sunset-sessions-presents-dj-mike-webster-dj-xzac/)[Rise and Dine with the Animals: PandasMultiple dates until 09/28/2024](https://discoveratlanta.com/event/detail/rise-and-dine-with-the-animals-pandas/)[Stand \u0026#038; Witness: Art in the Time of COVID-199/24/24 \u2013 10/24/24](https://discoveratlanta.com/event/detail/stand-witness-art-in-the-time-of-covid-19/)[Beginner\u2019s Salsa Lessons @ PasoFino Dance Studio9/24/24 \u2013 6/30/25](https://discoveratlanta.com/event/detail/beginner-s-salsa-lessons-pasofino-dance-studio/)[Tyler Mitchell: Idyllic Space9/24/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/tyler-mitchell-idyllic-space/)[ZooDunit Mystery SeriesMultiple dates until 10/11/2024](https://discoveratlanta.com/event/detail/zoodunit-mystery-series/)[Jazzy Thursday\u2019s at PricciEvery Thursday until 12/26/2024](https://discoveratlanta.com/event/detail/jazzy-thursday-s-at-pricci/)[Banfi MondaysEvery Monday until 12/30/2024](https://discoveratlanta.com/event/detail/banfi-mondays/)[Candlelight: A Tribute to Whitney Houston9/24/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/candlelight-a-tribute-to-whitney-houston-2/)[ARMHC To Host 15th Annual Gala9/24/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/armhc-to-host-15th-annual-gala/)[Dining in the Dark9/24/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/dining-in-the-dark/)[The Gaze9/24/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/the-gaze/)[Latin Brunch w/ Live Music \u0026#038; $3 Mimosas \u002B SangriasEvery Sunday until 12/01/2024](https://discoveratlanta.com/event/detail/latin-brunch-w-live-music-3-mimosas-sangrias/)[Titanic: An Immersive Voyage9/24/24 \u2013 1/22/25](https://discoveratlanta.com/event/detail/titanic-an-immersive-voyage/)[Improvised Dungeons \u0026#038; Dragons Returns!Every Friday until 09/27/2024](https://discoveratlanta.com/event/detail/improvised-dungeons-dragons-returns/)[Trivia Night at Wild Leap AtlantaEvery Tuesday until 11/19/2024](https://discoveratlanta.com/event/detail/trivia-night-at-wild-leap-atlanta/)[Elements- Surya Ensemble9/24/24 \u2013 10/10/24](https://discoveratlanta.com/event/detail/elements-surya-ensemble/)[Your Great Neighbor Deal!9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/your-great-neighbor-deal/)[Felt Street9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/felt-street/)[Felt Street Now Showing at Dad\u2019s Garage!Every Saturday until 09/28/2024](https://discoveratlanta.com/event/detail/felt-street-now-showing-at-dad-s-garage/)[Test Your Trivia Knowledge at SocialBites Food HubEvery Monday until 09/30/2024](https://discoveratlanta.com/event/detail/test-your-trivia-knowledge-at-socialbites-food-hub/)[Visit SocialBites Every Wednesday for Poker NightEvery Wednesday until 09/25/2024](https://discoveratlanta.com/event/detail/visit-socialbites-every-wednesday-for-poker-night/)[\u201CMix It Up\u201D Ladies NightEvery Wednesday until 12/25/2024](https://discoveratlanta.com/event/detail/mix-it-up-ladies-night/)[The Dinner Detective Comedy Mystery Dinner Show \u2013Every Saturday until 10/26/2024](https://discoveratlanta.com/event/detail/the-dinner-detective-comedy-mystery-dinner-show/)[Golden Opportunity: Botanical Illustration9/24/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/golden-opportunity-botanical-illustration/)[Chastian Challenge9/24/24 \u2013 10/15/24](https://discoveratlanta.com/event/detail/chastian-challenge/)[The Mill Town Square Dinners9/24/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/the-mill-town-square-dinners/)[Hold The Line Vol 2: Celebrating Black Masculinity9/24/24 \u2013 10/26/24](https://discoveratlanta.com/event/detail/hold-the-line-vol-2-celebrating-black-masculinity/)[Kids\u2019 Holiday Show Creation Program9/24/24 \u2013 11/30/24](https://discoveratlanta.com/event/detail/kids-holiday-show-creation-program/)[Fright Fest Atlanta9/24/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/fright-fest-atlanta/)[Face Your Fears at NETHERWORLD Haunted House!9/24/24 \u2013 11/9/24](https://discoveratlanta.com/event/detail/face-your-fears-at-netherworld-haunted-house/)[Mystery Monday \u2013 Drag ShowMultiple dates until 11/11/2024](https://discoveratlanta.com/event/detail/mystery-monday-drag-show/)[Cat in the Hat4/8/25 \u2013 5/25/25](https://discoveratlanta.com/event/detail/cat-in-the-hat/)[Justin Timberlake Live at State Farm ArenaMultiple dates until 11/16/2024](https://discoveratlanta.com/event/detail/justin-timberlake-live-at-state-farm-arena/)[Trick-or-Treat at CMOAMultiple dates until 10/26/2024](https://discoveratlanta.com/event/detail/trick-or-treat-at-cmoa/)[Editors\u2019 Pick September24Tuesday College Football Hall of Fame9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/college-football-hall-of-fame/)[Editors\u2019 Pick September24Tuesday Pumpkin Festival9/24/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/pumpkin-festival/)[Editors\u2019 Pick September28Saturday Sweet Auburn Music Fest9/28/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/sweet-auburn-music-fest/)[Editors\u2019 Pick October01Tuesday Scarecrows in the GardenMultiple days until 11/03/2024](https://discoveratlanta.com/event/detail/scarecrows-in-the-garden/)[What\u2019s Hot September26Thursday Georgia Latino International Film Festival9/26/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/georgia-latino-international-film-festival/)[What\u2019s Hot September27Friday 3rd Annual Baldie Con9/27/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/3rd-annual-baldie-con/)[What\u2019s Hot September27Friday Atlanta Food Staples Wine Dinner9/27/24](https://discoveratlanta.com/event/detail/atlanta-food-staples-wine-dinner/)[What\u2019s Hot September28Saturday Westside Beer, Bourbon \u0026#038; Bites Fest9/28/24](https://discoveratlanta.com/event/detail/westside-beer-bourbon-bites-fest/)[What\u2019s Hot September30Monday Atlanta Art Week9/30/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/atlanta-art-week/)[What\u2019s Hot October01Tuesday Games Week Georgia10/1/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/games-week-georgia/)[What\u2019s Hot October04Friday DreamHack Atlanta10/4/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/dreamhack-atlanta/)[What\u2019s Hot October04Friday Haunted Seas10/4/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/haunted-seas/)[What\u2019s Hot October05Saturday Candler Park Fall Festival10/5/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/candler-park-fall-festival/)[What\u2019s Hot October19Saturday Little 5 Points Halloween Festival \u0026#038; Parade10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/little-5-points-halloween-festival-parade/)[What\u2019s Hot October19Saturday Stone Mountain Scottish Festival and Highland Games10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/stone-mountain-scottish-festival-and-highland-games/)[What\u2019s Hot October20Sunday Goblins in the Garden10/20/24](https://discoveratlanta.com/event/detail/goblins-in-the-garden/)[What\u2019s Hot November02Saturday Cabbagetown Chomp \u0026#038; Stomp11/2/24](https://discoveratlanta.com/event/detail/cabbagetown-chomp-stomp/)[What\u2019s Hot November16Saturday Atlanta Chili Cookoff11/16/24](https://discoveratlanta.com/event/detail/atlanta-chili-cookoff/)[What\u2019s Hot November28Thursday Invesco QQQ Thanksgiving Day Half Marathon11/28/24](https://discoveratlanta.com/event/detail/invesco-qqq-thanksgiving-day-half-marathon/)[What\u2019s Hot September24Tuesday The Horizon of Khufu9/24/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/the-horizon-of-khufu/)[What\u2019s Hot October17Thursday Usher Performing in AtlantaMultiple dates until 10/20/2024](https://discoveratlanta.com/event/detail/usher-performing-in-atlanta/)[What\u2019s Hot September24Tuesday Atlanta History Center9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/atlanta-history-center/)[What\u2019s Hot September24Tuesday Georgia Aquarium9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/georgia-aquarium/)[September24Tuesday La Boheme9/24/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/la-boheme/)[September24Tuesday Pancho Rabbit and the Coyote9/24/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/pancho-rabbit-and-the-coyote/)[September24Tuesday RENT \u2013 The Musical9/24/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/rent-the-musical/)[September28Saturday Elevate AtlantaEvery weekend until 10/13/2024](https://discoveratlanta.com/event/detail/elevate-atlanta/)[September24Tuesday Little Shop of Horrors9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/little-shop-of-horrors/)[September24Tuesday Placita Latina Exhibition9/24/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/placita-latina-exhibition/)[September24Tuesday Walt DisneyxUNCF Scholar Program Now Open!9/24/24 \u2013 10/1/24](https://discoveratlanta.com/event/detail/walt-disneyxuncf-scholar-program-now-open/)[September28Saturday Oktoberfest at Der BiergartenEvery Saturday until 10/26/2024](https://discoveratlanta.com/event/detail/oktoberfest-at-der-biergarten/)[October05Saturday Sangria n\u2019 Scents DIY Candle Workshop9/24/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/sangria-n-scents-diy-candle-workshop/)[September24Tuesday Indulge9/22/24 \u2013 9/24/24](https://discoveratlanta.com/event/detail/indulge/)[September24Tuesday Bulla Atlanta 7th Anniversary9/24/24](https://discoveratlanta.com/event/detail/bulla-atlanta-7th-anniversary/)[September24Tuesday DISCOVER ADAC9/24/24 \u2013 9/26/24](https://discoveratlanta.com/event/detail/discover-adac/)[September24Tuesday EBONY HBCU Yard Fest9/24/24](https://discoveratlanta.com/event/detail/ebony-hbcu-yard-fest/)[September24Tuesday Edgewood Comedy Night at Georgia Beer Garden9/24/24](https://discoveratlanta.com/event/detail/edgewood-comedy-night-at-georgia-beer-garden/)[September24Tuesday F.A.C.E. Fulton County: ATLSFW9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-atlsfw/)[September24Tuesday Hey there! Connections9/24/24](https://discoveratlanta.com/event/detail/hey-there-connections/)[September24Tuesday KultureCity Fundraiser at Lazy Betty9/24/24](https://discoveratlanta.com/event/detail/kulturecity-fundraiser-at-lazy-betty/)[September24Tuesday Meka James Book Launch9/24/24](https://discoveratlanta.com/event/detail/meka-james-book-launch/)[September24Tuesday Oktoberfest Tasting Experience9/24/24](https://discoveratlanta.com/event/detail/oktoberfest-tasting-experience/)[September24Tuesday September Networking Social9/24/24](https://discoveratlanta.com/event/detail/september-networking-social/)[September24Tuesday \u201CAt Home in France with Timothy Corrigan\u201D at ADAC9/24/24](https://discoveratlanta.com/event/detail/at-home-in-france-with-timothy-corrigan-at-adac/)[September24Tuesday \u201CDesigning with a Botanical Accent\u201D at ADAC9/24/24](https://discoveratlanta.com/event/detail/designing-with-a-botanical-accent-at-adac/)[September24Tuesday \u201CThe Power of Teamwork in Residential Design\u201D ADAC9/24/24](https://discoveratlanta.com/event/detail/the-power-of-teamwork-in-residential-design-adac/)[September25Wednesday BEAT: Steve Vai, Adrian Belew, Tony Levin, Danny Carey9/25/24](https://discoveratlanta.com/event/detail/beat-steve-vai-adrian-belew-tony-levin-danny-carey/)[September25Wednesday Community Movie Night presented by Disney on Ice9/25/24](https://discoveratlanta.com/event/detail/community-movie-night-presented-by-disney-on-ice/)[September25Wednesday Curated Conversations \u2013 Intro to Wellness9/25/24](https://discoveratlanta.com/event/detail/curated-conversations-intro-to-wellness/)[September25Wednesday Kings of Leon9/25/24](https://discoveratlanta.com/event/detail/kings-of-leon/)[September25Wednesday Leanne Morgan \u2013 What in the World?! \u2013 Book Event9/25/24](https://discoveratlanta.com/event/detail/leanne-morgan-what-in-the-world-book-event/)[September25Wednesday Los Bravos Night9/25/24](https://discoveratlanta.com/event/detail/los-bravos-night/)[September25Wednesday Make @ MODA: Beats \u0026#038; Bricks9/25/24](https://discoveratlanta.com/event/detail/make-moda-beats-bricks/)[September25Wednesday Oktoberfest Crafts \u0026#038; Romcom Night9/25/24](https://discoveratlanta.com/event/detail/oktoberfest-crafts-romcom-night/)[September26Thursday Bradley Tusk Talks about Vote With Your Phone9/26/24](https://discoveratlanta.com/event/detail/bradley-tusk-talks-about-vote-with-your-phone/)[September26Thursday Disney on Ice at Gas South Arena9/26/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/disney-on-ice-at-gas-south-arena/)[September26Thursday Fall Native Plant Sale9/26/24](https://discoveratlanta.com/event/detail/fall-native-plant-sale/)[September26Thursday Geology Walk and Talk9/26/24](https://discoveratlanta.com/event/detail/geology-walk-and-talk/)[September26Thursday Les Dames d\u2019 Escoffier present \u201CPerfect Pairings\u201D9/26/24](https://discoveratlanta.com/event/detail/les-dames-d-escoffier-present-perfect-pairings/)[September26Thursday Make It Last Forever \u2013 Keith Sweat, Pretty Ricky9/26/24](https://discoveratlanta.com/event/detail/make-it-last-forever-keith-sweat-pretty-ricky/)[September26Thursday New South Kitchen\u2019s Wine Dinner Series9/26/24](https://discoveratlanta.com/event/detail/new-south-kitchen-s-wine-dinner-series/)[September26Thursday Piedmont Park Celebrates Park\u2019s 120th Birthday9/26/24](https://discoveratlanta.com/event/detail/piedmont-park-celebrates-park-s-120th-birthday/)[September26Thursday Piedmont Park Conservancy 35th Anniversary9/26/24](https://discoveratlanta.com/event/detail/piedmont-park-conservancy-35th-anniversary/)[September26Thursday Southeast Community Health Worker Network Summit9/26/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/southeast-community-health-worker-network-summit-2/)[September26Thursday \u201CTHE MARK OF ZORRO\u201D (1920) w/ organist Ken Double9/26/24](https://discoveratlanta.com/event/detail/the-mark-of-zorro-1920-w-organist-ken-double/)[September27Friday 1259 Burlesque Show in Atlanta9/27/24](https://discoveratlanta.com/event/detail/1259-burlesque-show-in-atlanta/)[September27Friday Arte Agave Atlanta9/27/24](https://discoveratlanta.com/event/detail/arte-agave-atlanta/)[September27Friday Atlanta Tattoo Festival9/27/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/atlanta-tattoo-festival/)[September27Friday Autumn Art Market \u0026#038; Pop-Up Party at Snack Bar9/27/24](https://discoveratlanta.com/event/detail/autumn-art-market-pop-up-party-at-snack-bar/)[September27Friday BUNT Live9/27/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/bunt-live/)[September27Friday Candlelight: Tribute to Beyonc\u00E99/27/24](https://discoveratlanta.com/event/detail/candlelight-tribute-to-beyonc/)[September27Friday Centennial Yards Presents Friday Vibe9/27/24](https://discoveratlanta.com/event/detail/centennial-yards-presents-friday-vibe/)[September27Friday Chromeo \u0026#038; The Midnight9/27/24](https://discoveratlanta.com/event/detail/chromeo-the-midnight/)[September27Friday Howls Moving Castle screening at Aurora Cineplex9/27/24 \u2013 10/3/24](https://discoveratlanta.com/event/detail/howls-moving-castle-screening-at-aurora-cineplex/)[September27Friday Movies at The Strand: Tootsie (1982)9/27/24](https://discoveratlanta.com/event/detail/movies-at-the-strand-tootsie-1982/)[September27Friday SEPHORia 20249/27/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/sephoria-2024/)[September27Friday THE REFLEX PERFORMING LIVE on FTE\u2019s BACK PORCH!9/27/24](https://discoveratlanta.com/event/detail/the-reflex-performing-live-on-fte-s-back-porch/)[September27Friday Unleash Your Inner Worth: Women\u2019s Summit9/27/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/unleash-your-inner-worth-women-s-summit/)[September28Saturday 9th Annual Refuge Coffee Run: 5k and Mile Fun Run9/28/24](https://discoveratlanta.com/event/detail/9th-annual-refuge-coffee-run-5k-and-mile-fun-run/)[September28Saturday AIDS Walk Atlanta Music Festival \u0026#038; 5K Run9/28/24](https://discoveratlanta.com/event/detail/aids-walk-atlanta-music-festival-5k-run/)[September28Saturday AMADOR Fest9/28/24](https://discoveratlanta.com/event/detail/amador-fest/)[September28Saturday ATL Women\u2019s Polo Series9/28/24](https://discoveratlanta.com/event/detail/atl-women-s-polo-series/)[September28Saturday Afro Street Market Festival9/28/24](https://discoveratlanta.com/event/detail/afro-street-market-festival/)[September28Saturday Afro Street Market Festival: Celebrate Culture9/28/24](https://discoveratlanta.com/event/detail/afro-street-market-festival-celebrate-culture/)[September28Saturday An Evening with CAKE9/28/24](https://discoveratlanta.com/event/detail/an-evening-with-cake/)[September28Saturday Autism Speaks WALK9/28/24](https://discoveratlanta.com/event/detail/autism-speaks-walk/)[September28Saturday Carrollton Pride Festival 20249/28/24](https://discoveratlanta.com/event/detail/carrollton-pride-festival-2024/)[September28Saturday Divinely Inspired Victoriously Aware High Tea9/28/24](https://discoveratlanta.com/event/detail/divinely-inspired-victoriously-aware-high-tea/)[September28Saturday East Atlanta Strut Dog Adoption Event9/28/24](https://discoveratlanta.com/event/detail/east-atlanta-strut-dog-adoption-event/)[September28Saturday Exercise Your Faith 5k Walk/Run9/28/24](https://discoveratlanta.com/event/detail/exercise-your-faith-5k-walk-run/)[September28Saturday Fall KickBack ~ Block Party9/28/24](https://discoveratlanta.com/event/detail/fall-kickback-block-party/)[September28Saturday French Market Atlanta9/28/24](https://discoveratlanta.com/event/detail/french-market-atlanta/)[September28Saturday Georgia Southern Eagles at Georgia State Panthers Football9/28/24](https://discoveratlanta.com/event/detail/georgia-southern-eagles-at-georgia-state-panthers-football/)[September28Saturday Give Sanctuary Festival9/28/24](https://discoveratlanta.com/event/detail/give-sanctuary-festival/)[September28Saturday Half-Day Wine Tastings in the North Georgia9/28/24](https://discoveratlanta.com/event/detail/half-day-wine-tastings-in-the-north-georgia/)[September28Saturday JC100 Film Festival9/28/24](https://discoveratlanta.com/event/detail/jc100-film-festival/)[September28Saturday JC100 Film Festival-President Carter\u2019s 100th Bday9/28/24](https://discoveratlanta.com/event/detail/jc100-film-festival-president-carter-s-100th-bday/)[September28Saturday Jigsaw Dating\u00AE : Atlanta Coffee Shop Speed Dating9/28/24](https://discoveratlanta.com/event/detail/jigsaw-dating-atlanta-coffee-shop-speed-dating/)[September28Saturday Members Mart at The Overline9/28/24](https://discoveratlanta.com/event/detail/members-mart-at-the-overline/)[September28Saturday PERC Coffee Dog Adoption Event9/28/24](https://discoveratlanta.com/event/detail/perc-coffee-dog-adoption-event/)[September28Saturday Pixel Putt \u2013 A Mini Golf Tournament at CMoA9/28/24](https://discoveratlanta.com/event/detail/pixel-putt-a-mini-golf-tournament-at-cmoa/)[September28Saturday Plant-Oberfest: Plants \u0026#038; Oktoberfest at 6 Bridges9/28/24](https://discoveratlanta.com/event/detail/plant-oberfest-plants-oktoberfest-at-6-bridges/)[September28Saturday Rival Entertainment Presents an Evening with CAKE9/28/24](https://discoveratlanta.com/event/detail/rival-entertainment-presents-an-evening-with-cake/)[September28Saturday Roswell Wine FestivalMultiple dates until 09/29/2024](https://discoveratlanta.com/event/detail/roswell-wine-festival/)[September28Saturday Sangria n\u2019 Skincare DIY Facial Serum Workshop9/28/24](https://discoveratlanta.com/event/detail/sangria-n-skincare-diy-facial-serum-workshop/)[September28Saturday The Four Horsemen \u2013 Album Quality Metallica Tribut9/28/24](https://discoveratlanta.com/event/detail/the-four-horsemen-album-quality-metallica-tribut/)[September28Saturday The Original Atlanta Korean Festival 20249/28/24](https://discoveratlanta.com/event/detail/the-original-atlanta-korean-festival-2024/)[September28Saturday Unleash Your Inner Worth 1st Annual Black Tie Gala9/28/24](https://discoveratlanta.com/event/detail/unleash-your-inner-worth-1st-annual-black-tie-gala/)[September28Saturday Unleash Your Inner Worth 1st Annual Black-Tie Gala9/28/24](https://discoveratlanta.com/event/detail/unleash-your-inner-worth-1st-annual-black-tie-gala-2/)[September28Saturday Westside Chattahoochee Paddle 20249/28/24](https://discoveratlanta.com/event/detail/westside-chattahoochee-paddle-2024/)[September29Sunday ELEVATED RHYTHMS9/29/24](https://discoveratlanta.com/event/detail/elevated-rhythms/)[September29Sunday Ethel\u2019s Birthday Jam Session at Elsewhere Brewing9/29/24](https://discoveratlanta.com/event/detail/ethel-s-birthday-jam-session-at-elsewhere-brewing/)[September29Sunday FACEs of Fashion: A Celestial Experience9/29/24](https://discoveratlanta.com/event/detail/faces-of-fashion-a-celestial-experience/)[September29Sunday Georgia Plant Swap \u002B Sale Peachy Autumn9/29/24](https://discoveratlanta.com/event/detail/georgia-plant-swap-sale-peachy-autumn/)[September29Sunday Silent Film at The Strand: College (1927)9/29/24](https://discoveratlanta.com/event/detail/silent-film-at-the-strand-college-1927/)[September29Sunday THE FACEs OF FASHION:9/29/24](https://discoveratlanta.com/event/detail/the-faces-of-fashion/)[September29Sunday Tea Party Socials9/29/24](https://discoveratlanta.com/event/detail/tea-party-socials/)[September29Sunday Village Nanny Community Brand Activation9/29/24](https://discoveratlanta.com/event/detail/village-nanny-community-brand-activation/)[September30Monday Corps De Ballet 26th Fashion Show \u0026#038; Luncheon9/30/24](https://discoveratlanta.com/event/detail/corps-de-ballet-26th-fashion-show-luncheon/)[September30Monday F.A.C.E. Fulton County: Behind the Seams9/30/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-behind-the-seams/)[September30Monday F.A.C.E. Fulton County: Fulton County NAP Exhibit9/30/24 \u2013 10/25/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-fulton-county-nap-exhibit/)[September30Monday Fletcher \u2013 In Search Of The Antidote Tour9/30/24](https://discoveratlanta.com/event/detail/fletcher-in-search-of-the-antidote-tour/)[September30Monday Military Influencer Conference9/30/24 \u2013 10/3/24](https://discoveratlanta.com/event/detail/military-influencer-conference/)[September30Monday Tequila Dinner Featuring Pedro Furtivo9/30/24](https://discoveratlanta.com/event/detail/tequila-dinner-featuring-pedro-furtivo/)[October01Tuesday A Haunted Lake Lanier Experience10/1/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/a-haunted-lake-lanier-experience/)[October01Tuesday Brooks Nielsen10/1/24](https://discoveratlanta.com/event/detail/brooks-nielsen/)[October01Tuesday Cue \u0026#038; Cocktails, Presented by Taste of Atlanta10/1/24](https://discoveratlanta.com/event/detail/cue-cocktails-presented-by-taste-of-atlanta/)[October01Tuesday Dine With Gratitude \u0026#038; Support Giving Kitchen10/1/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/dine-with-gratitude-support-giving-kitchen/)[October01Tuesday Games Week Georgia 202410/1/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/games-week-georgia-2024/)[October01Tuesday InnovATL202410/1/24 \u2013 10/19/24](https://discoveratlanta.com/event/detail/innovatl2024/)[October01Tuesday The Moth GrandSLAM: FIRED UP10/1/24](https://discoveratlanta.com/event/detail/the-moth-grandslam-fired-up/)[October01Tuesday The Texas Chainsaw Massacre (1974)Multiple dates until 10/02/2024](https://discoveratlanta.com/event/detail/the-texas-chainsaw-massacre-1974/)[October02Wednesday Atlanta Fashion Week 202410/2/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/atlanta-fashion-week-2024/)[October02Wednesday CF Montr\u00E9al at Atlanta United10/2/24](https://discoveratlanta.com/event/detail/cf-montral-at-atlanta-united/)[October02Wednesday Curated Conversations: Finding Work-Life Balance10/2/24](https://discoveratlanta.com/event/detail/curated-conversations-finding-work-life-balance/)[October03Thursday A Third Way10/3/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/a-third-way/)[October03Thursday Atlanta Art Fair10/3/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/atlanta-art-fair/)[October03Thursday Charli XCX and Troye Sivan10/3/24](https://discoveratlanta.com/event/detail/charli-xcx-and-troye-sivan/)[October03Thursday Every Girl Shines Mentor Mixer10/3/24](https://discoveratlanta.com/event/detail/every-girl-shines-mentor-mixer/)[October03Thursday F.A.C.E. Fulton County: Atlanta Fashion Week10/3/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-atlanta-fashion-week/)[October03Thursday Four Scythes Haunted Attraction10/3/24 \u2013 10/26/24](https://discoveratlanta.com/event/detail/four-scythes-haunted-attraction/)[October03Thursday Mud by Theater Emory10/3/24 \u2013 10/12/24](https://discoveratlanta.com/event/detail/mud-by-theater-emory/)[October03Thursday Navigating Youth Sports Today As Parents (Free)10/3/24](https://discoveratlanta.com/event/detail/navigating-youth-sports-today-as-parents-free/)[October03Thursday Scary Movies at The Strand: Scream (1996)10/3/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-scream-1996/)[October04Friday Atlanta SCI-FI Film Festival10/4/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/atlanta-sci-fi-film-festival/)[October04Friday Conan Gray10/4/24](https://discoveratlanta.com/event/detail/conan-gray/)[October04Friday Elizabeth Gilbert BIG MAGIC Creativity Retreat10/4/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/elizabeth-gilbert-big-magic-creativity-retreat/)[October04Friday Expo Contratista10/4/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/expo-contratista/)[October04Friday How To Train Your Dragon: Live In Concert10/4/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/how-to-train-your-dragon-live-in-concert/)[October04Friday LAYTON GIORDANI10/4/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/layton-giordani/)[October04Friday Luxury Date Night with Live Music by Trey Daniels10/4/24](https://discoveratlanta.com/event/detail/luxury-date-night-with-live-music-by-trey-daniels/)[October04Friday SCAD Presents: aja monet10/4/24](https://discoveratlanta.com/event/detail/scad-presents-aja-monet/)[October04Friday Saxsquatch Rave10/4/24](https://discoveratlanta.com/event/detail/saxsquatch-rave/)[October04Friday Signature Sounds at The Waldorf: Michelle Winters10/4/24](https://discoveratlanta.com/event/detail/signature-sounds-at-the-waldorf-michelle-winters/)[October04Friday The Purple Madness \u2013 Prince Tribute10/4/24](https://discoveratlanta.com/event/detail/the-purple-madness-prince-tribute/)[October05Saturday 20th Annual NAMIWalks Georgia10/5/24](https://discoveratlanta.com/event/detail/20th-annual-namiwalks-georgia/)[October05Saturday 6th Annual Grove Park Community 5K10/5/24](https://discoveratlanta.com/event/detail/6th-annual-grove-park-community-5k/)[October05Saturday Atlanta Light The Night10/5/24](https://discoveratlanta.com/event/detail/atlanta-light-the-night/)[October05Saturday Breast Cancer Line Dance Fundraiser 202410/5/24](https://discoveratlanta.com/event/detail/breast-cancer-line-dance-fundraiser-2024/)[October05Saturday Classical K-pop Concert10/5/24](https://discoveratlanta.com/event/detail/classical-k-pop-concert/)[October05Saturday Cops and Cones Motor Skills Course10/5/24](https://discoveratlanta.com/event/detail/cops-and-cones-motor-skills-course/)[October05Saturday EVECE Walk For Life 5k Walk/Run10/5/24](https://discoveratlanta.com/event/detail/evece-walk-for-life-5k-walk-run/)[October05Saturday F.A.C.E. Fulton County: Les Farfadais CELESTIAL10/5/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-les-farfadais-celestial/)[October05Saturday Fall Festival on PonceMultiple dates until 10/06/2024](https://discoveratlanta.com/event/detail/fall-festival-on-ponce/)[October05Saturday Fela! Under the Stars10/5/24](https://discoveratlanta.com/event/detail/fela-under-the-stars/)[October05Saturday Georgia Tasting Festival 202410/5/24](https://discoveratlanta.com/event/detail/georgia-tasting-festival-2024/)[October05Saturday Georgia Tech Football vs Duke10/5/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-duke/)[October05Saturday My Plus Size Wedding Expo10/5/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/my-plus-size-wedding-expo/)[October05Saturday Punch Card Machine Demo @ CMoA, October10/5/24](https://discoveratlanta.com/event/detail/punch-card-machine-demo-cmoa-october/)[October05Saturday Rise and Dine with the Animals: GorillasMultiple dates until 10/12/2024](https://discoveratlanta.com/event/detail/rise-and-dine-with-the-animals-gorillas/)[October05Saturday Roberta Lea Live10/5/24](https://discoveratlanta.com/event/detail/roberta-lea-live/)[October05Saturday Strand Ole Opry: Fa-BOO-lous Halloween Hits10/5/24](https://discoveratlanta.com/event/detail/strand-ole-opry-fa-boo-lous-halloween-hits/)[October06Sunday Cirque Kalabante \u2013 Afrique en Cirque10/6/24](https://discoveratlanta.com/event/detail/cirque-kalabante-afrique-en-cirque/)[October06Sunday Dean Owens Live10/6/24](https://discoveratlanta.com/event/detail/dean-owens-live/)[October06Sunday Fall SCRAPFest: a comedy fundraiser \u0026#038; craft fair10/6/24](https://discoveratlanta.com/event/detail/fall-scrapfest-a-comedy-fundraiser-craft-fair/)[October06Sunday Flower Power: A Fall Flower-Arranging Workshop10/6/24](https://discoveratlanta.com/event/detail/flower-power-a-fall-flower-arranging-workshop/)[October06Sunday Halloween Brick-or-Treat Monster Party10/6/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/halloween-brick-or-treat-monster-party/)[October06Sunday Jazz at The Strand: The Music of Louis Armstrong10/6/24](https://discoveratlanta.com/event/detail/jazz-at-the-strand-the-music-of-louis-armstrong/)[October06Sunday Maize \u0026#038; Mingle10/6/24](https://discoveratlanta.com/event/detail/maize-mingle/)[October06Sunday OYE Fest10/6/24](https://discoveratlanta.com/event/detail/oye-fest/)[October06Sunday Oktoberfest at Chateau Elan10/6/24](https://discoveratlanta.com/event/detail/oktoberfest-at-chateau-elan/)[October06Sunday Side by Side by Side: A Sondheim Celebration10/6/24](https://discoveratlanta.com/event/detail/side-by-side-by-side-a-sondheim-celebration/)[October06Sunday Sip \u0026#038; Style: A Fall Fashion \u0026#038; Wine Showcase10/6/24](https://discoveratlanta.com/event/detail/sip-style-a-fall-fashion-wine-showcase/)[October06Sunday Sunday Jazz Brunch at One Flew South BeltLine10/6/24](https://discoveratlanta.com/event/detail/sunday-jazz-brunch-at-one-flew-south-beltline/)[October07Monday Association for Talent Development\u2019s OrgDev2410/7/24 \u2013 10/9/24](https://discoveratlanta.com/event/detail/association-for-talent-development-s-orgdev24/)[October07Monday Avant South10/7/24](https://discoveratlanta.com/event/detail/avant-south/)[October07Monday F.A.C.E. Fulton County: DIY Workshop Trendy Totes10/7/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-diy-workshop-trendy-totes/)[October07Monday F.A.C.E. Fulton County: Workshop Trendy Totes10/7/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-workshop-trendy-totes/)[October07Monday Nygren Placemaking Conference10/7/24 \u2013 10/9/24](https://discoveratlanta.com/event/detail/nygren-placemaking-conference/)[October07Monday Open House: Focused Community Strategies10/7/24 \u2013 10/8/24](https://discoveratlanta.com/event/detail/open-house-focused-community-strategies/)[October08Tuesday Venture Atlanta 202410/8/24 \u2013 10/9/24](https://discoveratlanta.com/event/detail/venture-atlanta-2024/)[October09Wednesday Atlanta miniFEST10/9/24 \u2013 10/10/24](https://discoveratlanta.com/event/detail/atlanta-minifest/)[October09Wednesday Homeshake10/9/24](https://discoveratlanta.com/event/detail/homeshake/)[October09Wednesday Inclusive Health Equity Conference at Agnes Scott10/9/24](https://discoveratlanta.com/event/detail/inclusive-health-equity-conference-at-agnes-scott/)[October09Wednesday Walter Parks \u0026#038; The Unlawful Assembly10/9/24](https://discoveratlanta.com/event/detail/walter-parks-the-unlawful-assembly/)[October10Thursday ART \u0026#038; SURVIVAL: Honoring Indigenous People\u2019s Day10/10/24](https://discoveratlanta.com/event/detail/art-survival-honoring-indigenous-people-s-day/)[October10Thursday Atlanta UserCon10/10/24](https://discoveratlanta.com/event/detail/atlanta-usercon/)[October10Thursday Scary Movies at The Strand: Young Frankenstein10/10/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-young-frankenstein/)[October11Friday 2024 Georgia Aquarium Pride Kickoff10/11/24](https://discoveratlanta.com/event/detail/2024-georgia-aquarium-pride-kickoff/)[October11Friday Aerosmith: PEACE OUT The Farewell Tour with The Black Crowes10/11/24](https://discoveratlanta.com/event/detail/aerosmith-peace-out-the-farewell-tour-with-the-black-crowes/)[October11Friday Akoni Eyewear \u0026#038; Sunglasses \u2013 Private Event10/11/24 \u2013 10/12/24](https://discoveratlanta.com/event/detail/akoni-eyewear-sunglasses-private-event/)[October11Friday Dana and Alden10/11/24](https://discoveratlanta.com/event/detail/dana-and-alden/)[October11Friday F.A.C.E. Fulton County: ATL Fashion \u0026#038; Polo Classic10/11/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/f-a-c-e-fulton-county-atl-fashion-polo-classic/)[October11Friday FashionFest 202410/11/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/fashionfest-2024/)[October11Friday No Words ATL Instrumental Music Festival10/11/24 \u2013 10/14/24](https://discoveratlanta.com/event/detail/no-words-atl-instrumental-music-festival/)[October11Friday NoWordsATL3.0 Traveling Instrumental Music Fest10/11/24 \u2013 10/14/24](https://discoveratlanta.com/event/detail/nowordsatl3-0-traveling-instrumental-music-fest/)[October11Friday Nurko United Tour10/11/24 \u2013 10/12/24](https://discoveratlanta.com/event/detail/nurko-united-tour/)[October11Friday Scary Movies at The Strand: Poltergeist (1982)10/11/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-poltergeist-1982/)[October11Friday Seven Days Band- A Tribute to Sting and The Police10/11/24](https://discoveratlanta.com/event/detail/seven-days-band-a-tribute-to-sting-and-the-police/)[October12Saturday BIG GAY BRUNCH Returns to Park Tavern10/12/24](https://discoveratlanta.com/event/detail/big-gay-brunch-returns-to-park-tavern/)[October12Saturday Cherokee Tales10/12/24](https://discoveratlanta.com/event/detail/cherokee-tales/)[October12Saturday Old Fourth Ward Arts FestivalMultiple dates until 10/13/2024](https://discoveratlanta.com/event/detail/old-fourth-ward-arts-festival/)[October12Saturday Spooky Mill10/12/24](https://discoveratlanta.com/event/detail/spooky-mill/)[October12Saturday Spooky Spirits 5K10/12/24](https://discoveratlanta.com/event/detail/spooky-spirits-5k/)[October12Saturday Taste \u0026#038; Brews Fall Festival10/12/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/taste-brews-fall-festival/)[November03Sunday Creative Caviar: A Unique Caviar Pairing Workshop10/13/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/creative-caviar-a-unique-caviar-pairing-workshop/)[October13Sunday Creed Fisher10/13/24](https://discoveratlanta.com/event/detail/creed-fisher/)[October13Sunday Halloween Spooktacular10/13/24](https://discoveratlanta.com/event/detail/halloween-spooktacular/)[October13Sunday Jeff Lynne\u2019s ELO10/13/24](https://discoveratlanta.com/event/detail/jeff-lynne-s-elo/)[October13Sunday Romeo and Juliet10/13/24](https://discoveratlanta.com/event/detail/romeo-and-juliet/)[October14Monday 26th Annual Special Needs Cobb Golf Invitational10/14/24](https://discoveratlanta.com/event/detail/26th-annual-special-needs-cobb-golf-invitational/)[October14Monday World Commerce and Contracting Americas Summit10/14/24 \u2013 10/15/24](https://discoveratlanta.com/event/detail/world-commerce-and-contracting-americas-summit/)[October15Tuesday QS Discover for Masters and MBA in Atlanta10/15/24](https://discoveratlanta.com/event/detail/qs-discover-for-masters-and-mba-in-atlanta/)[October16Wednesday Feed Your Senses Concert: Nikki Speake/Anna Caller10/16/24](https://discoveratlanta.com/event/detail/feed-your-senses-concert-nikki-speake-anna-caller/)[October17Thursday I Know What You Did Last Summer (1997)10/17/24](https://discoveratlanta.com/event/detail/i-know-what-you-did-last-summer-1997/)[October17Thursday Midtown Movie Night: Nightmare Before Christmas10/17/24](https://discoveratlanta.com/event/detail/midtown-movie-night-nightmare-before-christmas/)[October17Thursday Planned PEThood of Georgia Murder Mystery Dinner10/17/24](https://discoveratlanta.com/event/detail/planned-pethood-of-georgia-murder-mystery-dinner/)[October17Thursday ShaoConMultiple dates until 10/20/2024](https://discoveratlanta.com/event/detail/shaocon/)[October18Friday 1 Night with Diplo at Underground Atlanta on 10/1810/18/24](https://discoveratlanta.com/event/detail/1-night-with-diplo-at-underground-atlanta-on-10-18/)[October18Friday Comedy at The Strand: The Mary Mack Comedy Show10/18/24](https://discoveratlanta.com/event/detail/comedy-at-the-strand-the-mary-mack-comedy-show/)[October18Friday Jesse Lynn Madera and Jill Sobule Live10/18/24](https://discoveratlanta.com/event/detail/jesse-lynn-madera-and-jill-sobule-live/)[October18Friday Jessica Kirson10/18/24](https://discoveratlanta.com/event/detail/jessica-kirson/)[October18Friday Jill Sobule And Jesse Lynn Madera10/18/24 \u2013 10/19/24](https://discoveratlanta.com/event/detail/jill-sobule-and-jesse-lynn-madera/)[October18Friday Jos\u00E9 Gonz\u00E1lez10/18/24](https://discoveratlanta.com/event/detail/jos-gonzalez/)[October18Friday PeriCon(ATL) Menopause Transition Summit10/18/24](https://discoveratlanta.com/event/detail/pericon-atl-menopause-transition-summit/)[October19Saturday 25th Annual Trees Atlanta Tree Sale10/19/24](https://discoveratlanta.com/event/detail/25th-annual-trees-atlanta-tree-sale/)[October19Saturday Candlelight: A Haunted Evening of Halloween10/19/24](https://discoveratlanta.com/event/detail/candlelight-a-haunted-evening-of-halloween/)[October19Saturday Cars \u0026#038; \u2018Q for the Cause10/19/24](https://discoveratlanta.com/event/detail/cars-q-for-the-cause/)[October19Saturday Georgia Tech Football vs Notre Dame10/19/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-notre-dame/)[October19Saturday Jackson County Lane10/19/24](https://discoveratlanta.com/event/detail/jackson-county-lane/)[October19Saturday John Vincent III: The Roadtrip10/19/24](https://discoveratlanta.com/event/detail/john-vincent-iii-the-roadtrip/)[October19Saturday Macon Beer Fest10/19/24](https://discoveratlanta.com/event/detail/macon-beer-fest/)[October19Saturday NBAF Celebration Under the Stars10/19/24](https://discoveratlanta.com/event/detail/nbaf-celebration-under-the-stars/)[October19Saturday Paws and Pastries10/19/24](https://discoveratlanta.com/event/detail/paws-and-pastries/)[October19Saturday Pine Lake Lakefest10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/pine-lake-lakefest/)[October19Saturday Pride Showcase10/19/24 \u2013 10/20/24](https://discoveratlanta.com/event/detail/pride-showcase/)[October19Saturday Race for Rest 202410/19/24](https://discoveratlanta.com/event/detail/race-for-rest-2024/)[October19Saturday Safe and Secure Online \u2013 Cybersecurity for Kids10/19/24](https://discoveratlanta.com/event/detail/safe-and-secure-online-cybersecurity-for-kids/)[October19Saturday Sweet Auburn Barbecue Watch Party10/19/24](https://discoveratlanta.com/event/detail/sweet-auburn-barbecue-watch-party/)[October19Saturday Talking Tech: Det. Zack Kowalske \u2013 Astroforenics10/19/24](https://discoveratlanta.com/event/detail/talking-tech-det-zack-kowalske-astroforenics/)[October19Saturday To The Limit: An Eagles Tribute10/19/24](https://discoveratlanta.com/event/detail/to-the-limit-an-eagles-tribute/)[October19Saturday Tower of Power10/19/24](https://discoveratlanta.com/event/detail/tower-of-power/)[October20Sunday Dr. Jekyll and Mr. Hyde (1920)10/20/24](https://discoveratlanta.com/event/detail/dr-jekyll-and-mr-hyde-1920/)[October20Sunday Experience Castleberry Hill: Loft Tour \u0026#038; Wine Tast10/20/24](https://discoveratlanta.com/event/detail/experience-castleberry-hill-loft-tour-wine-tast/)[October20Sunday Litner\u002BDeganian Fall Family Fun Fest Dog Adoption10/20/24](https://discoveratlanta.com/event/detail/litner-deganian-fall-family-fun-fest-dog-adoption/)[October20Sunday Sip \u0026#038; Scent: A Halloween Candle-Making Workshop10/20/24](https://discoveratlanta.com/event/detail/sip-scent-a-halloween-candle-making-workshop/)[October20Sunday The Atlanta Symphony Orchestra at Spivey Hall10/20/24](https://discoveratlanta.com/event/detail/the-atlanta-symphony-orchestra-at-spivey-hall/)[October20Sunday The Wilson Springs Hotel w/ Conrad Moore10/20/24](https://discoveratlanta.com/event/detail/the-wilson-springs-hotel-w-conrad-moore/)[October21Monday DJ Shadow10/21/24](https://discoveratlanta.com/event/detail/dj-shadow/)[October22Tuesday PINK\u2019s Global 20th Fall Women\u2019s Empowerment Event10/22/24](https://discoveratlanta.com/event/detail/pink-s-global-20th-fall-women-s-empowerment-event/)[October23Wednesday The Headless Horseman of Silly Hollow10/23/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/the-headless-horseman-of-silly-hollow/)[October24Thursday ATLANTA BUSINESS OWNERS POP-UP MIXER10/24/24](https://discoveratlanta.com/event/detail/atlanta-business-owners-pop-up-mixer/)[October24Thursday An Evening with Witches10/24/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/an-evening-with-witches/)[October24Thursday Candlelight: Horror Movie Soundtracks10/24/24](https://discoveratlanta.com/event/detail/candlelight-horror-movie-soundtracks/)[October24Thursday Christopher Mosley \u2013 Exchange: Behind The Music10/24/24](https://discoveratlanta.com/event/detail/christopher-mosley-exchange-behind-the-music/)[October24Thursday HAIRSPRAY10/24/24 \u2013 11/9/24](https://discoveratlanta.com/event/detail/hairspray/)[October24Thursday Scary Movies at The Strand: Friday The 13th (1980)10/24/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-friday-the-13th-1980/)[October25Friday Candlelight: A Haunted Evening of Halloween Classi10/25/24](https://discoveratlanta.com/event/detail/candlelight-a-haunted-evening-of-halloween-classi/)[October25Friday Footwork \u0026#038; Foodtrucks Trunk or Treat10/25/24](https://discoveratlanta.com/event/detail/footwork-foodtrucks-trunk-or-treat/)[October25Friday Halloween Chocolate \u0026#038; Wine Pairing10/25/24](https://discoveratlanta.com/event/detail/halloween-chocolate-wine-pairing/)[October25Friday Nero Simon \u0026#038; The Sunsetters W/ The Explorers Club10/25/24](https://discoveratlanta.com/event/detail/nero-simon-the-sunsetters-w-the-explorers-club/)[October25Friday The Rocky Horror Picture Show (1975)10/25/24 \u2013 10/26/24](https://discoveratlanta.com/event/detail/the-rocky-horror-picture-show-1975/)[October26Saturday 18th PF HALLOWEEN PARTY \u0026#038; COSTUME CONTEST10/26/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/18th-pf-halloween-party-costume-contest/)[October26Saturday 2024 Komen Georgia MORE THAN PINK Walk10/26/24](https://discoveratlanta.com/event/detail/2024-komen-georgia-more-than-pink-walk/)[October26Saturday Boris10/26/24](https://discoveratlanta.com/event/detail/boris/)[October26Saturday DEHD10/26/24](https://discoveratlanta.com/event/detail/dehd/)[October26Saturday Dinosaur Trick-or-Treat10/26/24](https://discoveratlanta.com/event/detail/dinosaur-trick-or-treat/)[October26Saturday Fall Country Affair10/26/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/fall-country-affair/)[October26Saturday Good Mews Fundraising Gala10/26/24](https://discoveratlanta.com/event/detail/good-mews-fundraising-gala/)[October26Saturday Halloween Magic Show10/26/24](https://discoveratlanta.com/event/detail/halloween-magic-show/)[October26Saturday INTERNATIONAL LIFE CHANGERS AWARDS \u0026#038; FUNDRAISER10/26/24](https://discoveratlanta.com/event/detail/international-life-changers-awards-fundraiser/)[October26Saturday JAZZ in the PARK ATL10/26/24 \u2013 10/27/24](https://discoveratlanta.com/event/detail/jazz-in-the-park-atl/)[October26Saturday Jelly Roll10/26/24](https://discoveratlanta.com/event/detail/jelly-roll/)[October26Saturday Lightning 5K/10K10/26/24](https://discoveratlanta.com/event/detail/lightning-5k-10k/)[October26Saturday Marietta PAL Fall Festival Family Fun House10/26/24](https://discoveratlanta.com/event/detail/marietta-pal-fall-festival-family-fun-house/)[October26Saturday Monster Mash10/26/24](https://discoveratlanta.com/event/detail/monster-mash/)[October26Saturday Official Atlanta Halloween Bar Crawl10/26/24](https://discoveratlanta.com/event/detail/official-atlanta-halloween-bar-crawl/)[October26Saturday Saje Grammy-Winning Female Jazz Vocal Quartet10/26/24](https://discoveratlanta.com/event/detail/saje-grammy-winning-female-jazz-vocal-quartet/)[October26Saturday Scary Movies at The Strand: Hocus Pocus (1991)10/26/24](https://discoveratlanta.com/event/detail/scary-movies-at-the-strand-hocus-pocus-1991/)[October26Saturday Stone Mountain Chorus \u2013 Back to Barbershop!10/26/24](https://discoveratlanta.com/event/detail/stone-mountain-chorus-back-to-barbershop/)[October26Saturday Tales From the T!t Halloween Burlesque Show10/26/24](https://discoveratlanta.com/event/detail/tales-from-the-t-t-halloween-burlesque-show/)[October26Saturday The Official Halloween Bar Crawl \u2013 Atlanta10/26/24](https://discoveratlanta.com/event/detail/the-official-halloween-bar-crawl-atlanta/)[October27Sunday 2nd Annual Festa Dell\u2019Uva10/27/24](https://discoveratlanta.com/event/detail/2nd-annual-festa-dell-uva/)[October27Sunday 2nd Annual Festa Dell\u2019uva at Pricci10/27/24](https://discoveratlanta.com/event/detail/2nd-annual-festa-dell-uva-at-pricci/)[October27Sunday Adam W. Sadberry, flute and Nathan Cheung, piano10/27/24](https://discoveratlanta.com/event/detail/adam-w-sadberry-flute-and-nathan-cheung-piano/)[October28Monday Dylan Gossett \u2013 The Back 40 Tour10/27/24 \u2013 10/28/24](https://discoveratlanta.com/event/detail/dylan-gossett-the-back-40-tour/)[October27Sunday Halloween Kids Chocolate Making Event10/27/24](https://discoveratlanta.com/event/detail/halloween-kids-chocolate-making-event/)[October30Wednesday METALCON10/30/24 \u2013 11/1/24](https://discoveratlanta.com/event/detail/metalcon/)[October31Thursday 21st Annual Rome International Film Festival10/31/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/21st-annual-rome-international-film-festival/)[October31Thursday Candlelight: A Haunted Evening of Halloween10/31/24](https://discoveratlanta.com/event/detail/candlelight-a-haunted-evening-of-halloween-2/)[October31Thursday Nightmare On Elm Street (1984)10/31/24](https://discoveratlanta.com/event/detail/nightmare-on-elm-street-1984/)[October31Thursday ShakesBEER!10/31/24](https://discoveratlanta.com/event/detail/shakesbeer/)[October31Thursday The 4th Annual Vampire Ball10/31/24](https://discoveratlanta.com/event/detail/the-4th-annual-vampire-ball/)[October31Thursday Trick or Drink Atlanta Halloween Bar Crawl10/31/24](https://discoveratlanta.com/event/detail/trick-or-drink-atlanta-halloween-bar-crawl/)[November01Friday Chayanne11/1/24](https://discoveratlanta.com/event/detail/chayanne/)[November01Friday David Briggs: The Phantom of the Opera (1925)11/1/24](https://discoveratlanta.com/event/detail/david-briggs-the-phantom-of-the-opera-1925/)[November01Friday Ghostbusters in Concert with the ASO11/1/24 \u2013 11/2/24](https://discoveratlanta.com/event/detail/ghostbusters-in-concert-with-the-aso/)[November01Friday Ghostbusters: Live In Concert11/1/24 \u2013 11/2/24](https://discoveratlanta.com/event/detail/ghostbusters-live-in-concert/)[November01Friday We CARE Vet Fair11/1/24](https://discoveratlanta.com/event/detail/we-care-vet-fair/)[November02Saturday Atlanta Whiskey Riot 202411/2/24](https://discoveratlanta.com/event/detail/atlanta-whiskey-riot-2024/)[November02Saturday Atlanta Whiskey Riot 2024 at Southern Exchange Ballroom (The Cellar)11/2/24](https://discoveratlanta.com/event/detail/atlanta-whiskey-riot-2024-at-southern-exchange-ballroom-the-cellar/)[November02Saturday Atlanta Women\u2019s Expo11/2/24](https://discoveratlanta.com/event/detail/atlanta-women-s-expo/)[November02Saturday Billie Eilish11/2/24 \u2013 11/4/24](https://discoveratlanta.com/event/detail/billie-eilish/)[November02Saturday Dino-Light11/2/24](https://discoveratlanta.com/event/detail/dino-light/)[November02Saturday Garden \u0026#038; Gun\u2019s 12th Annual Shoot11/2/24](https://discoveratlanta.com/event/detail/garden-gun-s-12th-annual-shoot/)[November02Saturday Paws \u0026#038; Pepper Chili Cook-Off11/2/24](https://discoveratlanta.com/event/detail/paws-pepper-chili-cook-off/)[November02Saturday Phenomenal Women\u2019s Health11/2/24](https://discoveratlanta.com/event/detail/phenomenal-women-s-health/)[November02Saturday Punch Card Machine Demo @ CMoA, November11/2/24](https://discoveratlanta.com/event/detail/punch-card-machine-demo-cmoa-november/)[November02Saturday Radney Foster, An Acoustic Evening; Opener11/2/24](https://discoveratlanta.com/event/detail/radney-foster-an-acoustic-evening-opener/)[November02Saturday Suwanee Wine Fest11/2/24](https://discoveratlanta.com/event/detail/suwanee-wine-fest/)[November03Sunday Dallas Cowboys at Atlanta Falcons11/3/24](https://discoveratlanta.com/event/detail/dallas-cowboys-at-atlanta-falcons/)[November03Sunday Nature Walk with Chattahoochee National Conservanc11/3/24](https://discoveratlanta.com/event/detail/nature-walk-with-chattahoochee-national-conservanc/)[November07Thursday Caloncho11/7/24](https://discoveratlanta.com/event/detail/caloncho/)[November07Thursday Escape to Margaritaville11/7/24 \u2013 11/16/24](https://discoveratlanta.com/event/detail/escape-to-margaritaville/)[November07Thursday Highlands Food \u0026#038; Wine Festival11/7/24 \u2013 11/10/24](https://discoveratlanta.com/event/detail/highlands-food-wine-festival/)[November07Thursday Reins of Hope11/7/24](https://discoveratlanta.com/event/detail/reins-of-hope/)[November08Friday Black Women In Clinical Research Conference11/8/24 \u2013 11/10/24](https://discoveratlanta.com/event/detail/black-women-in-clinical-research-conference/)[November08Friday IrishFest Atlanta 202411/8/24 \u2013 11/10/24](https://discoveratlanta.com/event/detail/irishfest-atlanta-2024-2/)[November08Friday Mom Talks \u2013 Atlanta11/8/24](https://discoveratlanta.com/event/detail/mom-talks-atlanta/)[November09Saturday Beyond Limits Ride11/9/24](https://discoveratlanta.com/event/detail/beyond-limits-ride/)[November09Saturday Decatur Wine Festival11/9/24](https://discoveratlanta.com/event/detail/decatur-wine-festival/)[November09Saturday Georgia Tech Football vs Miami11/9/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-miami/)[November09Saturday JUNNY: The Dopamine Tour w/ DAUL11/9/24](https://discoveratlanta.com/event/detail/junny-the-dopamine-tour-w-daul/)[November09Saturday Magnificent Marimba11/9/24](https://discoveratlanta.com/event/detail/magnificent-marimba/)[November13Wednesday Champions Classic (Kansas Jayhawks vs. Michigan State Spartans, Kentucky Wildcats vs. Duke Blue Devils)11/13/24](https://discoveratlanta.com/event/detail/champions-classic-kansas-jayhawks-vs-michigan-state-spartans-kentucky-wildcats-vs-duke-blue-devils/)[November14Thursday P!NK Concert at State Farm Arena11/14/24](https://discoveratlanta.com/event/detail/p-nk-concert-at-state-farm-arena/)[November15Friday NPCF Fashion Funds the Cure11/15/24](https://discoveratlanta.com/event/detail/npcf-fashion-funds-the-cure/)[November15Friday The Tim Shleton Syndicate11/15/24](https://discoveratlanta.com/event/detail/the-tim-shleton-syndicate/)[November15Friday We Are The World Festival11/15/24 \u2013 11/17/24](https://discoveratlanta.com/event/detail/we-are-the-world-festival/)[November16Saturday Arkansas State Red Wolves at Georgia State Panthers Football11/16/24](https://discoveratlanta.com/event/detail/arkansas-state-red-wolves-at-georgia-state-panthers-football/)[November16Saturday Bohemian Rhapsody11/16/24](https://discoveratlanta.com/event/detail/bohemian-rhapsody/)[November16Saturday Devastation On The Nation Tour 2024.11/16/24](https://discoveratlanta.com/event/detail/devastation-on-the-nation-tour-2024/)[November16Saturday Locals \u0026#038; Legends11/16/24](https://discoveratlanta.com/event/detail/locals-legends/)[November16Saturday The All Black Experience11/16/24 \u2013 11/17/24](https://discoveratlanta.com/event/detail/the-all-black-experience/)[November16Saturday Uncommon Bizarre Historic Fourth WardMultiple dates until 11/17/2024](https://discoveratlanta.com/event/detail/uncommon-bizarre-historic-fourth-ward/)[November17Sunday Coat Check \u2013 Morgan Jay \u2013 Not a Concert Ticket**11/17/24](https://discoveratlanta.com/event/detail/coat-check-morgan-jay-not-a-concert-ticket/)[November17Sunday Great Atlanta Dog Show @ Wild Heaven Beer WEST END11/17/24](https://discoveratlanta.com/event/detail/great-atlanta-dog-show-wild-heaven-beer-west-end/)[November20Wednesday 52nd NABSE Conference-Empowering Black Excellence11/20/24 \u2013 11/24/24](https://discoveratlanta.com/event/detail/52nd-nabse-conference-empowering-black-excellence/)[November20Wednesday Feed Your Senses Lunchtime Concert: Chamber Cartel11/20/24](https://discoveratlanta.com/event/detail/feed-your-senses-lunchtime-concert-chamber-cartel/)[November21Thursday Georgia Tech Football vs NC State11/21/24](https://discoveratlanta.com/event/detail/georgia-tech-football-vs-nc-state/)[November21Thursday ISM TEST EVENT11/21/24](https://discoveratlanta.com/event/detail/ism-test-event/)[November22Friday 39th Annual Cobb County GMS Show11/22/24 \u2013 11/24/24](https://discoveratlanta.com/event/detail/39th-annual-cobb-county-gms-show/)[November22Friday Adrianne Lenker11/22/24](https://discoveratlanta.com/event/detail/adrianne-lenker/)[November22Friday Shawn Phillips Live11/22/24](https://discoveratlanta.com/event/detail/shawn-phillips-live/)[November23Saturday Becky Bullers Band XMas Show11/23/24](https://discoveratlanta.com/event/detail/becky-bullers-band-xmas-show/)[November23Saturday Exhale: An Evening of Sound Meditation and Music11/23/24](https://discoveratlanta.com/event/detail/exhale-an-evening-of-sound-meditation-and-music/)[November24Sunday Animals As Leaders11/24/24](https://discoveratlanta.com/event/detail/animals-as-leaders/)[November24Sunday Holiday Photos with Santa11/24/24](https://discoveratlanta.com/event/detail/holiday-photos-with-santa/)[November27Wednesday Grand Kyiv Ballet Presents the Magic of Nutcracker11/27/24](https://discoveratlanta.com/event/detail/grand-kyiv-ballet-presents-the-magic-of-nutcracker/)[November28Thursday Atlanta Turkey Trot11/28/24](https://discoveratlanta.com/event/detail/atlanta-turkey-trot/)[December07Saturday Cirque Du Soleil Twas The Night Before11/30/24 \u2013 12/15/24](https://discoveratlanta.com/event/detail/cirque-du-soleil-twas-the-night-before/)[November30Saturday Coastal Carolina Chanticleers at Georgia State Panthers Football11/30/24](https://discoveratlanta.com/event/detail/coastal-carolina-chanticleers-at-georgia-state-panthers-football/)[November30Saturday Frozen in Concert with the ASO11/30/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/frozen-in-concert-with-the-aso/)[November30Saturday Gift of the Magi 2.011/30/24 \u2013 12/8/24](https://discoveratlanta.com/event/detail/gift-of-the-magi-2-0/)[December01Sunday American Aquarium12/1/24](https://discoveratlanta.com/event/detail/american-aquarium/)[December01Sunday Coat Check \u2013 Ocie Elliott \u2013 Not a Concert Ticket12/1/24](https://discoveratlanta.com/event/detail/coat-check-ocie-elliott-not-a-concert-ticket/)[December01Sunday Frozen: Live In Concert12/1/24](https://discoveratlanta.com/event/detail/frozen-live-in-concert/)[December05Thursday Creed12/5/24](https://discoveratlanta.com/event/detail/creed/)[December07Saturday Kathy Griffin: My Life On The PTSD-List12/7/24](https://discoveratlanta.com/event/detail/kathy-griffin-my-life-on-the-ptsd-list/)[December07Saturday \u2019Twas the Night Before Christmas12/7/24](https://discoveratlanta.com/event/detail/twas-the-night-before-christmas/)[December08Sunday 27th Annual Holiday Gala by GSU School of Music12/8/24](https://discoveratlanta.com/event/detail/27th-annual-holiday-gala-by-gsu-school-of-music/)[December08Sunday FINAL FANTASY VII REBIRTH Orchestra World Tour12/8/24](https://discoveratlanta.com/event/detail/final-fantasy-vii-rebirth-orchestra-world-tour/)[December08Sunday ISMO: Watch Your Language Tour12/8/24](https://discoveratlanta.com/event/detail/ismo-watch-your-language-tour/)[December08Sunday Twas the Night Before by Cirque du Soleil12/8/24](https://discoveratlanta.com/event/detail/twas-the-night-before-by-cirque-du-soleil/)[December13Friday 356 Porsche Dinner Series // Edible Engineering12/13/24](https://discoveratlanta.com/event/detail/356-porsche-dinner-series-edible-engineering/)[December14Saturday Chloe Agnew And The Brave Collide Christmas Show12/14/24](https://discoveratlanta.com/event/detail/chloe-agnew-and-the-brave-collide-christmas-show/)[December14Saturday Holidays With the Johns Creek Symphony Orchestra12/14/24](https://discoveratlanta.com/event/detail/holidays-with-the-johns-creek-symphony-orchestra/)[December14Saturday North Pole Fa-La-La-La FolliesMultiple dates until 12/14/2024](https://discoveratlanta.com/event/detail/north-pole-fa-la-la-la-follies/)[December14Saturday Plush Romance: V Day Luxury Picnic at Piedmont12/14/24 \u2013 12/17/24](https://discoveratlanta.com/event/detail/plush-romance-v-day-luxury-picnic-at-piedmont/)[December14Saturday Roswell Ugly Sweater Bar Crawl: 4th Annual12/14/24](https://discoveratlanta.com/event/detail/roswell-ugly-sweater-bar-crawl-4th-annual/)[December16Monday CHUU12/16/24](https://discoveratlanta.com/event/detail/chuu/)[December18Wednesday Feed Your Senses Lunch Concert: GSU Jazz Combo12/18/24](https://discoveratlanta.com/event/detail/feed-your-senses-lunch-concert-gsu-jazz-combo/)[December27Friday Blue Dogs12/27/24](https://discoveratlanta.com/event/detail/blue-dogs/)[January05Sunday Carolina Panthers at Atlanta Falcons1/5/25](https://discoveratlanta.com/event/detail/carolina-panthers-at-atlanta-falcons/)[January08Wednesday Squirrel Stole My Underpants1/8/25 \u2013 1/19/25](https://discoveratlanta.com/event/detail/squirrel-stole-my-underpants/)[January12Sunday \u0026#038; Juliet1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/juliet/)[January09Thursday 2025 DSC Convention \u0026#038; Sporting Expo1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/2025-dsc-convention-sporting-expo/)[January21Tuesday College Football Playoff National Championship1/21/25](https://discoveratlanta.com/event/detail/college-football-playoff-national-championship/)[January24Friday FSR Presents: Life Be Lifin\u2019 Starring Monet X Change1/24/25](https://discoveratlanta.com/event/detail/fsr-presents-life-be-lifin-starring-monet-x-change/)[January24Friday First Arab Elbow Course- The Traumatic1/24/25 \u2013 1/26/25](https://discoveratlanta.com/event/detail/first-arab-elbow-course-the-traumatic/)[January25Saturday Heroes Among Us: A Musical Tribute1/25/25](https://discoveratlanta.com/event/detail/heroes-among-us-a-musical-tribute/)[January25Saturday Lunar New Year Celebration1/25/25](https://discoveratlanta.com/event/detail/lunar-new-year-celebration/)[February01Saturday Clue The Musical1/30/25 \u2013 2/1/25](https://discoveratlanta.com/event/detail/clue-the-musical/)[February01Saturday FLOTAX test2/1/25](https://discoveratlanta.com/event/detail/flotax-test/)[February02Sunday Hot Chocolate Run Atlanta2/2/25](https://discoveratlanta.com/event/detail/hot-chocolate-run-atlanta/)[February08Saturday FREE Movie Night: Frozen2/8/25](https://discoveratlanta.com/event/detail/free-movie-night-frozen/)[March12Wednesday A Beautiful Noise: The Neil Diamond Musical2/9/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/a-beautiful-noise-the-neil-diamond-musical/)[February14Friday Martin Amini2/14/25](https://discoveratlanta.com/event/detail/martin-amini/)[February15Saturday African-American Folktales2/15/25](https://discoveratlanta.com/event/detail/african-american-folktales/)[February15Saturday Trailblazers2/15/25](https://discoveratlanta.com/event/detail/trailblazers/)[March01Saturday Suwanee Beer Fest Beer Chaser 5K3/1/25](https://discoveratlanta.com/event/detail/suwanee-beer-fest-beer-chaser-5k/)[March02Sunday Dan Forrest\u2019s Newest Work \u201CCreation\u201D3/2/25](https://discoveratlanta.com/event/detail/dan-forrest-s-newest-work-creation/)[March05Wednesday Galapagos George, The Little Tortoise that Could3/5/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/galapagos-george-the-little-tortoise-that-could/)[March08Saturday Blacktop Playground: Improv for Kids3/8/25](https://discoveratlanta.com/event/detail/blacktop-playground-improv-for-kids/)[March10Monday PACK EXPO 20253/10/25 \u2013 3/12/25](https://discoveratlanta.com/event/detail/pack-expo-2025/)[March19Wednesday Milo the Magnificent3/19/25 \u2013 3/30/25](https://discoveratlanta.com/event/detail/milo-the-magnificent/)[March29Saturday Debussy, Ravel, \u0026#038; Sibelius: A Birthday Celebration3/29/25](https://discoveratlanta.com/event/detail/debussy-ravel-sibelius-a-birthday-celebration/)[March29Saturday It\u2019s Electric!3/29/25](https://discoveratlanta.com/event/detail/it-s-electric/)[April09Wednesday Spring Break Magic Show4/9/25](https://discoveratlanta.com/event/detail/spring-break-magic-show/)[April19Saturday Dr. Dinosaur!4/19/25](https://discoveratlanta.com/event/detail/dr-dinosaur/)[May03Saturday Fairy God Doctor5/3/25](https://discoveratlanta.com/event/detail/fairy-god-doctor/)[May04Sunday May the 4th Be With You: Sun, Moon, Air, \u0026#038; Space5/4/25](https://discoveratlanta.com/event/detail/may-the-4th-be-with-you-sun-moon-air-space/)[May17Saturday The Firebird5/17/25](https://discoveratlanta.com/event/detail/the-firebird/)[October14Tuesday Oral History Association 2025 Annual Meeting10/14/25 \u2013 10/19/25](https://discoveratlanta.com/event/detail/oral-history-association-2025-annual-meeting/)[September24Tuesday 9.99 Burger Night at Elsewhere9/24/24 \u2013 9/26/24](https://discoveratlanta.com/event/detail/9-99-burger-night-at-elsewhere/)[September24Tuesday Johnnie \u0026#038; Clyde\u2019s Pop Up at Elsewhere Brewing9/2/24 \u2013 9/24/24](https://discoveratlanta.com/event/detail/johnnie-clyde-s-pop-up-at-elsewhere-brewing/)[September24Tuesday Center for Puppetry Arts9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/center-for-puppetry-arts/)[September27Friday Children\u2019s Museum of AtlantaEvery Friday until 09/30/2024](https://discoveratlanta.com/event/detail/children-s-museum-of-atlanta/)[September24Tuesday David J. Sencer CDC Museum9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/david-j-sencer-cdc-museum/)[September24Tuesday Fernbank Museum9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/fernbank-museum/)[September24Tuesday High Museum of ArtMultiple dates until 09/30/2024](https://discoveratlanta.com/event/detail/high-museum-of-art/)[September24Tuesday Jimmy Carter Presidential Library and Museum9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/jimmy-carter-presidential-library-and-museum/)[September24Tuesday The Breman9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/the-breman/)[September24Tuesday World of Coca-Cola9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/world-of-coca-cola/)[September24Tuesday A Sunset Cruise aboard The Experience9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/a-sunset-cruise-aboard-the-experience/)[September24Tuesday First Responder Appreciation Month9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/first-responder-appreciation-month/)[September24Tuesday Geoffrey Chaucer\u2019s \u201CThe Canterbury Tales\u201D9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/geoffrey-chaucer-s-the-canterbury-tales/)[September24Tuesday A Raisin In The Sun9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/a-raisin-in-the-sun/)[September27Friday Atlanta Full-Day Wine Country Wine Tastings9/24/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/atlanta-full-day-wine-country-wine-tastings-2/)[September24Tuesday Daddy Long Legs9/24/24 \u2013 10/6/24](https://discoveratlanta.com/event/detail/daddy-long-legs/)[October21Monday Queer QuizMultiple dates until 10/21/2024](https://discoveratlanta.com/event/detail/queer-quiz/)[September28Saturday Food and Street Art Tour of the Atlanta BeltLineEvery Saturday until 11/16/2024](https://discoveratlanta.com/event/detail/food-and-street-art-tour-of-the-atlanta-beltline/)[September24Tuesday History with Chutzpah9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/history-with-chutzpah/)[October19Saturday Black Child Book Fair10/19/24](https://discoveratlanta.com/event/detail/black-child-book-fair/)[November09Saturday Eatonton Mistletoe Market11/9/24](https://discoveratlanta.com/event/detail/eatonton-mistletoe-market/)[October06Sunday Barks and Bites: Doggie Crawl on the BeltLineMultiple dates until 11/17/2024](https://discoveratlanta.com/event/detail/barks-and-bites-doggie-crawl-on-the-beltline/)[December01Sunday Best Artist Ever9/24/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/best-artist-ever/)[December06Friday Christmas with The Lanier Chamber Singers9/24/24 \u2013 12/6/24](https://discoveratlanta.com/event/detail/christmas-with-the-lanier-chamber-singers/)[November16Saturday Holiday Juggling Show9/24/24 \u2013 11/16/24](https://discoveratlanta.com/event/detail/holiday-juggling-show/)[December19Thursday NUTCRACKER! Magical Christmas Ballet9/24/24 \u2013 12/20/24](https://discoveratlanta.com/event/detail/nutcracker-magical-christmas-ballet/)[September24Tuesday Candlelight Concerts 20249/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/candlelight-concerts-2024/)[September24Tuesday WQEB 97.3 DINE \u0026#038; DONATE, a Food \u0026#038; Clothing Drive9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/wqeb-97-3-dine-donate-a-food-clothing-drive/)[September24Tuesday Adopt Any Senior Pet for FREE in 20249/24/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/adopt-any-senior-pet-for-free-in-2024/)[September29Sunday Bottomless BrunchEvery Sunday until 12/29/2024](https://discoveratlanta.com/event/detail/bottomless-brunch/)[September24Tuesday (ad)Vantage Point9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/ad-vantage-point/)[September24Tuesday Next Level Boys Academy Mentoring9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/next-level-boys-academy-mentoring/)[September24Tuesday UrbanTree After Dark: A Disco Pop-Up Bar9/24/24 \u2013 10/4/24](https://discoveratlanta.com/event/detail/urbantree-after-dark-a-disco-pop-up-bar/)[September24Tuesday Free Self-guided Street Art Walking Tours9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/free-self-guided-street-art-walking-tours/)[September24Tuesday Midnight Basketball League: Season 6Multiple days until 09/30/2024](https://discoveratlanta.com/event/detail/midnight-basketball-league-season-6/)[March22Saturday Wing \u0026#038; Rock Fest9/24/24 \u2013 3/23/25](https://discoveratlanta.com/event/detail/wing-rock-fest/)[October05Saturday Yoga at Van Gogh: The Immersive Experience9/24/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/yoga-at-van-gogh-the-immersive-experience-2/)[September24Tuesday Punchline Comedy Night9/24/24 \u2013 10/7/24](https://discoveratlanta.com/event/detail/punchline-comedy-night/)[December07Saturday Punch Card Machine Demo @ CMoA9/24/24 \u2013 12/7/24](https://discoveratlanta.com/event/detail/punch-card-machine-demo-cmoa/)[September29Sunday Atlanta Comic Convention9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/atlanta-comic-convention/)[September25Wednesday Broad Street Boardwalk Wednesday Lunchtime SeriesEvery Wednesday until 10/02/2024](https://discoveratlanta.com/event/detail/broad-street-boardwalk-wednesday-lunchtime-series/)[September29Sunday Sip \u0026#038; Scent: A Candle-Making Workshop9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/sip-scent-a-candle-making-workshop-2/)[November02Saturday Electronics Recycling Drive9/24/24 \u2013 11/2/24](https://discoveratlanta.com/event/detail/electronics-recycling-drive-2/)[September28Saturday Free Yoga Classes on the Broad Street BoardwalkEvery Saturday until 09/28/2024](https://discoveratlanta.com/event/detail/free-yoga-classes-on-the-broad-street-boardwalk/)[November09Saturday Protect Yo Energy Con9/24/24 \u2013 11/9/24](https://discoveratlanta.com/event/detail/protect-yo-energy-con/)[September28Saturday Weekend Creekwater Alpaca Farm Barn Tour9/24/24 \u2013 9/29/24](https://discoveratlanta.com/event/detail/weekend-creekwater-alpaca-farm-barn-tour-2/)[September24Tuesday Pick Your Perks Group Promotion9/24/24 \u2013 5/31/25](https://discoveratlanta.com/event/detail/pick-your-perks-group-promotion/)[September24Tuesday Create Your Own Scented Candle!9/24/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/create-your-own-scented-candle/)[November16Saturday Diva Royale Drag Queen Show9/24/24 \u2013 12/14/24](https://discoveratlanta.com/event/detail/diva-royale-drag-queen-show/)[September24Tuesday Little Acorns9/24/24 \u2013 10/22/24](https://discoveratlanta.com/event/detail/little-acorns/)[September26Thursday Jazzy Thursdays and 1/2 off Bottles of WineEvery Thursday until 12/26/2024](https://discoveratlanta.com/event/detail/jazzy-thursdays-and-1-2-off-bottles-of-wine/)[September24Tuesday Sunset Sessions Presents DJ Mike Webster \u0026#038; DJ Xzac9/24/24 \u2013 10/13/24](https://discoveratlanta.com/event/detail/sunset-sessions-presents-dj-mike-webster-dj-xzac/)[September28Saturday Rise and Dine with the Animals: PandasMultiple dates until 09/28/2024](https://discoveratlanta.com/event/detail/rise-and-dine-with-the-animals-pandas/)[September24Tuesday Stand \u0026#038; Witness: Art in the Time of COVID-199/24/24 \u2013 10/24/24](https://discoveratlanta.com/event/detail/stand-witness-art-in-the-time-of-covid-19/)[September24Tuesday Beginner\u2019s Salsa Lessons @ PasoFino Dance Studio9/24/24 \u2013 6/30/25](https://discoveratlanta.com/event/detail/beginner-s-salsa-lessons-pasofino-dance-studio/)[September24Tuesday Tyler Mitchell: Idyllic Space9/24/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/tyler-mitchell-idyllic-space/)[October11Friday ZooDunit Mystery SeriesMultiple dates until 10/11/2024](https://discoveratlanta.com/event/detail/zoodunit-mystery-series/)[September26Thursday Jazzy Thursday\u2019s at PricciEvery Thursday until 12/26/2024](https://discoveratlanta.com/event/detail/jazzy-thursday-s-at-pricci/)[September30Monday Banfi MondaysEvery Monday until 12/30/2024](https://discoveratlanta.com/event/detail/banfi-mondays/)[September27Friday Candlelight: A Tribute to Whitney Houston9/24/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/candlelight-a-tribute-to-whitney-houston-2/)[September24Tuesday ARMHC To Host 15th Annual Gala9/24/24 \u2013 10/5/24](https://discoveratlanta.com/event/detail/armhc-to-host-15th-annual-gala/)[September24Tuesday Dining in the Dark9/24/24 \u2013 9/27/24](https://discoveratlanta.com/event/detail/dining-in-the-dark/)[September24Tuesday The Gaze9/24/24 \u2013 12/1/24](https://discoveratlanta.com/event/detail/the-gaze/)[September29Sunday Latin Brunch w/ Live Music \u0026#038; $3 Mimosas \u002B SangriasEvery Sunday until 12/01/2024](https://discoveratlanta.com/event/detail/latin-brunch-w-live-music-3-mimosas-sangrias/)[September24Tuesday Titanic: An Immersive Voyage9/24/24 \u2013 1/22/25](https://discoveratlanta.com/event/detail/titanic-an-immersive-voyage/)[September27Friday Improvised Dungeons \u0026#038; Dragons Returns!Every Friday until 09/27/2024](https://discoveratlanta.com/event/detail/improvised-dungeons-dragons-returns/)[September24Tuesday Trivia Night at Wild Leap AtlantaEvery Tuesday until 11/19/2024](https://discoveratlanta.com/event/detail/trivia-night-at-wild-leap-atlanta/)[September24Tuesday Elements- Surya Ensemble9/24/24 \u2013 10/10/24](https://discoveratlanta.com/event/detail/elements-surya-ensemble/)[September24Tuesday Your Great Neighbor Deal!9/24/24 \u2013 9/30/24](https://discoveratlanta.com/event/detail/your-great-neighbor-deal/)[September24Tuesday Felt Street9/24/24 \u2013 9/28/24](https://discoveratlanta.com/event/detail/felt-street/)[September28Saturday Felt Street Now Showing at Dad\u2019s Garage!Every Saturday until 09/28/2024](https://discoveratlanta.com/event/detail/felt-street-now-showing-at-dad-s-garage/)[September30Monday Test Your Trivia Knowledge at SocialBites Food HubEvery Monday until 09/30/2024](https://discoveratlanta.com/event/detail/test-your-trivia-knowledge-at-socialbites-food-hub/)[September25Wednesday Visit SocialBites Every Wednesday for Poker NightEvery Wednesday until 09/25/2024](https://discoveratlanta.com/event/detail/visit-socialbites-every-wednesday-for-poker-night/)[September25Wednesday \u201CMix It Up\u201D Ladies NightEvery Wednesday until 12/25/2024](https://discoveratlanta.com/event/detail/mix-it-up-ladies-night/)[September28Saturday The Dinner Detective Comedy Mystery Dinner Show \u2013Every Saturday until 10/26/2024](https://discoveratlanta.com/event/detail/the-dinner-detective-comedy-mystery-dinner-show/)[September24Tuesday Golden Opportunity: Botanical Illustration9/24/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/golden-opportunity-botanical-illustration/)[September24Tuesday Chastian Challenge9/24/24 \u2013 10/15/24](https://discoveratlanta.com/event/detail/chastian-challenge/)[September24Tuesday The Mill Town Square Dinners9/24/24 \u2013 10/31/24](https://discoveratlanta.com/event/detail/the-mill-town-square-dinners/)[September24Tuesday Hold The Line Vol 2: Celebrating Black Masculinity9/24/24 \u2013 10/26/24](https://discoveratlanta.com/event/detail/hold-the-line-vol-2-celebrating-black-masculinity/)[September24Tuesday Kids\u2019 Holiday Show Creation Program9/24/24 \u2013 11/30/24](https://discoveratlanta.com/event/detail/kids-holiday-show-creation-program/)[September24Tuesday Fright Fest Atlanta9/24/24 \u2013 11/3/24](https://discoveratlanta.com/event/detail/fright-fest-atlanta/)[September24Tuesday Face Your Fears at NETHERWORLD Haunted House!9/24/24 \u2013 11/9/24](https://discoveratlanta.com/event/detail/face-your-fears-at-netherworld-haunted-house/)[September30Monday Mystery Monday \u2013 Drag ShowMultiple dates until 11/11/2024](https://discoveratlanta.com/event/detail/mystery-monday-drag-show/)[April08Tuesday Cat in the Hat4/8/25 \u2013 5/25/25](https://discoveratlanta.com/event/detail/cat-in-the-hat/)[November16Saturday Justin Timberlake Live at State Farm ArenaMultiple dates until 11/16/2024](https://discoveratlanta.com/event/detail/justin-timberlake-live-at-state-farm-arena/)[October19Saturday Trick-or-Treat at CMOAMultiple dates until 10/26/2024](https://discoveratlanta.com/event/detail/trick-or-treat-at-cmoa/)## Discover ATL Events\r\n\r\n\r\n\r\nDates to searchDATESTODAYTOMORROWTHIS WEEKTHIS WEEKENDTHIS MONTHCUSTOM DATE RANGEEvent categoryCATEGORYALL CATEGORIESAutomotiveBenefitComedyDay CampsDiningDrinksEducation \u0026amp; LecturesFamily \u0026amp; KidsFashionFestivalsFilmFitness \u0026amp; HealthFundraiserGamingHolidayHoliday Pop-upHome ToursLGBTLiteraryMental HealthMuseums \u0026amp; AttractionsMusic \u0026amp; ConcertsNeighborhoodNetworkingNightlifeOther ActivitiesOutdoors \u0026amp; RecreationPetsShoppingSpanish-speaking/LatinSpecial EventsSpiritualitySportsTheater \u0026amp; Performing ArtsVirtual \u0026amp; OnlineVisual Arts \u0026amp; GalleriesWatch PartyYear of the YouthEvent keywordStart dateEnd dateSearch[View All Events](/events/all)## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[EventsCelebrate Atlanta with a festival for the season Whether it\u2019s honoring a holiday or just celebrating the season, Atlanta\u2019s neighborhood communities love to find any reason to gather for food, music, and family fun. Festivals and Annual Events](https://discoveratlanta.com/events/festivals/)[EventsThere is no shortage of family fun in Atlanta. Outdoors or in, summer or fall, Atlanta is the place to be for family-friendly events. Family-Friendly Events](https://discoveratlanta.com/events/family/)[EventsBounce through the incredible events happening around Atlanta today, tomorrow, this week or this month. Upcoming Events](https://discoveratlanta.com/events/all/)[ExplorePlan fun family getaways to Atlanta Fun Atlanta vacation ideas for families There are countless fun events and things to do in Atlanta for family getaways. Family Trip Ideas](https://discoveratlanta.com/explore/trip-ideas/family-trip-ideas/)[EventsOnline \u0026#038; virtual events in Atlanta Discover fun virtual events in Atlanta to stream online from your favorite attractions, museums and theatres in the comfort of your own home. Virtual Events](https://discoveratlanta.com/events/virtual/)[Arts \u0026amp; CultureThere are many authentic Mexican flavors in Atlanta. Discover some of the best Mexican dishes throughout the city of Atlanta. Where to Enjoy Authentic Mexican Food in Atlanta](https://discoveratlanta.com/stories/things-to-do/where-to-enjoy-authentic-mexican-food-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/"}
```
and got this result:
```
"The Best Things to Do in Atlanta in September - Unexpected Atlanta![fbpx](https://www.facebook.com/tr?id=2689885501249973\u0026ev=PageView\u0026noscript=1)\r\n[Skip to main content](#ajax-content-wrap)* Blog\r\n* About Our Team Hiring Press Contact\r\n* Tour Calendar\r\n* Gift Certificates\r\n* Call Us\r\n* Buy Tickets\r\n\r\n[](https://unexpectedatlanta.com)[Menu](#sidewidgetarea)* Public Tours Atlanta Market Food Tour \u002B Biscuit Cooking Class Prohibition Progressive Dinner \u2013 Atlanta Cocktails and Food Experience Grant Park Atlanta Food Tour Martin Luther King Jr Atlanta Tour Cabbagetown \u0026#038; BeltLine Street Art Tour\r\n* Team Building Private Prohibition Progressive Dinner Private Grant Park Food \u0026#038; History Tour Private Market Food Tour and Biscuit Class Private Martin Luther King Jr Atlanta Tour Private Cabbagetown \u0026#038; BeltLine Street Art Tour Private Dinner and Street Art Tour 1920s Big Bash Party Conference Networking Games\r\n* Family and Student Groups Family Reunions Party Packages MLK Tour for Kids Student Martin Luther King Jr Atlanta Tour\r\n* Book Now!\r\n\r\n* Menu\r\n\r\n[Close Search](#)[Things to Do in Atlanta](https://unexpectedatlanta.com/category/things-to-do-atlanta/)# The Best Things to Do in Atlanta in September\r\n\r\nBy[Akila McConnell](https://unexpectedatlanta.com/author/akila/)June 19, 2024August 21st, 2024[No Comments](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/#respond)![The Best Things to Do in Atlanta in September](https://unexpectedatlanta.com/wp-content/uploads/2023/08/The-Best-Things-to-Do-in-Atlanta-in-September.png)\r\nSoak up the last of summer and celebrate the start of fall with the best things to do in Atlanta in September 2024!\r\n\r\nThis month brings comfortable temperatures as summer cools down \u0026#8211; with long, golden days that spill into evening. Make the most of it with the best things to do in Atlanta!\r\n\r\nFrom exciting sports matchups to concerts, food festivals, apple picking, IG and TikTok-worthy sunflower fields, and more \u0026#8211; we\u0026#8217;re sharing our favorite activities around the city. Your Atlanta September memories start here!\r\n\r\nThe Best Things to Do in Atlanta in SeptemberRead on for additional details on these Atlanta September events and more ATL fun!\r\n\r\n* Cheer on the Atlanta Braves at a home game\r\n* Taste the best of Atlanta\u0026#8217;s Soul Food and Southern favorites\r\n* See musical acts like SOFI TUKKER, Childish Gambino, The Avett Brothers, and more\r\n* Experience culture and cuisine at A Taste of Nigeria\r\n* Take a guided walking food tour to taste 8 delicious modern Southern dishes\r\n* Explore Fernbank\u0026#8230;but Later\r\n\r\nIn this post, you\u0026#8217;ll find:* Fall festivals: Atlanta and beyond\r\n* Ideas to enjoy Atlanta in September\r\n* Concerts, food events, and sports games in Atlanta September 2024\r\n* September family-friendly events in Atlanta\r\n* Atlanta fall date ideas\r\n* Fun things to do during a business trip to Atlanta in September\r\n* FAQs about visiting Atlanta in September, including Atlanta September weather\r\n\r\n## Atlanta Events: September 2024\r\n\r\n### Experience the Best of Atlanta\u0026#8217;s Food \u0026amp; History\r\n\r\n\r\n\r\nTry some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u0026#8217;s food and history. On this tour, you\u0026#8217;ll taste eight modern and Southern foods from three outstanding locally owned food stops.\r\n\r\nAlong the way, delve into Atlanta\u0026#8217;s history with an expert guide who will share fascinating stories of our city and answer questions. You\u0026#8217;ll also explore the unique architecture of Atlanta\u0026#8217;s historic Grant Park neighborhood. The 2.5 hour guided experience includes a jaunt through Historic Oakland Cemetery, where many noteworthy Atlantans have been laid to rest.\r\n\r\nWith more than 1,500 five-star TripAdvisor reviews, Unexpected Atlanta offers outstanding and unique walking tours for Atlanta locals and visitors!\r\n\r\n### Attend the Taste of Soul Festival: August 30 and September 1, 2024\r\n\r\nCraving even more Soul food in the heart of America\u0026#8217;s Soul Food capital? Head to the Taste of Soul Festival! Saturday, August 30 and Sunday, September 1 bring two days packed full of delicious Soul food and culture!\r\n\r\nTaste of Soul will take place across from Mercedes-Benz Stadium near Downtown Atlanta. This free, family-friendly festival is open to all. However, those who wish to attend the \u0026#8220;Soul in the City\u0026#8221; concert as part of this event must purchase tickets.\r\n\r\nThe festival includes Soul and Southern food for purchase, live music, arts and crafts vendors, kids\u0026#8217; activities, and a special cocktail and wine festival.\r\n\r\n### Cheer On the Home Teams at Falcons \u0026amp; Braves Games\r\n\r\n\r\n\r\nSeptember is a great month to be a sports fan! During September 2024, the Atlanta Braves baseball team will take on the LA Dodgers, Colorado Rockies, Kansas City Royals, New York Mets, and Toronto Blue Jays at Truist Park.\r\n\r\nFor college football games, check out Georgia Tech, Georgia State University, and \u0026#8211; if you\u0026#8217;re up for a day trip \u0026#8211; UGA in Athens, GA!\r\n\r\n### Experience Fernbank\u0026#8230;but Later: Sept. 27, 2024\r\n\r\nExplore Fernbank\u0026#8230;but Later with Stellar Science on Friday, September 27, 2024. This opportunity to see exhibits with later access to Fernbank is a fun way to socialize. You\u0026#8217;ll experience all the great indoor exhibit discoveries and outdoor nature explorations.\r\n\r\nAll ages are welcome, but if you\u0026#8217;re 21\u002B, there will be alcoholic beverages for purchase. Non-alcoholic drinks and snacks also available for purchase.\r\n\r\n### Experience Culture \u0026amp; Cuisine at A Taste of Nigeria\r\n\r\nOn Saturday, September 28, bring the family to A Taste of Nigeria at legendary Ford Hall in Hapeville, GA!\r\n\r\nAt this immersive Atlanta summer festival, you\u0026#8217;ll see a vibrant celebration filled Nigerian art, music, dance, comedy, drama, a fashion show, market, and more! Kids get free festival entry, with plenty of family-friendly activities to enjoy.\r\n\r\n### Attend Concerts in Atlanta: September 2024\r\n\r\nAs the capital of the South, Atlanta is a hotspot for concert-goers of all genres. In September 2023, Atlanta will attract chart-topping talent, with concerts for all tastes. From country to pop, hip-hop to classic rock, alternative, and more \u0026#8211; it\u0026#8217;s all here in Atlanta!\r\n\r\nAtlanta concerts in September 2024 \u0026#8211; a few highlights:* Childish Gambino: September 2 at State Farm Arena\r\n* Lynyrd Synyrd and The Outlaws: September 5 at Ameris Bank Amphitheatre\r\n* Mitski: September 6, 7, and 8 at the Fox Theater\r\n* Young Miko: September 6 at Coca-Cola Roxy\r\n* John Legend and Atlanta Symphony Orchestra: September 9 at Ameris Bank Amphitheatre\r\n* The Avett Brothers: September 14 at Gas South Arena\r\n* Zen Selekta: September 14 at Terminal West\r\n* Darius Rucker: September 20 at Ameris Bank Amphitheatre\r\n* Hail the sun and A Lot Like Birds: September 20 at The Masquerade\r\n* SOFI TUKKER: September 24 at Tabernacle\r\n* Severdust: September 25 at the The Eastern\r\n* Gracie Adams: September 30 at Coca-Cola Roxy\r\n\r\n### Take a Day Trip to North Georgia: Apple-Picking \u0026amp; Sunflowers\r\n\r\n### \r\n\r\nLet\u0026#8217;s face it \u0026#8211; once back-to-school season rolls around, we\u0026#8217;re ready for fall! While it\u0026#8217;s not quite time for pumpkins, haunted houses, and endless screenings of Hocus Pocus just yet \u0026#8211; there\u0026#8217;s plenty of early fall fun to experience in North Georgia!\r\n\r\nFor apple-picking, we love Mercier Orchards in Blue Ridge, GA. For over 80 years, they\u0026#8217;ve delighted guests with \u0026#8220;U-Pick\u0026#8221; experiences and tractor tours. Plus, visit their market to purchase delicious pies, ice cream, baked goods, fresh produce, and more! Mercier\u0026#8217;s apple season is August-October.\r\n\r\nIn mid-late September, head to family-owned Faucett Farms to see endless rows of 900,000\u002B beautiful sunflowers. Be sure to check their website before visiting, as the exact season opening date is TBD based on weather! Located just south of Amicalola Falls and near Burt\u0026#8217;s Pumpkin Farm, this day trip could also be a fun weekend trip! Head to Burt\u0026#8217;s Pumpkin Farm starting September 1, 2024 for pumpkin picking, hayrides, a country store, and more.\r\n\r\n### Additional Atlanta Ideas for September\r\n\r\nLooking for more Atlanta things to do during September? Other popular Atlanta activities in September include:\r\n\r\n* Learn about Dr. King\u0026#8217;s life and legacy on Unexpected Atlanta\u0026#8217;s top-rated Martin Luther King, Jr. Atlanta History Tour\r\n* Explore the walking and cycling paths of the Atlanta Beltline or the Silver Comet Trail\r\n* Visit one of Atlanta\u0026#8217;s many local farmers markets\r\n* See late-summer flowers and the beginnings of fall at the Atlanta Botanical Garden\r\n* Experience on-trend shopping, delicious cuisine, and a rooftop amusement park at Ponce City Market\r\n* Get wild at Zoo Atlanta or take a journey under the sea at Georgia Aquarium\r\n* Take a prohibition themed progressive dinner with a character guide from the 1920s!\r\n* Take a picnic break or toss a football at Centennial Olympic Park in Downtown Atlanta\r\n* Learn about the world\u0026#8217;s most iconic soft drink at the World of Coca-Cola\r\n* Check out bustling Atlanta suburbs like Dunwoody, Roswell, and Chamblee\r\n\r\n## FAQs: Visiting Atlanta in September\r\n\r\n### \r\n\r\n### Is September a good time of year to visit Atlanta?\r\n\r\nYes. In fact, September is one of the best times to visit Atlanta! In September, summer temperatures cool down, there\u0026#8217;s still plenty of sunlight after the 9-5 workday, and back-to-school season means that the city\u0026#8217;s popular destinations are likely to be less crowded.\r\n\r\nPlus, September in Atlanta brings a wide variety of fun \u0026#8211; from sporting events to concerts, festivals, foodie events, and more.\r\n\r\n### What is Atlanta September weather like?\r\n\r\nSeptember is a beautiful and generally comfortable time of year to visit Atlanta. September weather in Atlanta brings sunny days with mild temperatures. In fact, many people are comfortable visiting this time of year with a light sweater or jacket.\r\n\r\nAccording to the National Weather Service, September Atlanta temperatures range from lows around 66 F (19 C) to highs of 84 F (29 C), with occasional rain. During September, Atlanta typically sees 3.8 inches of rain, making this one of the least rainy months to visit. Depending when in September you\u0026#8217;re visiting, average sunrise times in Atlanta in September range from 7:11-7:30 AM. Atlanta September sunsets range from 8:03-7:23 PM as the days get shorter.\r\n\r\nTo make the most of your time in Atlanta, experience the best of the city with expert guides! Unexpected Atlanta\u0026#8217;s unique walking tours are a can\u0026#8217;t-miss activity for anyone who loves good food and history. Join us and see why we have more than 1,500 five-star reviews on TripAdvisor!\r\n\r\nBack to all posts\r\n\r\nRecent Posts* 30\u002B Best Cocktail Bars: Atlanta \u0026#038; Beyond September 1, 2024\r\n* 25\u002B Unique \u0026#038; Fun Date Ideas: Atlanta August 29, 2024\r\n* Atlanta Airport: Ultimate Guide for Travelers August 26, 2024\r\n\r\n## \r\n\r\n### See Tours Available On Your Preferred Date\r\n\r\n[](/tours/grant-park-atlanta-food-tour/)**Book Atlanta\u0027s #1 rated food tour now.**8 tastings from 3 restaurants in a 2.5 hour guided tour through historic Grant Park.### Get Atlanta Tips and Fun In Your Inbox!\r\n\r\n[](#)[Share](#)[Tweet](#)[Share](#)[Pin](#)\u0026copy; 2024 Unexpected Atlanta. All Rights Reserved.\r\n\r\n[Waiver \u0026amp; Release](/waiver-release/)|[Terms \u0026amp; Conditions](/terms-conditions/)|[Privacy Policy](/privacy-policy/)* twitter\r\n* facebook\r\n* pinterest\r\n* instagram\r\n* yelp\r\n* tripadvisor\r\n\r\n[Close Menu](#)* Home\r\n* Public Tours Atlanta Market Food Tour \u002B Biscuit Cooking Class Martin Luther King Jr Tour Cabbagetown \u0026#038; BeltLine Street Art Tour Grant Park Food Tour Prohibition Progressive Dinner\r\n* Team Building\r\n* Family and Student Groups\r\n* Things To Do: Atlanta Blog\r\n* Buy Tickets\r\n* Gift Certificates\r\n\r\n[](#)[Call Us](tel:\u002B14702232203)[Buy Tickets](https://fareharbor.com/embeds/book/atlantafoodwalks/?full-items=yes)[View Tours](/tours/)"
```

Here are some exciting events happening in Atlanta today, September 24, 2024:

### Concerts and Shows
1. **Kehlani** - Lakewood Amphitheatre, 7:00 PM
2. **Signs of The Swarm** - Hell at The Masquerade, 6:00 PM
3. **DPR** - Coca-Cola Roxy, 8:00 PM
4. **Juan Fernando Velasco** - City Winery Atlanta, 8:00 PM
5. **Sofi Tukker** - The Tabernacle, 8:00 PM
6. **Magdalena Bay** - Variety Playhouse, 8:00 PM
7. **Rocco** - Aisle 5, 8:00 PM

### Sports Events
1. **Atlanta Braves vs. New York Mets** - Truist Park, 7:20 PM
2. **Atlanta United II vs. Inter Miami II** - Fifth Third Stadium, 7:30 PM

### Other Events
1. **Victoria Canal** - Vinyl, 8:00 PM
2. **Nervosa** - Boggs Social & Supply, 7:30 PM

For more details and tickets, you can visit the respective venue websites or ticketing platforms. Enjoy your day in Atlanta!

**Sources:**
- [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
- [Atlanta GA Events](https://atlanta-ga.events/september/)
- [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
- [Unexpected Atlanta](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/)

![Kehlani Concert](https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1)

![Atlanta Braves](https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1)


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events happening in Atlanta today, September 24, 2024:

### Concerts and Shows
1. **Kehlani** - Lakewood Amphitheatre, 7:00 PM
2. **Signs of The Swarm** - Hell at The Masquerade, 6:00 PM
3. **DPR** - Coca-Cola Roxy, 8:00 PM
4. **Juan Fernando Velasco** - City Winery Atlanta, 8:00 PM
5. **Sofi Tukker** - The Tabernacle, 8:00 PM
6. **Magdalena Bay** - Variety Playhouse, 8:00 PM
7. **Rocco** - Aisle 5, 8:00 PM

### Sports Events
1. **Atlanta Braves vs. New York Mets** - Truist Park, 7:20 PM
2. **Atlanta United II vs. Inter Miami II** - Fifth Third Stadium, 7:30 PM

### Other Events
1. **Victoria Canal** - Vinyl, 8:00 PM
2. **Nervosa** - Boggs Social & Supply, 7:30 PM

For more details and tickets, you can visit the respective venue websites or ticketing platforms. Enjoy your day in Atlanta!

**Sources:**
- [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
- [Atlanta GA Events](https://atlanta-ga.events/september/)
- [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
- [Unexpected Atlanta](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/)

![Kehlani Concert](https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1)

![Atlanta Braves](https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1)

In [6]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the dates for the next four weekends (Saturday and Sunday)
weekends = [(today + timedelta(days=(5 - today.weekday()) % 7 + i * 7), today + timedelta(days=(6 - today.weekday()) % 7 + i * 7)) for i in range(4)]
weekends
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta September 28 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta September 28 2024"},"webPages":{"value":[{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Find out what\u0026#39;s happening in Atlanta in September 2024, from golf tournaments and circus shows to comedy and music festivals. Browse the list of events by date, category, venue and price, and book your tickets online.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=qjsX0iIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"Find out what\u0026#39;s happening in Atlanta in September 2023, from food festivals and burger battles to haunted attractions and music festivals. Whether you\u0026#39;re into art, culture, nature or sports, there\u0026#39;s something for everyone in this guide.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=Styw6CksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"Find out what\u0026#39;s happening in Atlanta in September 2024, from festivals and concerts to comedy and sports. See dates, venues and details for events such as Dragon Con, Atlanta Food \u0026amp; Wine Festival, Atlanta Ballet and more.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4712253893443652\u0026mkt=en-US\u0026setlang=en-US\u0026w=PlBFAVbynJFe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"Find out the best fall festivals in Atlanta in 2024, from Labor Day Weekend to Thanksgiving. Enjoy carnival rides, music, arts, food, and more at state and county fairs, harvest celebrations, and cultural events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=Huo5zztTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"There\u2019s so much going on in Atlanta this September, so keep scrolling for our monthly round-up of unmissable things to do in the city. Now it\u2019s time to enjoy the remainder of summer and get ready for the exciting season change that\u2019s just around the corner! 1. Take a journey into the tragic past with Titanic: An Immersive Voyage","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=dO4bzEX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cGQLTADJ7EIEL8n56X81XwHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://deenaqrubetta.pages.dev/wbsyfbe-things-to-do-in-atlanta-september-2024-lvjgugj/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Yac324FtG-dH_KY8XaUTZQHaME\u0026pid=Api","contentUrl":"https://www.frostandsun.com/wp-content/uploads/2017/11/things-to-do-in-atlanta-pin-me-1-628x1024.jpg.webp","hostPageUrl":"https://www.frostandsun.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.M9nQL4kFM4rwNQiRkhAekAHaEo\u0026pid=Api","contentUrl":"https://travellersworldwide.com/wp-content/uploads/2022/05/shutterstock_1039281055.jpg","hostPageUrl":"https://travellersworldwide.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.9mBzTpymEhPfPfnOyuyhzAAAAA\u0026pid=Api","contentUrl":"https://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.eKakKX-7lUL-8AAx78alPQHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.SAR7OZFxENgNvYAs2Dxz5wHaE8\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/eb/f7/cc/ebf7cc88f32b7dea83f4ad2f824751e9.jpg","hostPageUrl":"https://www.pinterest.com/pin/13-best-things-to-do-in-downtown-atlanta-for-firsttime-visitors--40532465387430446/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zco_XrsR7HWXyvmCHgOF7wAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/f_auto,q_auto/v1602606068/newAtlanta.com/Atlanta_OneMusicFest_Crowd/Atlanta_OneMusicFest_Crowd.jpg","hostPageUrl":"https://discoveratlanta.com/things-to-do/nightlife/best-festivals-in-the-atl/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.2BMgx8s6cNmdIkBlj10UIgHaEK\u0026pid=Api","contentUrl":"https://cdn.getyourguide.com/img/location/545b660b0923a.jpeg/88.jpg","hostPageUrl":"https://www.getyourguide.co.uk/atlanta-l503/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Tg7nOHaEU_re0ukKIjO-QwHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/uploads/aVxdjP6v0zGPok5GHYOee8lHRMVN0GMFiNomqe5s.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026pid=Api","contentUrl":"https://gosouthatlanta.com/images/atlanta-tree-canopy-social-media.jpg","hostPageUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H1JPxx3mkr-XvYjeXre46AHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.8fGU9Kl3J5zAfIfkKg-H5AHaGN\u0026pid=Api","contentUrl":"https://blog.trekaroo.com/wp-content/uploads/2021/03/Things-to-do-in-Atlanta-with-Kids-Dowtown.jpg","hostPageUrl":"https://blog.trekaroo.com/top-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Q1-IywcFZIjlWKI_LcqH_gHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/fun-things-to-do-in-Atlanta-3.jpg","hostPageUrl":"https://deenaqrubetta.pages.dev/wbsyfbe-things-to-do-in-atlanta-september-2024-lvjgugj/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.AsEhWz77ez1CIx9RVZsvFAHaEh\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/v1636654462/newAtlanta.com/Atlanta_OneMusicFest_COP-1/Atlanta_OneMusicFest_COP-1.jpg?_i=AA","hostPageUrl":"https://discoveratlanta.com/stories/things-to-do/is-atlanta-a-good-place-to-visit-here-are-the-top-35-reasons/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg?is-pending-load=1","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yFuF5f3VNHx1W79aqvNTMAHaEa\u0026pid=Api","contentUrl":"https://familydestinationsguide.com/wp-content/uploads/2020/11/Best-Things-to-Do-in-Atlanta-GA-FTR.jpg","hostPageUrl":"https://familydestinationsguide.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Z_ovzOowOSo8cB8hxjKVigHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.PzRE6SJDtJ9H3Z9jEfK5cAHaE8\u0026pid=Api","contentUrl":"https://image.jimcdn.com/app/cms/image/transf/dimension=640x10000:format=jpg/path/s9524a0afdc5b40b0/image/i88ad01ce4abaee84/version/1483187737/8-fun-things-to-do-in-atlanta-travel-city-guide-via-just1wayticket.jpg","hostPageUrl":"https://www.justonewayticket.com/2016/06/14/8-fun-things-to-do-in-atlanta/"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta October 5 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta October 5 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=75Rt4YP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=2TJUg1LZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=rWg4vvDfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 5, 2024 Town Center Park Suwanee FREE admission. This all-day music festival takes place in downtown Suwanee. Admission, parking, and entertainment is free. The BBQ tasting tickets s are $1 each, and each taste requires one to four tickets. You can also buy a full plated barbecue meal. There is also a kids fun area and an artist market.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=Huo5zztTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta October 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/october","displayUrl":"https://www.experism.com/atlanta/october","snippet":"Atlanta Horror Film Festival. Oct 4-6, 2024 Limelight Theater View on Map. You are cordially invited to the 18th Annual Atlanta Horror Film Festival held at Limelight Theater from October 4 - 6, 2024. Don\u0026#39;t miss this spooktacular collection of the best indie horror films from across the globe, including world premieres, short films, animation ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5\u002B2024\u0026d=5050860516094956\u0026mkt=en-US\u0026setlang=en-US\u0026w=PvVBBJkZyN343ZtIpGRATFj2b4uP5Oth","thumbnailUrl":"https://www.bing.com/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta October 12 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta October 12 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=75Rt4YP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=2TJUg1LZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=rWg4vvDfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=Huo5zztTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fall festivals in metro Atlanta and North Georgia | 2024","url":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","displayUrl":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","snippet":"When: Oct. 12-15, 2024 Where : Piedmont Park, Atlanta What : One of the largest LGBTQ\u002B pride events in the Southeast, featuring a parade, live performances, food, and family-friendly activities.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12\u002B2024\u0026d=3169614236550\u0026mkt=en-US\u0026setlang=en-US\u0026w=Fpd9OLcRuACRcKNV8UVNHXKEd0PX1nap"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta October 19 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta October 19 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 | Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Things to do in Atlanta, GA in October, including upcoming festivals, fall and Halloween events, Pride events, comedy, musicals, plays, races, conventions, neighborhood events, cultural events, and more. ... \u2013 PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024 Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=DKKAeoP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... 19 tickets remaining! Tickets; Oct. 02. 2024. 8:00 PM Wed. The Hives. The Tabernacle - GA | Capacity: 2600. 30303, 152 Luckie St, Atlanta, GA, US","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"26 Thrilling Things To Do In Atlanta This October | Secret Atlanta","url":"https://secretatlanta.co/things-to-do-october/","displayUrl":"https://secretatlanta.co/things-to-do-october","snippet":"Without further ado, here are 26 incredible things to do in Atlanta this October! 1. ... 19. Get lost in these enchanting corn mazes near Atlanta. Jaemor Farms. ... September 20, 2024. 18 Frightfully Fun Halloween Events Happening In Atlanta. September 20, 2024 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19\u002B2024\u0026d=4673096676868625\u0026mkt=en-US\u0026setlang=en-US\u0026w=BDZ6Krx8ATVNFCQACSuK0PesDCkO7vK5","thumbnailUrl":"https://www.bing.com/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/september/"}
```
and got this result:
```
"Atlanta Events September 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* September\r\n\r\n# Atlanta Events September 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices.\r\n\r\nStart DateSep 24, 2024ResetCancelApplyEnd DateSep 26, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 120* Sep 24 2024 6:00 PM Tue Signs of The Swarm Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $42 16 tickets remaining! Tickets\r\n* Sep 24 2024 7:00 PM Tue Kehlani Lakewood Amphitheatre | Capacity: 19000 30315, 2002 Lakewood Avenue Southeast, Atlanta, GA, US Prices from $51 Avg. price ~ $140 Available Tickets: 227 Tickets\r\n* Sep 24 2024 7:20 PM Tue Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $15 Avg. price ~ $96 Available Tickets: 1074 Tickets\r\n* Sep 24 2024 7:30 PM Tue Atlanta United II vs. Inter Miami II Fifth Third Stadium | Capacity: 10200 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Prices from $21 Avg. price ~ $25 6 tickets remaining! Tickets\r\n* Sep 24 2024 7:30 PM Tue Nervosa Boggs Social \u0026 Supply 30310, 1310 White St. SW, Atlanta, GA, US Prices from $73 Avg. price ~ $101 15 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue DPR Coca-Cola Roxy | Capacity: 3600 30339, 800 Battery Ave SE, Atlanta, GA, US Prices from $53 Avg. price ~ $78 21 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Victoria Canal Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $31 Avg. price ~ $36 16 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Juan Fernando Velasco City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $89 Avg. price ~ $141 50 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Sofi Tukker The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $31 Avg. price ~ $80 Available Tickets: 175 Tickets\r\n* Sep 24 2024 8:00 PM Tue Magdalena Bay Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $57 Avg. price ~ $90 18 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Rocco Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $26 Avg. price ~ $98 15 tickets remaining! Tickets\r\n* Sep 25 2024 6:30 PM Wed Lorna Shore The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $34 Avg. price ~ $148 Available Tickets: 110 Tickets\r\n* Sep 25 2024 6:30 PM Wed Sevendust The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $44 Avg. price ~ $51 20 tickets remaining! Tickets\r\n* Sep 25 2024 7:00 PM Wed Creed, 3 Doors Down \u0026 Finger Eleven Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $49 Avg. price ~ $223 Available Tickets: 289 Tickets\r\n* Sep 25 2024 7:20 PM Wed Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $8 Avg. price ~ $62 Available Tickets: 2487 Tickets\r\n* Sep 25 2024 7:30 PM Wed Kings of Leon \u0026 Phantogram State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $10 Avg. price ~ $132 Available Tickets: 978 Tickets\r\n* Sep 25 2024 8:00 PM Wed Gritz and Jelly Butter City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $74 Avg. price ~ $89 8 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed Lasso - Artist Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $58 Avg. price ~ $91 Available Tickets: 64 Tickets\r\n* Sep 25 2024 8:00 PM Wed Shaboozey Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $79 Avg. price ~ $119 27 tickets remaining! Tickets\r\n* Sep 26 2024 6:30 PM Thu Rain City Drive Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $35 Avg. price ~ $41 22 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $94 Avg. price ~ $111 8 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Mico Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $83 29 tickets remaining! Tickets\r\n* Sep 26 2024 7:20 PM Thu Atlanta Braves vs. New York Mets VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $15 Avg. price ~ $150 Available Tickets: 306 Tickets\r\n* Sep 26 2024 7:30 PM Thu Khruangbin Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $51 Avg. price ~ $131 Available Tickets: 296 Tickets\r\n* Sep 26 2024 7:30 PM Thu Disney On Ice Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $41 Avg. price ~ $93 Available Tickets: 1084 Tickets\r\n* Sep 26 2024 8:00 PM Thu The California Honeydrops Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $33 Avg. price ~ $54 25 tickets remaining! Tickets\r\n* Sep 26 2024 8:00 PM Thu Karina Rykman Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $36 Avg. price ~ $96 8 tickets remaining! Tickets\r\n* Sep 26 2024 8:00 PM Thu The Buena Vista Social Orchestra Center Stage Theatre | Capacity: 1050 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Prices from $64 Avg. price ~ $101 Available Tickets: 255 Tickets\r\n* Sep 26 2024 8:00 PM Thu Lawrence The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $29 Avg. price ~ $88 Available Tickets: 108 Tickets\r\n* Sep 26 2024 8:00 PM Thu Wasia Project Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $26 Avg. price ~ $35 16 tickets remaining! Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[Signs of the Swarm Upcoming Events: 2 Hell at The Masquerade - GA Sep 24, 2024 6:00 PM](https://atlanta-ga.events/tickets/signs-of-the-swarm/)[Kehlani Upcoming Events: 2 Lakewood Amphitheatre Sep 24, 2024 7:00 PM](https://atlanta-ga.events/tickets/kehlani/)[New York Mets Upcoming Events: 9 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/new-york-mets/)[Atlanta Braves Upcoming Events: 18 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/atlanta-braves/)[Atlanta United II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/atlanta-united-ii/)[Nervosa Upcoming Events: 2 Boggs Social \u0026 Supply Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/nervosa/)[Inter Miami II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/inter-miami-ii/)[Rocco Upcoming Events: 2 Aisle 5 Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/rocco/)[Victoria Canal Upcoming Events: 2 Vinyl - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/victoria-canal/)[Sofi Tukker Upcoming Events: 2 The Tabernacle - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/sofi-tukker/)[DPR Live Upcoming Events: 2 Coca-Cola Roxy Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/dpr-live/)[Magdalena Bay Upcoming Events: 2 Variety Playhouse Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/magdalena-bay/)[Juan Fernando Velasco Upcoming Events: 2 City Winery - Atlanta Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/juan-fernando-velasco/)[Lorna Shore Upcoming Events: 2 The Tabernacle - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/lorna-shore/)[Sevendust Upcoming Events: 2 The Eastern - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/sevendust/)[Creed Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/creed/)[3 Doors Down Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/3-doors-down/)[Finger Eleven Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/finger-eleven/)[Kings Of Leon Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/kings-of-leon/)[Phantogram Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/phantogram/)[Shaboozey Upcoming Events: 2 Variety Playhouse Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/shaboozey/)[Gritz and Jelly Butter Upcoming Events: 2 City Winery - Atlanta Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)[Lasso - Artist Upcoming Events: 2 Buckhead Theatre Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/lasso-artist/)[Rain City Drive Upcoming Events: 2 Hell at The Masquerade - GA Sep 26, 2024 6:30 PM](https://atlanta-ga.events/tickets/rain-city-drive/)[Lyfe Jennings Upcoming Events: 6 City Winery - Atlanta Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/lyfe-jennings/)[Mico Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/mico/)[Disney On Ice Upcoming Events: 24 Tulsa Expo Square - Pavilion Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/disney-on-ice/)[Khruangbin Upcoming Events: 6 Fabulous Fox Theatre - Atlanta Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/khruangbin/)[The California Honeydrops Upcoming Events: 2 Variety Playhouse Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-california-honeydrops/)[Wasia Project Upcoming Events: 2 Terminal West At King Plow Arts Center Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/wasia-project/)[Jesus Adrian Romero Upcoming Events: 2 Cobb Energy Performing Arts Centre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/jesus-adrian-romero/)[Atlanta Symphony Orchestra Upcoming Events: 6 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)[Nathalie Stutzmann Upcoming Events: 4 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/nathalie-stutzmann/)[Starletta Upcoming Events: 2 Vinyl - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/starletta/)[Lawrence - Band Upcoming Events: 2 The Tabernacle - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/lawrence-band/)[The Buena Vista Social Orchestra Upcoming Events: 2 Center Stage Theatre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-buena-vista-social-orchestra/)[Karina Rykman Upcoming Events: 2 Aisle 5 Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/karina-rykman/)[Red Lace Burlesque Show and Variety Show Upcoming Events: 4 Red Lace Burlesque - Myrtle Beach Sep 27, 2024 6:00 PM](https://atlanta-ga.events/tickets/red-lace-burlesque-show-and-variety-show/)[The Warning Upcoming Events: 2 Hell at The Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-warning/)[Coral Moons Upcoming Events: 2 Altar at Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/coral-moons/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html"}
```
and got this result:
```
"Atlanta Events - October 2024This site uses cookies. Opt out at any time. More details\r\n\r\nHome \u0026rsaquo; Things To Do \u0026rsaquo; October Events\r\n\r\n# Atlanta Events - October 2024\r\n\r\nInterested in a specific type of event? See Atlanta\u0026rsquo;s calendars of upcoming: \u0026ndash; Comedy shows \u0026ndash; Beer and wine festivals \u0026ndash; Music festivals \u0026ndash; Musicals \u0026ndash; Food festivals\r\n\r\nSee also: \u0026ndash; Concerts in Atlanta in October \u0026ndash; Atlanta\u0026rsquo;s biggest annual festivals \u0026ndash; Atlanta weather in October \u0026ndash; Things to do in Atlanta\r\n\r\n!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.\r\n\r\n## Events From October 1 To The Weekend Of October 5-6 2024\r\n\r\n\u0026rarr; See September events\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; The Moth Atlanta GrandSLAM, October 1 2024Storytelling competition at Center Stage. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Kumail Nanjiani: Doing This Again, October 4 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Stand Up Comedy at the Clay! Hosted by Yoshee, October 4 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; ExpoContratista, October 4-5 2024Hispanic construction trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; How to Train Your Dragon in Concert with the ASO, October 4-5 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Horror Film Festival, October 4-6 2024Annual festival of horror film at Limeliight Theater. More details\r\n\r\n\u0026ndash; DreamHack Atlanta, October 4-6 2024Annual esports convention at the Georgia World Congress Center with tournament prizes across 20 gaming contests. More details\r\n\r\n\u0026ndash; Atlanta Sci-Fi Film Festival, October 4-6 2024Annual science fiction film festival in Peachtree Corners. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, October 4-6 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; Wings \u0026amp; Whitewalls, October 5 2024Fall car show at the Aviation History \u0026amp; Technology Center, Marietta, with vintage aircraft, cars and more. More details\r\n\r\n\u0026ndash; Taste of Tucker, October 5 2024Annual food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music and more. More details\r\n\r\n\u0026ndash; EGGtoberfest, October 5 2024Barbecue festival at Coolray Field, Lawrenceville, with cook-off and cooking demonstrations, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; WWE Bad Blood, October 5 2024Wrestling event at State Farm Arena. More details\r\n\r\n\u0026ndash; Festival on Ponce, October 5-6 2024Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; Candler Park Fall Fest, October 5-6 2024Popular annual festival in Candler Park and neighborhood, with live music, art, food, rides, games and more. More details\r\n\r\n\u0026ndash; Atlanta Arab Festival, October 5-6 2024Annual festival of Arabic food and culture at the Alif Institute, with traditional dishes from Arabic countries, live performances, vendors, exhibits, workshops, games and more. More details\r\n\r\n\u0026ndash; Aladdin, October 5-6 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; OYE Fest, October 6 2024Annual festival in Historic Fourth Ward Park celebrating Latinx art and music. More details\r\n\r\n\u0026ndash; La Cotorrisa Tour, October 6 2024Spanish-language comedy podcast show at the Buckhead Theatre. More details\r\n\r\n## Events From October 7 To Columbus Day Weekend, October 12-14 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Atlanta miniFEST, October 9 2024Community music and arts festival at Believe Music Hall. More details\r\n\r\n\u0026ndash; Taste Brookhaven, October 10 2024Food-tasting event in the gardens of the Hyatt Regency Atlanta Perimeter, featuring Brookhaven chefs and restaurants plus drinks and live entertainment, to benefit Children\u0026rsquo;s Healthcare of Atlanta. More details\r\n\r\n\u0026ndash; Jake Shane, October 10 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Scott Antique Market, October 10-13 2024Monthly antiques market at the Atlanta Expo Center. More details\r\n\r\n\u0026ndash; Atsuko Okatsuka, October 11 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Pride Kickoff, October 11 2024After-hours LGBTQ\u002B party at the Georgia Aquarium. More details\r\n\r\n\u0026ndash; Wire \u0026amp; Wood Alpharetta, October 11-12 2024Songwriter music festival with performances across downtown Alpharetta. More details\r\n\r\n\u0026ndash; Joyce Meyer Live, October 11-12 2024Faith event at the Gas South Arena. More details\r\n\r\n\u0026ndash; Take Your Royal Position Conference, October 11-12 2024Faith and empowerment event in downtown Atlanta. More details\r\n\r\n\u0026ndash; Duluth Fall Home Expo, October 11-13 2024Home renovations trade show at the Gas South Convention Center. More details\r\n\r\n\u0026ndash; Oakhurst Porchfest, October 12 2024Annual community music festival in Decatur over 100 live performances on Oakhurst neighborhood porches. More details\r\n\r\n\u0026ndash; Madison Avenue Derby, October 12 2024Annual races in Decatur in homemade, human-powered soapbox cars, to benefit local children\u0026rsquo;s charities. More details\r\n\r\n\u0026ndash; Politics War Room with James Carville \u0026amp; Al Hunt, October 12 2024Talk event at Variety Playhouse. More details\r\n\r\n\u0026ndash; Atlanta Pride Festival, October 12-13 2024One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace and more. More details\r\n\r\n\u0026ndash; Taste \u0026amp; Brews Fall Festival, October 12-13 2024Annual food and beer festival at Etowah River Park, Canton, featuring southern food and beers, musicians from across Georgia, arts and crafts, farmers market, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Old Fourth Ward Fall Arts Festival, October 12-13 2024Twice-annual arts festival in Historic Fourth Ward Park featuring local and Georgian artists, demonstrations, live entertainment, art activities, food and more. More details\r\n\r\n\u0026ndash; Chalktoberfest, October 12-13 2024Popular annual art and beer festival in Marietta Square, featuring professional chalk artists, 120\u002B craft beers, live music, arts and crafts, food trucks, amateur chalk art competition, and more. More details\r\n\r\n\u0026ndash; FashionFest, October 12-13 2024Festival of urban-popular fashion and streetwear at Ambient Plus Studio, with fashion and footwear vendors, food, music and more. More details\r\n\r\n\u0026ndash; Gold Over America Tour, October 13 2024Gymnastics performance at the Gas South Arena. More details\r\n\r\n\u0026ndash; Roswell Town Square Dinner Series, October 13 2024Outdoor community dining event at Roswell Town Square with food from The Mill Kitchen \u0026amp; Bar. More details\r\n\r\n\u0026ndash; Pride Parade Viewing Party, October 13 2024Rooftop LGBTQ\u002B party at the Fox Theatre, to coincide with the annual Pride parade. More details\r\n\r\n\u0026ndash; Joe Wong, October 14 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n## Events From October 15 To The Weekend Of October 19-20 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; David Cross \u0026ndash; End of the Beginning of the End, October 16 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Jessica Kirson, October 18 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Sharon McMahon, October 18 2024Author talk at Center Stage. More details\r\n\r\n\u0026ndash; Stone Mountain Highland Games and Scottish Festival, October 18-20 2024Annual festival of Scottish heritage at Stone Mountain Park, with traditional Scottish athletics, music, dancing, food and drink, and vendors. All are welcome. More details\r\n\r\n\u0026ndash; Dracula: Presented by Gwinnett Ballet Theatre, October 18-20 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; Georgia Metropolitan Dance Theatre presents Dracula, October 18-20 2024Ballet performance at the KSU Dance Theatre, Marietta Campus. More details\r\n\r\n\u0026ndash; A Taste of Chamblee, October 19 2024Annual food festival in downtown Chamblee, with food from neighborhood restaurants, craft vendors, children\u0026rsquo;s activities, football screenings, live performances, wine tent, beer garden and more. More details\r\n\r\n\u0026ndash; HarvestFest and Scarecrows In The Square, October 19 2024Annual fall festival in Marietta Square, with arts and crafts show, touch-a-truck, scarecrow displays, costume and pie eating contests, and more. More details\r\n\r\n\u0026ndash; Pop-In For Family Fun: Boo! Halloween, October 19 2024Monthly free family day at the Marietta History Center. More details\r\n\r\n\u0026ndash; Cherokee Heights Arts Festival, October 19 2024Free annual arts festival in the Cherokee Heights neighborhood, with local artists and makers, local musicians, art activities, food, beer and more. More details\r\n\r\n\u0026ndash; NBAF Celebration Under The Stars, October 19 2024Fundraising event for the National Black Arts Festival at Wolf Creek Amphitheater, with live performances and food and drink. More details\r\n\r\n\u0026ndash; Rickey Smiley, October 19 2024Comedy show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Carlos Ballarta: Tlatoani, October 19 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Witch Perfect, October 19 2024Drag performance at Center Stage. More details\r\n\r\n\u0026ndash; L5P Halloween Festival and Parade, October 19-20 2024Popular annual Halloween celebration in Little Five Points, with live music, parade, haunted house, ghost tours and more. More details\r\n\r\n\u0026ndash; PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station. More details\r\n\r\n\u0026ndash; Johns Creek Arts Festival, October 19-20 2024Annual arts and crafts festival in Johns Creek featuring 140 artisans from across the country. More details\r\n\r\n\u0026ndash; Abnormal Market and Uncommon Bazaar, October 19-20 2024Art market in association with the Old Fourth Ward Park Arts Festival. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Landlocked 8, October 20 2024Oyster and BBQ festival at Legacy Park, Decatur. More details\r\n\r\n\u0026ndash; MrBallen: The Strange, Dark, \u0026amp; Mysterious Live Tour, October 20 2024Podcast live show at the Coca-Cola Roxy. More details\r\n\r\n\u0026ndash; National Ballet of Ukraine, October 20 2024Ballet performance at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Shakespeare Out of a Hat: Macbeth, October 20 2024Part-improvised play at the Shakespeare Tavern Playhouse. More details\r\n\r\n## Events From October 21 To The Weekend Of October 26-27 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Malcolm Gladwell in conversation with Virginia Prescott \u0026ndash; Revenge of the Tipping Point, October 22 2024Author talk at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Jim Henson\u0026rsquo;s Labyrinth: In Concert, October 23 2024Film screening with live musical accompaniment at The Eastern. More details\r\n\r\n\u0026ndash; A Conversation with Larry David, October 24 2024Talk event at the Fox Theatre. More details\r\n\r\n\u0026ndash; Whitney Cummings: Big Baby, October 25 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Midtown Halloween Block Party, October 25 2024Halloween-themed block party and costume contest at Politan Row at Colony Square. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part I, October 25 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Benteen Park Fall Fest, October 26 2024Annual fall festival in Benteen Park, with live music, artist market and food vendors. More details\r\n\r\n\u0026ndash; Yappy Hour, October 26 2024Pet-friendly event series at The Hamilton Hotel Alpharetta, with pet adoptions and specialty cocktails. More details\r\n\r\n\u0026ndash; Jerry Seinfeld, October 26 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Felipe Esparza, October 26 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Halloween in the Highland, October 26 2024Halloween-themed block party and costume contest in the Virginia-Highland neighborhood. More details\r\n\r\n\u0026ndash; Buckhead Creepy Crawl, October 26 2024Halloween-themed bar crawl and costume contest in the Buckhead entertainment district. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Air Dot Show Tour, October 26-27 2024Air show at Atlanta Regional Airport, Peachtree City. More details\r\n\r\n\u0026ndash; ONE Musicfest, October 26-27 2024Big annual festival of urban progressive music, in Piedmont Park. More details\r\n\r\n\u0026ndash; Menopause The Musical 2: Cruising Through \u0026lsquo;The Change\u0026rsquo;, October 26-27 2024Musical at the Gas South Theater. More details\r\n\r\n\u0026ndash; CompassionCon, October 27 2024Annual community wellness festival in Legacy Park, Decatur, with art, music, wellness activities, health services, food trucks and more. More details\r\n\r\n\u0026ndash; Hillary Clinton Live, October 27 2024Author talk at the Fox Theatre. More details\r\n\r\n## Events From October 28 To THe Weekend Of November 2-3 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Historical Marker Dedication: Booker T Washington High School, October 29 2024Historical marker dedication by the Georgia Historical Society at Atlanta\u0026rsquo;s first public secondary school for Black students, at Booker T Washington High School, 45 Whitehouse Drive SW. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part II, October 31 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Native American Festival and Pow Wow, October 31 - November 3 2024Big annual festival of Native American heritage at Stone Mountain Park, with live traditional dance, music and storytelling performances, craft demonstrations, wildlife presentations, arts and crafts and more. More details\r\n\r\n\u0026ndash; Two Hot Takes Podcast: No Takes Left Behind, November 1 2024Podcast live show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Black Girls Rock!, November 1 2024Annual awards ceremony at the Fox Theatre celebrating the accomplishments of Black women. More details\r\n\r\n\u0026ndash; Ghostbusters in Concert, November 1-2 2024Film screening with live orchestral accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Party With Impact \u0026ndash; Positive Impact Health Centers, November 1-2 2024Comedy show at the Gas South Theater to benefit LGBTQ\u002B healthcare services in Atlanta. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, November 1-3 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Rescue Dog Games: Howl-o-Ween Festival, November 2 2024Annual pet-friendly festival in Piedmont Park, with dog activities, dog costume parade and contest, pet-centric vendors, live music, food and beer, adoptable pets, and more. More details\r\n\r\n\u0026ndash; Chomp and Stomp, November 2 2024Annual neighborhood festival in Cabbagetown, with chili, beer, bluegrass music and more, to benefit the Friends of Cabbagetown Conservancy. More details\r\n\r\n\u0026ndash; D\u0026iacute;a de los Muertos Festival, November 2 2024Free family festival at Woodruff Arts Center, with live music, mariachi band, vendors, food trucks, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Suwanee Wine Fest, November 2 2024Annual festival in Suwanee Town Center Park, with wine samples from across the world, Georgia wines, live music, arts and crafts vendors, food and more. More details\r\n\r\n\u0026ndash; \u0026ldquo;Honoring Our Past Through Stories\u0026rdquo; Historic Landscape Preservation Grant Fund Benefit, November 2 2024Outdoor event at the Marietta Educational Garden Center to benefit the Garden Club of Georgia\u0026rsquo;s Historic Landscape Preservation Grant, with speakers, plein air painters, exhibits, music and more. More details\r\n\r\n\u0026ndash; Chastain Park Arts Festival, November 2-3 2024Popular twice-annual arts festival at Chastain Park, with 175\u002B arts and crafts vendors, food trucks, local musicians, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; The Elegant Elf Marketplace, November 2-3 2024Annual holiday marketplace with 80\u002B vendors at the Sandy Springs Performing Arts \u0026amp; Conference Center, to benefit The Sandy Springs Society. More details\r\n\r\n\u0026ndash; Jeff Arcuri: The Full Beans Fall Tour, November 3 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Disney Jr Live On Tour: Let\u0026rsquo;s Play, November 3 2024Children\u0026rsquo;s performance at the Fox Theatre. More details\r\n\r\n\u0026rarr; See November events\r\n\r\n\u0026copy; Melanie K Jones 2024\r\n\r\n* Privacy\r\n* Terms\r\n* Contact\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/"}
```
and got this result:
```
"Cool Things To Do in September in Atlanta - Discover Atlanta[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading# Cool Things To Do in September in Atlanta\r\n\r\nBy Mary Welch\r\n\r\nPublished August 23, 2024Last Updated September 11, 2024\r\n\r\nIt\u2019s September. In Atlanta that means it\u2019s time for festivals, costumed aliens walking down Peachtree Street and food \u2014 glorious food.\r\n\r\n## Food Extravaganza\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723648163/newAtlanta.com/ATL-food-Credit-raftermen-photography-copy/ATL-food-Credit-raftermen-photography-copy.jpg?_i=AA)\r\nTake a big bite. There is plenty to sample at the Atlanta Food \u0026#038; Wine Festival with more than 200 chefs, sommeliers and mixologists. (Photo by Raftermen Photography)The Atlanta Food \u0026amp; Wine Festival returns to Historic Old Fourth Ward Park for a four-day celebration of incomparable food, wine, beer and craft spirits. More than 200 of the most celebrated chefs, sommeliers, mixologists and industry insiders will offer bites and drinks that showcase the exceptional flavors of the region. The festival runs Sept. 12-15.\r\n\r\n## Burger Bash\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647613/newAtlanta.com/Burger-benefit-resized/Burger-benefit-resized.jpg?_i=AA)\r\nThe team from Fox Bros. BBQ takes to the grill to showcase their burger skills at the Burger Benefit Sept. 22. (Photo courtesy of NFA Burger)Some of the city\u2019s and the nation\u2019s leading chefs including personality Andrew Zimmern; Shai Livi of The Third Space; Billy Kramer of NFA Burger; and Nick Leahy of Vice Kitchen will duke it out for an all-day burger showdown at NFA Burger in Chamblee. The Burger Benefit takes place Sept. 22 and features four burger masters commanding the grill in shifts of 90 minutes. Last year the burger benefit raised more than $53,000 in support of The Giving Kitchen, an Atlanta-based nonprofit for hospitality workers. Tickets start at $30, which includes one burger and a side.\r\n\r\n## Spooky Time\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg?_i=AA)\r\nNecrotis is just one of the friendly characters waiting to greet \u2014 and scare \u2014 you at NETHERWORLD. (Photo courtesy of NETHERWORLD)As the haunt season approaches, NETHERWORLD is opening on weekends starting Sept. 20 and 21 and will continue for 34 nights of terror through September, October and November. For its 28th Season of Screams, the attraction is offering two terrifying new haunts, Wake the Dead and Mr. Grendel\u2019s Birthday Party of Horrors. Also returning is Dr. Octane\u2019s Bizarre Beverages featuring colorful, bubbling nonalcoholic drinks served in an optional collectible light-up glass. There\u2019s no shortage of Halloween things to do in Atlanta.\r\n\r\n## Harvest on the Hooch\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_790,h_1024,c_scale/v1723648296/newAtlanta.com/Harvest-on-the-hooch/Harvest-on-the-hooch.jpg?_i=AA)\r\nEnjoy getting together with friends at the ninth annual Harvest on the Hooch at Chattahoochee Nature Center. (Photo courtesy of Chattahoochee Nature Center)The Chattahoochee Nature Center is hosting the ninth annual Harvest on the Hooch Sept. 22 to raise awareness for the Nature Center\u2019s Unity Garden, which has supplied more than 50 tons of fresh produce to North Fulton Community Charities food pantry. The event will feature food tastings from local chefs, live bluegrass music, garden games, samples from local craft breweries, face painting, and meet and greets with animals.\r\n\r\n## Hispanic Heritage Month\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647690/newAtlanta.com/hispanic-festival/hispanic-festival.jpg?_i=AA)\r\nEnjoy a cultural experience Sept. 17 for the Fiestas Patrias, celebrating Mexican Independence.Plaza Fiesta on Buford Highway will host a Fiestas Patrias celebration Sept. 17. The Mexican Consulate\u2019s \u201CEl Grito \u2014 Mexican Independence Celebration\u201D has been a local favorite for more than 19 years. Last year more than 60,000 people celebrated. Plan to go and enjoy a cultural experience that embraces heritage, entertainment and community. Celebrate Hispanic Heritage Month around Atlanta.\r\n\r\n## Baldie Con\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg?_i=AA)\r\nBaldie Con is an empowerment conference for the bald community.Happening at the JW Marriott Atlanta Buckhead, Baldie Con\u0026nbsp;celebrates, empowers and unites women and children who are bald (Baldies) from alopecia, lupus, cancer or by choice. The three-day conference starts with a meet and greet on Sept. 27. There will be a Baldie Ball with an award ceremony and gala dinner on Saturday night. The weekend wraps up on Sept. 29 with a Sunday Jazz Brunch.\r\n\r\n## It\u2019s so Sweet\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg?_i=AA)\r\nAuburn Avenue is where you\u2019ll find the Sweet Auburn Music Fest.The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.\r\n\r\n## Forward Warrior\r\n\r\nThe annual Forward Warrior live art event is happening all day, 9 a.m. until 9 p.m. Saturday, on Wylie Street in Cabbagetown.\r\n\r\n## Celebrate a President\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723649158/newAtlanta.com/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb.jpg?_i=AA)\r\nThe Fox Theatre plans a musical extravaganza to celebrate President Carter\u2019s 100th birthday. (Photo courtesy of Fox Theatre)Jimmy Carter turns 100 on Oct. 1, and the Fox Theatre is celebrating in a big way. Presented by Delta Air Lines, a variety of artists including Chuck Leavell, a Georgia resident and keyboardist for the Rolling Stones, D-Nice, Eric Church, Aren Morris and the Atlanta Symphony Orchestra Chamber Chorus will take the stage Sept. 17 to present a musical extravaganza. The event is a fund-raiser for the Carter Center.\r\n\r\n## Sandy Springs Salutes Creativity\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_658,c_scale/v1723648070/newAtlanta.com/sandy-springs-arts-festival/sandy-springs-arts-festival.jpg?_i=AA)\r\nA couple takes a break from the Sandy Springs Arts Festival to enjoy people watching. (Photo courtesy of Caren West PR)The Sandy Springs Fall Arts Festival returns on Sept. 14-15, featuring more than 150 painters, photographers, sculptors, leather and metal craft persons, glass blowers, jewelers and more. Here are more September (and fall) festivals to visit.\r\n\r\nRead More* Cool Things to Do in November in Atlanta\r\n* Cool Things To Do in October in Atlanta\r\n* Cool Things To Do in August in Atlanta\r\n\r\n## Dragon Con Takes Over\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1723647734/newAtlanta.com/Media_Relations23-1024x682-1/Media_Relations23-1024x682-1.jpg?_i=AA)\r\nYou never know who you\u2019ll meet on Peachtree Street during Dragon Con. (Photo courtesy of Dragon Con Photography \u00C2\u00A9 Dragon Con, Inc.)Dragon Con, the worlds largest multimedia, popular culture convention focusing on science fiction and fantasy, gaming, comics, literature, art, music and film in the universe comes to Atlanta and, frankly, takes over the city. Walk the streets and meet thousands of costumed folks participating in the workshops, meet and greets with celebrities, the art show, and comic and pop artist alley. But, the biggest fun comes with the parade down Peachtree Street on Aug. 31. Among the celebrities appearing at Dragon Con, which takes place Aug. 29-Sept. 2, are: Gil Gerard, Erin Grey, Seth Green, Greg Grunberg, Cobie Smulders and Ming-Na Wen.\r\n\r\n## Piedmont Park Celebrates 120 Years\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_1024,c_scale/v1725563202/newAtlanta.com/P4PP-IG-Post/P4PP-IG-Post.png?_i=AA)\r\nParty like it\u2019s 1989 as Piedmont Parks celebrates its 120th birthday and the 35th anniversary of the Piedmont Park Conservatory! The nostalgia-themed party will have food from local restaurants, cocktails, a silent auction and other throwback vibes. All proceeds will support the Piedmont Park Conservatory and its ongoing work to maintain and enhance Atlanta\u2019s most visited public park. Find more information about the event here.\r\n\r\n## College Football is Back\r\n\r\nSeptember, of course, heralds the return of college football season. No matter your alma mater, there are bars all around town, which may show your team\u2019s games, which means you can cheer along with like-minded fans. Find out Where to Watch College Football in Atlanta.\r\n\r\n## Gone Fishin\u2019\r\n\r\nSummer is winding down so it\u2019s the perfect time to drop a line and enjoy a day of fishing. There are several places to fish including Jones Creek Park, Lullwater Park, the Chattahoochee River National Recreation Area, Morgan Falls Overlook Park, Sweetwater Creek State Park, Piedmont Park, Murphey Candler Park and Lake Lanier\r\n\r\n## Try an Omakase Restaurant\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647780/newAtlanta.com/Mujo-Andrew-Thomas-Lee-resized/Mujo-Andrew-Thomas-Lee-resized.jpg?_i=AA)\r\nMuj\u00C5\u008D is one of two Atlanta omakase restaurants to receive a MICHELIN star, but there are several other restaurants that offer this unique culinary experience. (Photo of Muj\u00C5\u008D by Andrew Thomas Lee)Omakase is a Japanese dining experience where guests leave the menu in the hands of the chef and receive a seasonal, creative and unforgettable dinner with a progression of small courses. Atlanta has several omakase restaurants, and two \u2014 Muj\u014D and Hayakawa \u2014 were awarded MICHELIN STARS. But Atlanta has several others that offer amazing experiences as well. Try Omakase Table (West Marietta Street); Nagomiya (West Peachtree Street); Saito (Andrew Young International Boulevard); Taka Sushi (Roswell Road); MF Sushi (North Highland Avenue); Chirori (14th Street); Umi (Peachtree Road); and Brush (Peachtree Road). Brush, in fact, just started a \u201Crestaurant-within-a restaurant\u201D concept by offering a chef-driven sushi omakase-only experience in addition to a regular dining experience.\r\n\r\n## News for Kids\r\n\r\nAtlanta\u2019s must-see attractions are stretching your summer fun \u2013 and dollars \u2013 with a September-only promotion for families. Kids free admission this September is available at most attractions and all offer exciting experiences.\u0026nbsp;Plan your Kids Free September Atlanta discovery outing today. All offers start after Labor Day (Sept. 3-30).\r\n\r\n## Yellow Daisy Festival\r\n\r\nVoted one of the top arts and crafts shows in the nation by Sunshine Artist magazine, the Yellow Daisy Festival at Stone Mountain Park draws more than 400 artists and crafters as well as food vendors, crafter demonstrations, live music and a beer garden. The festival, which takes place Sept. 5-8, is great family fun \u2014 and you can get a head start on holiday shopping.\r\n\r\n## Buckhead Arts Festival\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_819,c_scale/v1723647966/newAtlanta.com/Buckhead-arts-festival/Buckhead-arts-festival.jpg?_i=AA)\r\nThe Buckhead Fine Arts Festival is a great way to sip, snack and shop in the heart of Buckhead. (Photo courtesy of Caren West PR.)Sip, shop and stroll in the heart of Buckhead at the Buckhead Fine Arts Festival Sept. 21-22. Presented by the Atlanta Foundation for Public Spaces, the outdoor event will feature more than 100 painters, photographers, sculptors, metalworkers, glass artists, jewelers and more. Enjoy a variety of food and beverage options from vendors and merchants in Buckhead Village.\r\n\r\n## Inaugural Street Festival\r\n\r\nHosted by Blacks in Technology Foundation, the Atlanta Street Food Festival \u0026amp; Bazaar takes place Sept. 2 at Westside Park, Atlanta\u2019s largest park. The first-time festival is a family-friendly event with music, interactive demos and other entertainment. There\u2019s even a dedicated children\u2019s zone with games, activities and entertainment. Come hungry because it is a gastronomic adventure featuring an array of foods from the city\u2019s finest chefs, food trucks and local eateries. There will be everything from barbecue to gourmet burgers to vegan delicacies.\r\n\r\n## Opera in the Yards\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1723647893/newAtlanta.com/pullman-credit-Smily-Eyes-Media/pullman-credit-Smily-Eyes-Media.jpg?_i=AA)\r\nYes, the Atlanta Opera performs at Pullman Yards. (Photo by Smile Eyes Media)The Atlanta Opera Discoveries series continues the \u201CBoh\u00E8me Project\u201D with two immersive productions at Pullman Yards. Produced by\u0026nbsp;Tomer Zvulun, the project comprises repertory performances of a modern-day\u0026nbsp;\u201CLa boh\u00E8me\u201D \u2014 with the COVID-19 pandemic in place of tuberculosis \u2014 with the Broadway show it inspired,\u0026nbsp;\u201CRent,\u201D which updated Puccini\u2019s story of friendship, passion and art by setting it in the midst of the 1990\u2019s HIV/AIDS crisis. Both performed on the same set, with the action taking place in and around the seated audience, the productions will feature different casts, and musical forces and will alternate nights for most of the run.\r\n\r\n## JapanFest\r\n\r\nCelebrate 50 years of culture, friendship and cultural exchange between Japan and Georgia with JapanFest Sept. 21-22 at Gas South Convention Center in Duluth. Enjoy the mesmerizing beauty of traditional Japanese Rakugo, Kabuki, taiko drumming, martial arts demonstration and workshops in bonsai, calligraphy, ikebana flower design and Kimono fitting. Don\u2019t miss the Japanese Food Court where local restaurants will offer bento boxes, rice bowls, ramen noodles, sushi, yahisoba and shaved ice. In addition there will be three traditional Japanese craftsman giving demonstrations in stone carving, Kumihimo braided cords and tatami weaving.\r\n\r\n## PGA\u2019s TOUR Championship Returns\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1685605842/newAtlanta.com/18_TTC_0921_26792_CCp_cc_20604e9ab6/18_TTC_0921_26792_CCp_cc_20604e9ab6.jpg?_i=AA)\r\nFans of professional golf should head to East Lake Golf Club late August through Sept. 1 for the PGA TOUR Championship.East Lake Golf Club hosts the PGA\u2019s TOUR Championship Aug. 29 through Sept. 1.\r\n\r\nAlways keep an eye on our calendar of events for even more things to do in Atlanta.\r\n\r\n![](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/pixel.png)\r\nJournalist Mary Welch writes business and lifestyle stories for local and national publications.\r\n\r\n## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[AttractionsNovember in Atlanta brings popular festivals, a new Cirque du Soleil show and the beginning of holiday things to do. Cool Things to Do in November in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-november-in-atlanta/)[Things to doFrom end of summer festivals to new exhibits at the Atlanta History Center, discover cool things to do in August in Atlanta. Cool Things To Do in August in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-august-in-atlanta/)[Arts \u0026amp; CultureEnjoy all that Atlanta has to offer during the awesome month of April. Cool Things To Do in April in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-april-in-atlanta/)[Arts \u0026amp; CultureIn May, Atlantans head outdoors to enjoy the sun, flowers and the diversity of things to do. Cool Things To Do in May in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-may-in-atlanta/)[Arts \u0026amp; CultureOctober brings lots of cool things to do in Atlanta, and Halloween is just one. Cool Things To Do in October in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)[Arts \u0026amp; CultureWe\u2019ll admit it can get hot in Atlanta during July, so here are some cool things to do. Cool Things To Do in July in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-july-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://www.atlantaonthecheap.com/fall-festivals-atlanta/"}
```
and got this result:
```
"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in Sept., Oct., and Nov. - Atlanta on the Cheap[](https://clicky.com/101416864)\r\n\r\n* Skip to primary navigation\r\n* Skip to secondary navigation\r\n* Skip to main content\r\n* Skip to primary sidebar\r\n* Skip to secondary sidebar\r\n\r\n\r\n\r\n[](http://feeds.feedblitz.com/atlonthecheap\u0026x=1)[](https://app.feedblitz.com/f/?Sub=861848\u0026cids=1)Search this site...Atlanta on the Cheap\r\n\r\nHave fun. Be cheap. LIve the Good Life for Less in Atlanta.\r\n\r\n* Shopping\u0026#038; Daily Deals Groupon Activity Deals Groupon Automotive Deals Groupon Beauty \u0026#038; Spa Deals\r\n* Attractions \u0026#038; Museums\r\n* Entertainment\u0026#038; the Arts\r\n* Food \u0026#038; DrinkDeals Groupon Restaurant Deals Groupon Meal-Prep Deals\r\n* TicketDiscounts\r\n* Outdoor Rec \u0026#038; Sports / Fitness\r\n* Calendar Of Events\r\n* Travel On the Cheap\r\n* Resources \u0026#038; Coping Strategies for COVID-19\r\n* Cheapest Gas Prices\r\n\r\nSearch this site...* Home\r\n* Privacy Policy\r\n* More Cheap Cities\r\n* Submit an Event\r\n* About\r\n* Contact Us\r\n\r\nYou are here:[Home](https://www.atlantaonthecheap.com/)/[Arts \u0026amp; Entertainment](https://www.atlantaonthecheap.com/category/arts-and-entertainment/)/80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in Sept., Oct., and Nov.\u0026larr;[FREE: The Home by Dark concert series happens in Alpharetta this Saturday](https://www.atlantaonthecheap.com/brooke-street-park-free-concerts-alpharetta/)[September in Atlanta: Top things to do this month](https://www.atlantaonthecheap.com/september-things-to-do-atlanta/)\u0026rarr;# 80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in Sept., Oct., and Nov.\r\n\r\n![](https://secure.gravatar.com/avatar/fe99c35123550e6e4fd47ee92160be17?s=32\u0026#038;d=blank\u0026#038;r=g)\r\nPOSTED BYHildaShare this:[](https://www.addtoany.com/add_to/facebook?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/twitter?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/pinterest?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/email?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/tumblr?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/reddit?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/copy_link?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/share)The best fall festivals in Atlanta in 2024 \u0026#8212; including state fairs, county fairs, carnivals, harvest celebrations, arts and music festivals, and more!\u0026nbsp;\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2021/08/Fall-Festivals-GRAPHIC.png)\r\nFall festival season in Atlanta kicks off around Labor Day Weekend, when it still feels like summer, and continues until Thanksgiving \u0026#8212; and then it\u0026#8217;s time for tree and menorah lightings, ice rinks, and holiday fun.\r\n\r\nThis list includes festivals, carnivals, and fairs that are taking place in September, October, and November in 2024. This is the most complete listing you will find!\r\n\r\nThe events take place across the extended metro Atlanta region \u0026#8212; so you can plan a day trip or stick close to home.\r\n\r\nAs always, we emphasize festivals with FREE admission, but there are exceptions. The state and county fairs have an admission charge at the gate, and of course you\u0026#8217;ll pay for rides \u0026#8212; but for each one, we keep a list of ticket discounts and ways to save money.\r\n\r\nNOTE: We are still updating details for 2024, but much of the info is current already!\r\n\r\nWe also have a guide to Atlanta summer festivals and a humongous list of Atlanta spring festivals.\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S.jpg)\r\n\r\n\r\n# Festivals in September\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S.jpg)\r\n\r\n\r\nFrom Labor Day Weekend\u0026#8217;s end-of-summer vibe through the first cooler days of autumn, you\u0026#8217;ll find plenty to do.\r\n\r\nJump to October FestivalsJump to November Festivals\r\n\r\n## \u2666 Dragon Con Parade\r\n\r\nAugust 31, 2024Downtown Atlanta\r\n\r\nThis annual Labor Day Weekend tradition is more of a pop-culture convention than a traditional festival. The parade draws big crowds; it\u0026#8217;s some of the very best people-watching you\u0026#8217;ll find in downtown ATL!Details for the parade\r\n\r\n![Atlanta, GA, USA - September 5, 2015:  Science fiction creatures scare spectators at the annual Dragon Con Parade on Peachtree Street in Atlanta, GA.](https://www.atlantaonthecheap.com/wp-content/uploads/2019/08/dragon-con-stock-photo-54238809_s-e1567117522154.jpg)\r\nDragon Con 2015. Photo: 123RF## \u2666 Atlanta Black Pride Festival\r\n\r\nAug. 27th thru Sept. 2ndMultiple locations\r\n\r\nFREE. This outdoor festival celebrates diversity with live music, food, art, and more. There are also ticketed events related to Black Pride Weekend.Visit the Black Pride website\u0026nbsp;\r\n\r\n## \u2666 Pioneer Days Festival\r\n\r\nAugust 30 to September 2, 2024Sam Smith ParkCartersville\r\n\r\nThis annual event in Bartow County features carnival rides, games, food, and more. It is a Labor Day Weekend tradition, and takes place at Sam Smith Park.Details for Pioneer Days\r\n\r\n## \u2666 Atlanta Hip Hop Day Festival\r\n\r\nAugust 31 to September 1, 2024Historic 4th Ward Park\r\n\r\nThe FREE music festival will celebrate hip hop culture with entertainment, arts \u0026amp; crafts vendors, and food trucks.Details for Hip Hop Day\r\n\r\n## \u2666 Art in the Park on the Marietta Square\r\n\r\nAugust 31 to September 2, 2024Marietta Square\r\n\r\nThis is the 37th edition of the iconic Labor Day weekend event. Shop for fine art, listen to music, and visit the Chalk Art Spot. Free and family-friendly.Details for Art in the Park\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/09/marietta-art-in-the-park-610x226.jpg)\r\n## \u2666 House in the Park Music Festival\r\n\r\nSeptember 1, 2024Grant Park\r\n\r\nThis one day music festival is super-affordable, with general admission tickets just $15 when you purchase in advance online. Spend Labor Day weekend in Grant Park.Details for House in the Park\r\n\r\n## Noshfest\r\n\r\nNo info for 2024\r\n\r\nBack for its tenth year in 2022, Noshfest is a celebration of Jewish food and culture. The festival is open to the entire community, and everyone is welcome. A Labor Day Weekend tradition.Details for Noshfest\r\n\r\n## \u2666 Yellow Daisy Festival\r\n\r\nSeptember 5 to 8, 2024Stone Mountain ParkFREE with paid parking\r\n\r\nThis extensive arts-and-crafts festival features more than 300 vendors. You\u0026#8217;ll also find craft demonstrations, a beer garden, festival food, and a Vintage Village for antiques and collectibles. Note that while admission is free, there is a $20 parking fee at Stone Mountain.Go to the website\r\n\r\n## \u2666 Reynoldstown Wheelbarrow Festival\r\n\r\nSeptember 7, 2024Lang Carson ParkAtlanta\r\n\r\nThis fun community festival features an arts and crafts market, live music, a kids zone, and a cornhole tournament. You\u0026#8217;ll find food and drink vendors on site as well. Located within the charming Reynoldstown neighborhood, just a 5-minute walk from the MARTA Station. The festival is FREE to attend. Details for the Wheelbarrow Festival\r\n\r\n## \u2666 Taste of Smyrna\r\n\r\nSeptember 7, 2024Smyrna Village Green\r\n\r\nCome to sample small tastes from 30 local restaurants. There is also a Beer \u0026amp; Wine Garden, a Kids Zone, and live music on stage.Details for Taste of Smyrna\r\n\r\n## \u2666 Canton Multicultural Festival\r\n\r\nSeptember 7, 2024Harmon Park\r\n\r\nThe 4th annual Multicultural Festival moves to a new location, Harmon Park, in 2024. Come to taste traditional foods and enjoy traditional peformances.Details for the Multicultural Festival\r\n\r\n## \u2666 Gwinnett County Fair\r\n\r\nSeptember 12 to 22, 2024Gwinnett County FairgroundsLawrenceville\r\n\r\nThis annual fair features carnival rides, food, entertainment, garden and livestock exhibits, and more. You\u0026#8217;ll pay for both admission and rides, so it makes sense to use the advance purchase discount and buy online.Details \u0026amp; discounts for the Gwinnett Fair\r\n\r\n## \u2666 Coweta County Fair\r\n\r\nSeptember 12 to 22, 2024Coweta County FairgroundsNewnan\r\n\r\nThis annual fair features carnival rides, food, exhibits, shows, and more. There are daily deals that include FREE admission days and discounts on unlimited rides. There is an advance purchase discount on the fair MEGA Pass.Details \u0026amp; discounts for the Coweta County Fair\r\n\r\n## \u2666 Around the World in the DTL\r\n\r\nSeptember 13 and 14, 2024LawrencevilleFREE\r\n\r\nHead to downtown Lawrenceville for this multi-cultural festival that lets you experience diverse cultures from across the globe \u0026#8212; through food, entertainment, and activities. There is a concert with international food trucks on Friday night, with the main festival on Saturday.Details for Around the World in the DTL\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2022/09/around-the-world-in-the-dtl-e1662585030955.jpg)\r\n## \u0026nbsp;\u2666 West Ponce Music Stroll\r\n\r\nSeptember 14, 2024Downtown DecaturFREE\r\n\r\nHead to downtown Decatur for a day of live music in the patios and parking lots of businesses along West Ponce de Leon Avenue. You\u0026#8217;ll also find sidewalk chart artists, vendor booths, and more.Details for the Music Stroll\r\n\r\n## \u2666 Pup-a-Palooza\r\n\r\nSeptember 14, 2024Newtown ParkJohns Creek\r\n\r\nBring your canine companion to the amphitheater at the park for this fun event. Featuring food, fun, doggy costume contests, and a chance to show off your pup\u0026#8217;s best trick.Details for Pup-a-Palooza\r\n\r\n## \u2666 Taste of Snellville\r\n\r\nSeptember 14, 2024Snellville Towne Green\r\n\r\nAn evening of food, fun, and plenty for the kiddos \u0026#8212; a purchased armband gets them unlimited time on the train ride, rock wall, and bounce houses. Food tasting tickets are $1 each. Free admission to this event.Details for Taste of Snellville\r\n\r\n## \u2666 Sandy Springs Fall Arts Festival\r\n\r\nSeptember 14th \u0026amp; 15th, 20246100 Lake Forrest Dr.FREE\r\n\r\nThis is a new fall edition of the spring Artsapalooza festival. There\u0026#8217;s an emphasis on local and regional arts and crafts, along with gourmet food trucks, live music, interactive art stations, and activities for kids.Details for the Fall Arts Festival\r\n\r\n## \u2666 Stone Mountain\u0026#8217;s Pumpkin Festival\r\n\r\nSeptember 14th to November 3, 2024Stone Mountain Park\r\n\r\nThe theme is \u0026#8220;Play by Day; Glow by Night\u0026#8221; and we recommend being there at dusk for the light-up attractions. Walk thru storybook scenes and enjoy a special Pumpkin LaserShow with fireworks. Discounts available!Details \u0026amp; discounts for Pumpkin Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2020/09/pumpkin-festival-stone-mountain-e1601224154504.jpg)\r\nImage courtesy of Stone Mountain Park## \u2666 Alive in Roswell \u0026#8211; September edition\r\n\r\nSeptember 19, 2024(repeats on October 17th)Historic Downtown Roswell\r\n\r\nThis monthly street festival takes place on third Thursdays, from April through October. Live music, food trucks, vendors, kid activities, and more \u0026#8212; spread out along the historic square area.Details for Alive in Roswell\r\n\r\n## \u2666 North Georgia State Fair\r\n\r\nSeptember 19th to 29th, 2024Jim Miller ParkMarietta\r\n\r\nCarnival rides, festival food, live entertainment, and more!Check out our guide to all the FREE admission days \u0026#8212; for teachers, college students, seniors, first responders, etc. \u0026#8212; and the best deals on unlimited ride wristbands.The Fair is conveniently located in Marietta.Details \u0026amp; discounts for the fair\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2021/04/NGSF-2021-photo-9-1024x683.jpg)\r\nPhoto: N. GA State Fair## \u2666 Atlanta Greek Festival\r\n\r\nSeptember 19 to 22, 2024Greek Orthodox Cathedral of the AnunciationAtlanta\r\n\r\nThis festival is back in its traditional format for 2024! Enjoy delicious Greek cuisine to eat on-site or take home, plus traditional Greek music and dance. You can also tour the majestic Greek Orthodox cathedral and shop for Greek-inspired merchandise. Details \u002B discounts for the Greek Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/greek-festival-2024-557x360.jpg)\r\n## \u25CA Adair Park Porches \u0026amp; Pies Festival\r\n\r\nSeptember 21, 2024Adair ParkAtlanta\r\n\r\nA charming neighborhood festival with plenty of pie \u0026#8212; as well as a beer garden, artist market, kid activities and more. Admission is free, but a pie-tasting pass costs $10.Go to the website\r\n\r\n## \u2666 Urban Regatta\r\n\r\nSeptember 21, 2024Castleberry HillFree to attend\r\n\r\nThe Urban Regatta celebrates the artistic culture in Atlanta\u0026#8217;s Castleberry Hill neighborhood. It includes an artist market, music, pop-up galleries, and the main event \u0026#8212; a regatta-style race using bicycles instead of boats. Details for Urban Regatta\r\n\r\n## \u2666 Chamblee Chalk Walk \u0026amp; Artist Market\r\n\r\nSeptember 21, 2024Chamblee City HallFREE\r\n\r\nCome to see the sidewalks of downtown Chamblee transformed with amazing chalk art. You can try your own hand in the Community Chalk Spot, browse the artist market, eat and drink in the open container district, and have a day of fun.Details for Chalk Walk\r\n\r\n## \u2666 Buckhead Fine Arts Festival\r\n\r\nSeptember 21 and 22, 2024Buckhead Village DistrictFREE\r\n\r\nThis is a two-day arts festival, featuring 100 vendor booths, live music, artist demonstrations, food trucks, and more.Details for the Fine Arts Festival\r\n\r\n## \u2666 Suwanee Fest\r\n\r\nSeptember 21 and 22, 2024Town Center ParkSuwaneeFREE\r\n\r\nThis annual event kicks off with a parade on Saturday morning. There is also a juried art market, kids fun zone, plenty of food, and live entertainment all through the day. Free shuttle from designated parking areas.Details for Suwanee Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/09/suwanee-fest-no-year-e1722286789994-481x360.png)\r\n## \u2666 Shakerag Arts \u0026amp; Crafts Festival\r\n\r\nSeptember 21 and 22, 2024McIntosh Trail Recreation ComplexPeachtree CityFREE\r\n\r\nAn annual event in Peachtree City. This outdoor crafts fair takes place among the pines at Shakerag Knoll. Vendors, food, activities, live music, and more.Details for Shakerag\r\n\r\n## \u2666 Japan Fest\r\n\r\nSeptember 21 and 22, 2024Gas South Convention CenterDuluth\r\n\r\nExperience Japanese culture, food, music, dance, anime, traditions, martial arts and more at this huge exhibition. There are multiple entertainment stages and a dedicated food court. Advance purchase tickets are $16, and you\u0026#8217;ll pay $20 at the door.Details for Japan Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2016/06/japanfest-atl-2017-1024x575.png)\r\n## \u2666 Cumming Art Fest\r\n\r\nSeptember 21 and 22, 2024Cumming City CenterFREE\r\n\r\nThis event is returning for its third year. It is FREE to attend and takes place at the new Cumming City Center complex. You\u0026#8217;ll find art vendors, a Kidz Zone, festival food, and more.Details for Cumming Art Fest\r\n\r\n## \u2666 Roswell Fall Arts Festival\r\n\r\nSeptember 21 and 22, 2024Grounds of Roswell City HallFREE\r\n\r\nFun in historic downtown Roswell! Come for shopping, live music, food trucks, kids\u0026#8217; activities, and plenty of small town charm. Note the change in location, due to construction on the Town Square.Details for Roswell Arts Festival\r\n\r\n## \u2666 Peachtree Corners Festival\r\n\r\nSeptember 21 and 22, 2024The Peachtree Corners Town GreenFREE\r\n\r\nThe 13th annual edition of this event promises two days of fun.\u0026nbsp;You\u0026#8217;ll find live music, vendor booths, car show, and inflatables for the kids.A special Friday night concert kicks off the event. Free parking.Details for the festival\r\n\r\n## \u2666 Taste of Canton\r\n\r\nSeptember 26, 2024The Mill at Etowah FREE admission\r\n\r\nThis annual tasting event allows you to try out the local restaurants through small samples \u0026#8212; including tastes of appetizers, desserts, and entrees. Admission is FREE and tasting tickets are $1 \u0026#8212; note that some items require more than one ticket. There is usually a cash bar on site.Details for Taste of Canton\r\n\r\n## \u2666 Georgia State Fair\r\n\r\nSeptember 27th to October 6, 2024Atlanta Motor SpeedwayHampton, GA\r\n\r\nCome for the carnival rides, food, free concerts, exhibits, and more. The fair is here for about ten days, so you\u0026#8217;ll have two weekend options. There are daily deals on admission and unlimited ride wristbands \u0026#8212; so click through to see our guide to the best value days.Details \u0026amp; discounts for the State Fair\r\n\r\n## \u2666 Blue Stone Arts \u0026amp; Music Festival\r\n\r\nSeptember 27th \u0026amp; 28th, 2024City Springs districtSandy SpringsFREE\r\n\r\nThis is a new festival, featuring an artist market and live music on two stages. There is an extensive concert line-up both on Friday night, and all day Saturday. Plan to park off-site and use the FREE shuttle.Details for Blue Stone Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/09/Blue-Stone-Arts-and-Music-Festival-graphic-no-year-1-610x343.jpg)\r\n## \u2666 East Atlanta Strut\r\n\r\nSeptember 28, 2024East Atlanta VillageFREE\r\n\r\nThis iconic street festival celebrates East Atlanta\u0026#8217;s culture, art, and quirkiness! This is a \u0026#8220;porchfest\u0026#8221; style party that stretches throughout the neighborhood.Details for the Strut\r\n\r\n## \u2666 Collard Greens Cultural Festival\r\n\r\nSeptember 29, 2024Lake Sheryl 4540 Lee Rd SWSnellville, GA 30039\r\n\r\nCome for soul food, music genres from jazz to reggae, local vendors and more. Save room for the collard green ice cream! Visit the festival website\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/07/collard-greens-cultural-festival-e1722343884538-591x360.png)\r\n## \u2666 Alpharetta Art in the Park\r\n\r\nSeptember 28th and 29th, 2024(repeats again in October)Brooke Street ParkAlpharettaFREE\r\n\r\nThis art market places an emphasis on handcrafted goods by local artists and craftspeople. It occurs on the fourth weekend of the month, June thru September.Details for Art in the Park\r\n\r\n## \u2666 Riverfest\r\n\r\nSeptember 28th \u0026amp; 29th, 2024Etowah River ParkCanton\r\n\r\nThis is a fundraiser, so you\u0026#8217;ll pay a $7 admission fee (kids 10 and under are FREE). Come to shop for arts and crafts, enjoy local musicians and performers, eat great food, and browse the scarecrow display.Details for Riverfest\r\n\r\n## \u2666 Atlanta Streets Alive\r\n\r\nSeptember 15, 2024West End to Grant ParkFREE\r\n\r\nThis is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u0026#8212; so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun. It\u0026#8217;s the ultimate \u0026#8220;open streets\u0026#8221; event in metro Atlanta, and it\u0026#8217;s a heckuva good time.Details for Streets Alive\r\n\r\n## \u2666 Sweet Auburn Music Fest\r\n\r\nSeptember 28th and 29th, 2024Sweet Auburn DistrictDowntown AtlantaFREE\r\n\r\nCome the historic Sweet Auburn district for a street festival featuring R\u0026amp;B, hip-hop, gospel music, and more. You\u0026#8217;ll also find great food, a children\u0026#8217;s area, vendors, and more.Details for Sweet Auburn Fest\r\n\r\n## \u2666 Duluth Fall Festival\r\n\r\nSeptember 28th and 29th, 2024Duluth Town GreenFREE\r\n\r\nThis is the 37th year for this beloved fall festival. Two full days of arts/crafts vendors, food trucks, music and dance, a 5K, and a parade. You\u0026#8217;ll find non-stop entertainment on two stages.Details for Duluth Fall Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2017/09/duluth-fall-festival-logo-e1568676416984.jpg)\r\n## \u2666 Sunday in the Park \u0026amp; Tunes from the Tombs\r\n\r\nSeptember 29, 2024Oakland CemeteryAtlanta\r\n\r\nOakland Cemetery\u0026#8217;s beloved fall festival is based on the Victorian tradition of spending a day of leisure in the cemetery. It has always been a free event \u0026#8212; however, in 2023, it\u0026#8217;s being combined with Tunes from the Tombs and is a ticketed event.Details for this event\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/07/tunes-from-the-tombs-no-year-610x320.jpg)\r\n## \u2666 Serb Fest\r\n\r\nSeptember 28th and 29th, 2024Sts. Peter \u0026amp; Paul Serbian Orthodox ChurchLilburn\r\n\r\nA cultural experience like no other in Atlanta! You will experience first-hand Serbian hospitality, culture, history, live entertainment, colorful clothing, and mouth-watering food.Details for Serb Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S.jpg)\r\n\r\n\r\n# Festivals in October\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S.jpg)\r\n\r\n\r\nThe cooler autumn air has finally arrived and it\u0026#8217;s a great time to be outdoors! You\u0026#8217;ll find art markets, carnival rides, harvest celebrations, Oktoberfests, spooky fun, and more.\r\n\r\nJump to September FestivalsJump to November Festivals\r\n\r\n## \u2666 Decatur Wine Festival: Early Bird Ticket Sales\r\n\r\nThe festival is on November 9, 2024 \u0026#8212; but early bird ticket sales close in late October, so this is a reminder to buy early and save $12 in ticket costs and fees!Details for the Wine Festival\r\n\r\n## \u2666 Georgia State Fair\r\n\r\nThrough October 6th, 2024Atlanta Motor SpeedwayHampton, GA\r\n\r\nCome for the carnival rides, food, free concerts, exhibits, and more. The fair is here for ten days, which includes two weekends. There are daily deals on admission and unlimited ride wristbands \u0026#8212; so be sure to click through to see our guide on the best value days.Details \u0026amp; discounts for the State Fair\r\n\r\n## \u2666 Cumming Country Fair \u0026amp; Festival\r\n\r\nOctober 3 to 13, 2024Cumming Fairgrounds\r\n\r\nCarnival rides, the lights of the midway, fair food, live concerts, and daily attractions! If you plan to head to the Fairgrounds, check out our list of discounts and ride deals before you go.Details \u0026amp; discounts for the Cumming Fair\r\n\r\n## Decatur Book Festival\r\n\r\nOctober 4 and 5, 2024Downtown Decatur\r\n\r\nThe annual book festival has scaled down immensely \u0026#8212; and transitioned into a one-day format, with a ticketed keynote event the night before. It\u0026#8217;s now held on the grounds of a local church, with author sessions inside and vendor tents on the grounds.Details for the Book Festival\r\n\r\n## \u25CA Atlanta Fair\r\n\r\nOctober 4 to November 3, 2024Old Turner FieldAtlanta\r\n\r\nThis is the region\u0026#8217;s only urban fair, taking place at Turner Field in downtown Atlanta. There\u0026#8217;s a carnival midway with rides, plenty of food, nightly entertainment, and more. It will be here for all of October.Details for Atlanta Fair\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/03/Atlanta-Fair-graphic-1024x590.jpg)\r\nImage: Atlanta Fair## \u2666 Pumpkin Festival at Stone Mountain\r\n\r\nAll month, through November 3, 2024Stone Mountain Park\r\n\r\nThe theme is \u0026#8220;Play by Day; Glow by Night\u0026#8221; and we recommend being there at dusk for the light-up attractions. Walk thru storybook scenes and enjoy a special Pumpkin LaserShow with fireworks. Discounts available!Deals \u0026amp; discounts for Pumpkin Festival\r\n\r\n## Atlanta Korean Festival\r\n\r\nOctober 5, 2024Korean Community CenterNorcross\r\n\r\nA celebration of Korean and Korean-American culture and heritage! You\u0026#8217;ll find traditional music and cultural performances, martial arts, plenty of food, and family-friendly activities. There is a $10 admission fee for adults, and $5 for kids.Go to the website\r\n\r\n## \u2666 Milton Crabapple Fest\r\n\r\nOctober 5, 2024Downtown MiltonFREE\r\n\r\nMilton\u0026#8217;s annual fall festival takes place the first weekend in October along Crabapple Road. This is an arts and crafts festival, with approximately 100 vendor booths, food, and family activities.Details for Crabapple Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/07/crapabble-fest-milton-no-year-604x360.jpg)\r\n## \u2666 Taste of Tucker\r\n\r\nOctober 5, 2024Main StreetDowntown Tucker\r\n\r\nCome hungry and try out a wide variety of tasting samples, including appetizers, entrees, and desserts. There are also art vendors, a beer garden, and a Kids Zone for the young ones.Details for Taste of Tucker\r\n\r\n## \u2666 Suwanee Taste of BBQ \u0026amp; Music Festival\r\n\r\nOctober 5, 2024Town Center ParkSuwaneeFREE admission\r\n\r\nThis all-day music festival takes place in downtown Suwanee. Admission, parking, and entertainment is free. The BBQ tasting tickets s are $1 each, and each taste requires one to four tickets. You can also buy a full plated barbecue meal. There is also a kids fun area and an artist market.Details for this event\r\n\r\n## \u25CA Candler Park Fall Fest\r\n\r\nOctober 5 and 6, 2024Candler ParkAtlantaFREE\r\n\r\nThis popular street festival includes an artist market, live acoustic music on two stages, rides and special attractions, a tour of homes, food, kids activities, and more.Details for Fall Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2018/09/candler-park-street-signs-e1569941757312.jpg)\r\n## \u2666 Norcross Art Splash\r\n\r\nOctober 5 and 6, 2024Lillian Webb ParkDowntown NorcrossFREE\r\n\r\nThis annual fall festival features 90\u002B arts and crafts vendors, plenty of food, and an interactive Kid Zone with face painting, inflatables, and more. Located in the historic downtown area.Details for Art Splash\r\n\r\n## Prost on Perry\r\n\r\nNo info for 2024\r\n\r\nThis traditional Oktoberfest celebration takes place in downtown Lawrenceville, on the town square. Come for beer, food, live entertainment, and a good time. Family-friendly.Details for Prost on Perry\r\n\r\n## Fall Festival on Ponce\r\n\r\nOctober 5 and 6, 2024Olmsted Linear ParksAtlantaFREE\r\n\r\nThis intown festival features vendor booths of arts and crafts, a children\u0026#8217;s play area, local food and beverage vendors, and live acoustic music. It is spread across three of the Olmsted Linear Parks.Details for Festival on Ponce\r\n\r\n## \u25CA Conyers Latin Festival\r\n\r\nOctober 6, 2024Olde Town Conyers FREE\r\n\r\nThis annual celebration observes the history, culture, and contributions of those whose ancestors came from Spain, Mexico, the Caribbean, and Central and South America. You\u0026#8217;ll find food, an arts \u0026amp; crafts marketplace, live entertainment, a Kid Zone, and more. Visit the festival website\r\n\r\n## \u2666 Wire \u0026amp; Wood Music Festival\r\n\r\nOctober 11 and 12, 2024Downtown AlpharettaFREE\r\n\r\nThis FREE music fest features more than 30 performances by singer-songwriters on six outdoor stages located throughout the Downtown Alpharetta area. An Alpharetta cultural tradition since 2013.Details for Wire \u0026amp; Wood\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/wire-and-wood-2024-610x226.png)\r\n## \u2666 Taste of Acworth\r\n\r\nOctober 12, 2024Downtown Acworth\r\n\r\nThis affordable food festival features FREE admission, and small bites for just $1 to $5 each. It takes place along historic Main Street and the surrounding area.Details for Taste of Acworth\r\n\r\n## \u2666 Lilburn Daze\r\n\r\nOctober 12, 2024Lilburn City ParkFREE\r\n\r\nThis beloved fall arts \u0026amp; crafts is in its 50th year! It is extremely family-friendly, with plenty of activities for the kids. You\u0026#8217;ll also find food, vendor booths, and live entertainment all day long. Plan to use the free shuttle.Details for Lilburn Daze\r\n\r\n## \u2666 The Great Pumpkin Fest\r\n\r\nOctober 12, 2024Cherokee Veterans ParkCantonFREE\r\n\r\nThis annual fall and Halloween festival takes place in Cherokee County. You\u0026#8217;ll find craft vendors, food, a DJ, hay rides, games, bounce houses for the kids, trick-or-treating, and more.Details for Pumpkin Fest\r\n\r\n## Old Fourth Ward Park Arts Festival\r\n\r\nOctober 12 and 13, 2024Old 4th Ward ParkFREE\r\n\r\nThis two-day arts festival features live acoustic music, about 150 vendor booths, food, and fall fun \u0026#8212; located at historic Old 4th Ward Park.Details for the O4W Arts Festival\r\n\r\n## Chalktoberfest\r\n\r\nOctober 12 and 13, 2024Marietta SquareFREE\r\n\r\nIt\u0026#8217;s a festival that celebrates street art and craft beer! Enjoy a fall afternoon at Chalktoberfest, in the Marietta Square. Enjoy live music and eat great food, while you watch the work of chalk artists unfold in front of your eyes. FREE to attend, but the craft beer tasting is a ticketed event.Details for Chalktoberfest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2016/10/chalktoberfest-from-stock-37714032_s-e1570630137637.jpg)\r\nMarietta Chalktoberfest 2014 \u0026#8211; Image: 123RF## Atlanta Pride Festival \u0026amp; Parade\r\n\r\nOctober 12 and 13, 2024Piedmont Park\r\n\r\nAn annual October event! Atlanta Pride features a festival and parade in Midtown, plus a wide variety of associated events around town. Details for Atlanta Pride\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2018/10/pride-hands-forming-heart-43060357_s-e1570548745417.jpg)\r\nDeposit Photos## \u2666 Georgia Apple Festival\r\n\r\nOctober 12 and 13, 2024October 19 and 20, 2024Ellijay, GA\r\n\r\nThis is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!Details for the Apple Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/10/Apple-picking-114688623_s-e1570031034124.jpg)\r\nDeposit Photos## \u2666 Alive in Roswell Street Festival\r\n\r\nOctober 17, 2024Historic Downtown RoswellFREE\r\n\r\nThis monthly street festival takes place on third Thursdays, from April through October. Live music, food trucks, vendors, kid activities, and more \u0026#8212; spread out along the historic square area.Details for Alive in Roswell\r\n\r\n## \u2666 Halloween \u0026amp; Spooky Fun\r\n\r\n\u0026#8216;Tis the season to be spooky! Check out our huge list of celebrations for Halloween and Dia de los Muertos \u0026#8212; including festivals, ghost tours, trick-or-treat events, and more. The events start in mid-October and continue through October 31st. View our huge list of Halloween events\r\n\r\n![20948527 - monster movies with a zombie hand holding a cinema or theater ticket stub as a creepy halloween or scary entertainment symbol ](https://www.atlantaonthecheap.com/wp-content/uploads/2019/10/halloween-green-hand-holding-ticket-20948527_s-e1570221116120.jpg)\r\nDeposit Photos## Cumming Greek Festival\r\n\r\nOctober 18 to 20, 2024Saints Raphael and Nicholas Greek Orthodox ChurchCumming\r\n\r\nBe Greek for a day! Come out for authentic Greek cuisine, a vendor market, and continuous live entertainment and cultural performances. You\u0026#8217;ll also have a chance to tour the Greek Orthodox church.Details for the Greek Festival\r\n\r\n## Fall-0-Ween Fest\r\n\r\nOctober 19, 2024Swift-Cantrell ParkKennesawFREE\r\n\r\nThis event includes a candy trail, costume contests, scavenger hunts, games, crafts, music, fooc, and more. It\u0026#8217;s followed by a Halloween movie at sunset.Details for Fall-O-Ween\r\n\r\n## Oakhurst Porchfest\r\n\r\nOctober 19, 2024Oakhurst NeighborhoodAtlantaFREE\r\n\r\nThis grassroots music festival turns front porches into stages and yards into venues. You can wander at leisure through one square mile of the Oakhurst neighborhood, taking in mini-concerts. In past years, the event has featured over 200 performances.Details for Porchfest\r\n\r\n## Marietta HarvestFest\r\n\r\nOctober 19, 2024Marietta SquareFREE\r\n\r\nThis annual October event includes an arts/crafts market, scarecrow display, touch-a-truck event, and a special Halloween Happenings festival for kids 12 and under \u0026#8212; complete with trick-or-treating and a costume contest. In Glover Park on the Marietta Square.Details for Marietta HarvestFest\r\n\r\n## Snellville Fall Festival\r\n\r\nOctober 19, 2024Snellville Town GreenFREE\r\n\r\nFeaturing a Halloween costume contest for kids, adults, and dogs! You\u0026#8217;ll also find vendor booths, food, a concert, and plenty of activities for kids (an activity wristband is $15).Details for the Fall Festival\r\n\r\n## Red Oak Art Festival\r\n\r\nNo info for 2024 yet \u0026#8212; stay tuned!New Life AME Zion ChurchCity of South FultonFREE\r\n\r\nThis FREE and family-friendly event is all about the arts! It features interactive, community art projects, mini theater workshops, an instrument petting zoo, local food trucks, and more.Details for the festival\r\n\r\n## \u2666 Loganville Autumn Fest\r\n\r\nOctober 19, 2024Loganville Town GreenFREE\r\n\r\nA classic fall festival on the Loganville Towne Green. You\u0026#8217;ll find arts vendors, food, live entertainment, and plenty of stuff to do for the kiddos.Details for Autumn Fest\r\n\r\n## Woodstock International Food Festival\r\n\r\nNo 2024 dates yet \u0026#8212; stay tuned!St. Elizabeth Orthodox Church\r\n\r\nThis festival began in 2018 as a way to honor the diversity of the community. Besides ethnic cuisines from all corners of the world, there will be live music, dancing, church tours, entertainment, and children\u0026#8217;s activities.Details for the Food Festival\r\n\r\n## Spirits \u0026amp; Spice Festival\r\n\r\nOctober 19, 2024Downtown NewnanFREE\r\n\r\nEat, drink, and be spooky at this one-day street festival in charming downtown Newnan. Enjoy a chili cook-off and themed cocktails, plus live music, and activities for the kids.Details for Spirits \u0026amp; Spice\r\n\r\n## Olde Town Fall Festival\r\n\r\nOctober 19, 2024Olde Town ConyersFREE\r\n\r\nThis annual event in Conyers features a full day of arts and craft vendors, festival food, live entertainment, and kid\u0026#8217;s activities. You\u0026#8217;re invited to come prepared to enter a costume contest!Details for Fall Festival\r\n\r\n## Brookhaven International Festival\r\n\r\nOctober 19 and 20, 2024Peachtree Creek GreenwayFREE\r\n\r\nExpanded to two days in 2024! This event features live multi-cultural entertainment, international cuisine, a Kids Zone, interactive art stations, and more.Details for Brookhaven International Festival\r\n\r\n## Alpharetta PopFest\r\n\r\nNo 2024 date yet \u0026#8212; stay tuned!Brooke Street ParkAlpharetta\r\n\r\nA one-day outdoor music festival in support of Alzheimer research. Come for food, drink, and live music.Details for PopFest\r\n\r\n## Fall Festival in McDonough\r\n\r\nNo 2024 date yet \u0026#8212; stay tuned!87 Griffin St(green space across from the church)\r\n\r\nThis is a sensory-friendly fall festival for the community \u0026#8212; featuring face painting, carnival games, trunk-or-treating, and more. Hosted by First Baptist McDonough.Details for Fall Festival\r\n\r\n## Johns Creek Arts Festival\r\n\r\nOctober 19 and 20, 2024Atlanta Athletic ClubJohns CreekFREE\r\n\r\nThis art market features over 100 arts vendors from around the country, so come ready to shop. You\u0026#8217;ll also find festival food, a ticketed Kid Zone, and continuous live concerts by local musical performers.Details for this event\r\n\r\n## Romanian Festival\r\n\r\nNo 2024 dates yet \u0026#8212; stay tuned!Saints Constantine and Elena Romanian Orthodox ChurchLilburn\r\n\r\nCelebrate Romanian culture with delicious food, traditional music and dance, and more.Details for Romanian Festival\r\n\r\n## The Highland Games \u0026amp; Scottish Festival\r\n\r\nOctober 19 and 20, 2024Stone Mountain Park\r\n\r\nStone Mountain plays host to pipers, drummers, dancers, Scottish athletic competitions, traditional food, clan tents, vendors selling imported items, and more. A fun and educational cultural experience! This is a ticketed event. The gate price is $25 and there is a small discount for buying online in advance. Go to the festival website\r\n\r\n## \u2666 Hogansville Hummingbird Festival\r\n\r\nOctober 19 and 20, 2024Main StreetHogansvilleFREE\r\n\r\nA charming, small-town street fest in historic downtown Hogansville, which is about 30 minutes south of the ATL Airport. Vendor booths, music, food, fun.Details for Hummingbird Festival\r\n\r\n## \u2666 Magnolia Arts Festival\r\n\r\nOctober 19 and 20, 2024Drake FieldPeachtree CityFREE\r\n\r\nThis annual Peachtree City event includes a juried art sale, student art exhibit, food trucks, kids activities, live entertainment, and plenty of fall fun. This is the 7th year for this fall festival on the south side of town.Details for Magnolia Arts Festival\r\n\r\n## LakeFest\r\n\r\nOctober 19 and 20, 2024Lake Shore DrivePine Lake, GAFREE\r\n\r\nPine Lake may be the smallest city in Dekalb County, but it hosts a huge festival each fall. You\u0026#8217;ll find live music, food, artist booths, and Floatzilla \u0026#8212; a floating parade.Go to the website\r\n\r\n## Little 5 Points Halloween Festival \u0026amp; Parade\r\n\r\nOctober 19 and 20, 2024FREE\r\n\r\nCelebrate the spooky, the quirky, and the bizarre in L5P this Halloween! The parade is back with a new route along Moreland Ave. You\u0026#8217;ll also find vendors, food, booze, live music, Drag Queen Karaoke, and more.Details for L5P Halloween Festival\r\n\r\n## \u2666 Atlanta Streets Alive\r\n\r\nOctober 20, 2024West End to Grant ParkFREE\r\n\r\nThis is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u0026#8212; so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun. It\u0026#8217;s the ultimate \u0026#8220;open streets\u0026#8221; event in metro Atlanta, and it\u0026#8217;s a heckuva good time.Details for Streets Alive\r\n\r\n## \u25CA Decatur BBQ \u0026amp; Bands Festival\r\n\r\nOctober 26, 2024Oakhurst VillageFREE\r\n\r\nLocal BBQ restaurants present a variety of options, including vegan and gluten-free. Deserts, frozen treats, and adult beverages are also available to purchase. Live music from noon to 4 p.m. Admission is FREE; just bring your wallet for food and drink.Go to the website\r\n\r\n## \u2666 Alpharetta Art in the Park\r\n\r\nOctober 26th and 27th, 2024Brooke Street ParkAlpharettaFREE\r\n\r\nThis art market places an emphasis on handcrafted goods by local artists and craftspeople. It occurs on the fourth weekend of the month, June thru September.Details for Art in the Park\r\n\r\n## Smyrna Crafts \u0026amp; Drafts Festival\r\n\r\nOctober 26 and 27, 2024Smyrna Town GreenFREE\r\n\r\nFormerly known as the Fall Jonquil Festival, this newly re-branded fall is in its second year. It features 150\u002B arts and crafts vendors, food, live entertainment, and activities for the kids.Details for Crafts \u0026amp; Drafts\r\n\r\n## \u2666 Johns Creek Diwali Festival\r\n\r\nOctober 26 and 27, 2024Johns Creek City HallFREE\r\n\r\nCelebrate Diwali, the traditional Hindu festival of light! You\u0026#8217;ll find retail and food vendors, live entertainment, and a drone light show. The symbolic meaning of Diwali is the triumph of light over dark, good over evil, and knowledge over ignorance.Details for Diwali Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/diwali-601x360.jpg)\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S.jpg)\r\n\r\n\r\n# Festivals in November\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S.jpg)\r\n\r\n\r\nFestival season is winding down, but there are still plenty of opportunities to enjoy the cool autumn air \u0026#8212; and perhaps get a headstart on holiday gift shopping.\r\n\r\nJump to October FestivalsJump to September Festivals\r\n\r\n## Atlanta Fair\r\n\r\nThrough November 6thOld Turner FieldDowntown Atlanta\r\n\r\nThis is Atlanta\u0026#8217;s only urban fair, taking place in the lots outside the Old Turner Field. There\u0026#8217;s a carnival midway with rides, plenty of food, and more.Details for Atlanta Fair\r\n\r\n## Dia de los Muertos Festival\r\n\r\nNovember 2, 2024Lawrenceville Arts CenterFREE\r\n\r\nHave a cultural experience at this traditional Day of the Dead festival. Enjoy food, traditional music, and ceremonial traditions.Details for Dia de los Muertos\r\n\r\n## The Rescue Dog Games\r\n\r\nNovember 2, 2024\r\n\r\nThis is a fun festival for dogs and their humans in Piedmont Park. You\u0026#8217;ll find interactive games, pet-themed vendors, food trucks, photo booths, and a Howl-o-Ween doggy costume contest.Details for Rescue Dog Games\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/rescue-dog-games-2024-graphic-610x343.png)\r\n## Taste of Kennesaw\r\n\r\nNovember 2, 2024Kennesaw Depot Park\r\n\r\nThis one-day food festival allows you to sample food from local restaurants. The small plates range from $1 to $4 per taste. There\u0026#8217;s also a kids area with inflatables, a beer garden, and live music on two states.Details for Taste of Kennesaw\r\n\r\n## Chomp \u0026amp; Stomp Chili Cook-off and Bluegrass Festival\r\n\r\nNovember 2, 2024Cabbagetown neighborhoodAtlantaFREE\r\n\r\nThis one-day event is a fall favorite in the charming Cabbagetown neighborhood of Atlanta. You\u0026#8217;ll be able to taste chili samples, shop the artist market, and listen to bluegrass on five stages.Details for Chomp \u0026amp; Stomp\r\n\r\n## Suwanee Wine Festival\r\n\r\nNovember 2, 2024Suwanee Town Center Park\r\n\r\nSip and stroll your way around the park, enjoying samples of 150 wines from all over the world \u0026#8212; plus a Georgia winery section. There is also live music, art vendors, food trucks, a beer tent, and football on a big screen. You can save a few bucks with early-bird tickets, which usually go on sale in late summer.Go to the festival website\r\n\r\n## Chastain Park Fall Arts Festival\r\n\r\nNovember 2 and 3, 2024Chastain ParkAtlantaFREE\r\n\r\nThis is the fall edition of this popular festival, after a successful run in the spring. You can start your holiday shopping at the art market, plus enjoy the children\u0026#8217;s area, local gourmet food and drink, and acoustic musical performances.Details for Fall Arts Festival\r\n\r\n## Dia De Los Muertos \u0026#8212; Day of the Dead Festival in East Point\r\n\r\nNovember 4, 2024 \u0026#8212; not yet confirmed East Point City HallFREE\r\n\r\nA traditional Day of the Dead celebration with food, dance, games, face painting and more.Details for Day of the Dead\r\n\r\n## Lawrenceville Harvest Festival\r\n\r\nNovember 9, 2024The Lawrenceville LawnFREE\r\n\r\nBack for its third year! The fall fun includes hayrides around the Lawrenceville Square, a petting zoo, four live country music acts, food, vendors, and more.Details for Harvest Festival\r\n\r\n## \u2666 Decatur Wine Festival\r\n\r\nNovember 9, 2024Old Courthouse SquareDowntown Decatur\r\n\r\nTickets are not cheap for this fun event, so you\u0026#8217;ll want to take advantage of early-bird pricing, and buy your ticket in advance. Details for the Wine Festival\r\n\r\n## Suwanee Chili Cook-Off and Music Festival\r\n\r\nNovember 9, 2024Suwanee Town CenterFREE\r\n\r\nListen to live music, play in the Kids Zone, shop with arts and crafts vendors, and taste lots of great chili before you vote for your favorite. FREE to attend, but chili sampling requires a $10 wristband.Details for the Cook-Off\r\n\r\n## Stone Mountain Christmas\r\n\r\nNovember 9, 2024 through January 5, 2025Stone Mountain Park\r\n\r\nVisit Stone Mountain for this festival of holiday magic, dazzling lights, a nightly parade, Santa\u0026#8217;s flight through the sky, and spectacular shows. The Christmas Village is open on weekends and during the school holiday break. Click through for available discounts!Details \u0026amp; discounts for Stone Mountain Christmas\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/10/stone-mountain-christmas-large-flyer-1024x1024.jpg)\r\n## \u2666 Atlanta Chili Cook Off\r\n\r\nNovember 16, 2024Brook Run ParkDunwoody\r\n\r\nThis ticketed food festival promises a day of fun. Come to taste chili from 50\u002B amateur cooks, restaurants, and caterers, while listening to a DJ spin your favorite tunes. Details \u0026amp; ticket discounts for Chili Cook Off\r\n\r\n## \u2666 Atlanta Streets Alive\r\n\r\nNovember 17, 2024West End to Grant ParkFREE\r\n\r\nThis is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u0026#8212; so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun. It\u0026#8217;s the ultimate \u0026#8220;open streets\u0026#8221; event in metro Atlanta, and it\u0026#8217;s a heckuva good time.Details for Streets Alive\r\n\r\n## Atlanta Christkindl Market\r\n\r\nNovember 22nd through December 24, 2024Buckhead VillageFREE\r\n\r\nThis German-style street festival and holiday market is one of our favorite events. Enjoy authentic German food and shop your way through the charming vendor booths, styled as Bavarian chalets.Details for Christkindl Market\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/11/christkindl-market-1-e1574206875851.jpg)\r\n## Festival of Trees\r\n\r\nNovember 22nd thru December 1st, 2024(closed Thanksgiving Thursday \u002B Friday)Georgia World Congress CenterDowntown Atlanta\r\n\r\nThe Georgia Festival of Trees is a special five-day holiday event for all ages. Wander through a beautiful forest of dazzling holiday trees, shop for unique gift shops, and enjoy treats from local vendors.Details \u0026amp; discounts for Festival of Trees\r\n\r\n## \u2666 HOLIDAY FUN\r\n\r\nIt\u0026#8217;s late November and festival season is over \u0026#8212; but the fun doesn\u0026#8217;t stop here!\r\n\r\nStay with us for tons of holiday fun, including:\r\n\r\n* Spectacular holiday light displays\r\n* FREE photos with Santa (60\u002B opportunities)\r\n* Huge list of seasonal holiday \u0026amp; winter events\r\n* 40 great ways to celebrate the Winter Solstice\r\n\r\nOur big list of holiday fun includes tree \u0026amp; menorah lightings, skating rinks, live nativity scenes, parades, holiday markets, snow play areas, Hanukkah celebrations, Kwanzaa events, and more!FIND YOUR HOLIDAY FUN\r\n\r\n\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S-610x51.jpg)\r\n# Fall Festivals\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S-610x51.jpg)\r\n\r\n\r\n\r\n\r\nShare this:[](https://www.addtoany.com/add_to/facebook?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/twitter?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/pinterest?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/email?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/tumblr?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/reddit?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/copy_link?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/share)Posted by Hilda on September 14, 2024 | Updated September 21, 2024 Filed Under: Arts \u0026amp; Entertainment\u0026nbsp;\u0026middot; Featured\u0026nbsp;\u0026middot; Festivals Tagged With: Fall Fun\u0026middot; Roundups\r\n\r\n## Reader Interactions\r\n\r\n### Trackbacks\r\n\r\n1. The Best Free Things to Do in Atlanta in 2024 - says: August 20, 2024 at 3:57 pm [\u0026#8230;] Castleberry Hill Historic District Located just southwest of the city and loosely bounded by McDonald Street, Peters Street, Walker Street, and Northside Drive, the Castleberry Hill Historic District lies just across the railroad tracks from both the Georgia World Congress Center and Mercedes-Benz Stadium. Formerly an industrial hub of warehouses and factories, the district\u0026#8217;s history dates back to the late 19th century when it was an ideal site for railroad operations and warehouse storage. Today, Castleberry Hill has transformed into an art-focused district, where the old warehouses have been repurposed as art galleries, studios, boutiques, and creative spaces. Once a month, the district comes alive with art walks and other events, attracting art lovers from across Atlanta. [19]. [\u0026#8230;]\r\n\r\n## Primary Sidebar\r\n\r\n### Stay in the know!\r\n\r\nDo you suffer from FOMO, that pervasive fear of missing out? We can help.\r\n\r\nSign up for email updates from ATLANTA ON THE CHEAP and never miss a great deal or free event.\r\n\r\n### old google site search bar\r\n\r\n## Secondary Sidebar\r\n\r\n### Local Groupon Deals\r\n\r\n[](https://www.tkqlhce.com/click-100651436-15245379)### Groupon Experiences\r\n\r\n[](https://www.tkqlhce.com/click-100635924-15245432?sid=ATL)### See something you like?\r\n\r\nPlease tell your friends about it! We hope you\u0026#8217;ll like, share, follow, and spread the word about Atlanta On The Cheap!\r\n\r\n\r\n\r\n### Like. Follow.\r\n\r\n[](https://www.facebook.com/atlantaonthecheap)[](https://twitter.com/ATLcheap)### We use stock images by our partner, Deposit Photos!\r\n\r\n\u0026nbsp;\r\n\r\n\r\n\r\n**www.depositphotos.com**\u0026nbsp;\r\n\r\n### Member of the Living on the Cheap Network\r\n\r\n[](http://livingonthecheap.com/our-network/)Atlanta on the Cheap is a member of Living on the Cheap, a network of websites published by journalists and consumer advocates.[](https://livingonthecheap.com/)* Back to Top\r\n* Half-Off Deals\r\n* Free\r\n* Kids \u0026#038; Family\r\n* Events\r\n* Attractions\r\n* Concerts\r\n* Theater\r\n* Sports\r\n* Terms of Use and Privacy Policy\r\n\r\n\u0026#x000A9;\u0026nbsp;2024 Atlanta on the Cheap. Living On The Cheap\u0026reg; and On the Cheap\u0026reg; are registered trademarks owned by Living On The Cheap Inc. Site design by Five J\u0027s Design.All the stories on our site are protected by copyright and any unauthorized copying is strictly prohibited. Plus, it is just not nice.\r\n\r\n"
```

Here are some exciting things to do in Atlanta for each of the next four weekends:

### Weekend of September 28-29, 2024:
1. **Sweet Auburn Music Fest**: Located on Auburn Avenue, offering live music, international food, and vendors of all types. [Source](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/).
2. **Forward Warrior**: An all-day live art event on Wylie Street in Cabbagetown. [Source](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/).
3. **Baldie Con**: A conference celebrating and empowering women and children who are bald, held at JW Marriott Atlanta Buckhead. [Source](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/).

![Sweet Auburn Music Fest](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg)

### Weekend of October 5-6, 2024:
1. **Festival on Ponce**: An arts festival in Olmsted Linear Park featuring over 125 vendors, food and drink, and children’s activities. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
2. **Candler Park Fall Fest**: A popular annual festival with live music, art, food, rides, and games. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
3. **Atlanta Arab Festival**: Celebrating Arabic food and culture at the Alif Institute. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

![Festival on Ponce](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)

### Weekend of October 12-13, 2024:
1. **Atlanta Pride Festival**: One of the largest LGBTQ+ pride events in the Southeast, held in Piedmont Park. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
2. **Old Fourth Ward Fall Arts Festival**: Featuring local artists, demonstrations, live entertainment, and food. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
3. **Chalktoberfest**: An art and beer festival in Marietta Square with professional chalk artists and craft beers. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

![Atlanta Pride Festival](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)

### Weekend of October 19-20, 2024:
1. **Stone Mountain Highland Games and Scottish Festival**: Celebrating Scottish heritage at Stone Mountain Park. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
2. **L5P Halloween Festival and Parade**: A Halloween celebration in Little Five Points with live music, parade, and haunted house. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
3. **Boo at the Zoo**: Halloween-themed activities at Zoo Atlanta. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

![Stone Mountain Highland Games](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)

For more details and to plan your visit, refer to the respective sources linked above. Enjoy your weekends in Atlanta!


In [7]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output2.LastMessage.DisplayAs("text/markdown");


Here are some exciting things to do in Atlanta for each of the next four weekends:

### Weekend of September 28-29, 2024:
1. **Sweet Auburn Music Fest**: Located on Auburn Avenue, offering live music, international food, and vendors of all types. [Source](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/).
2. **Forward Warrior**: An all-day live art event on Wylie Street in Cabbagetown. [Source](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/).
3. **Baldie Con**: A conference celebrating and empowering women and children who are bald, held at JW Marriott Atlanta Buckhead. [Source](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/).

![Sweet Auburn Music Fest](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg)

### Weekend of October 5-6, 2024:
1. **Festival on Ponce**: An arts festival in Olmsted Linear Park featuring over 125 vendors, food and drink, and children’s activities. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
2. **Candler Park Fall Fest**: A popular annual festival with live music, art, food, rides, and games. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
3. **Atlanta Arab Festival**: Celebrating Arabic food and culture at the Alif Institute. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

![Festival on Ponce](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)

### Weekend of October 12-13, 2024:
1. **Atlanta Pride Festival**: One of the largest LGBTQ+ pride events in the Southeast, held in Piedmont Park. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
2. **Old Fourth Ward Fall Arts Festival**: Featuring local artists, demonstrations, live entertainment, and food. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
3. **Chalktoberfest**: An art and beer festival in Marietta Square with professional chalk artists and craft beers. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

![Atlanta Pride Festival](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)

### Weekend of October 19-20, 2024:
1. **Stone Mountain Highland Games and Scottish Festival**: Celebrating Scottish heritage at Stone Mountain Park. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
2. **L5P Halloween Festival and Parade**: A Halloween celebration in Little Five Points with live music, parade, and haunted house. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).
3. **Boo at the Zoo**: Halloween-themed activities at Zoo Atlanta. [Source](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

![Stone Mountain Highland Games](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)

For more details and to plan your visit, refer to the respective sources linked above. Enjoy your weekends in Atlanta!

## Clean Up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search and Scrape");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Web Search and Scrape");
}

Deleted assistant
